# KHAI BÁO TÊN CÁC FILE DỮ LIỆU ĐẦU VÀO

In [1]:
# ngày báo cáo là thứ hai thì lấy số liệu ngày thứ 6 tuần trước,
# 1. cập nhập file bcqt 20 vào file info
# 2. down file cib sme ngày báo cáo
# 3. cập nhập tk 43 44 vào file info
# 4. down file gl101 ngày báo cáo 
# 5. cập nhập file ủy thác phát sinh
# 6. cập nhập file kế hoạch vào file info (1 lần/tháng)
# 7. cập nhập file thực hiện vào file info (1 lần/tháng)
info_filename=r"D:\Bao Cao Ngay - Data\info.xlsx" #1
TraiPhieuPath=r'C:\Users\phatdd3\Downloads\File-trái-phiếu-ghi-nhận-tay-2023-update-18.08.2023.xlsx' # cập nhập file trái phiếu

# XÁC ĐỊNH NGÀY BÁO CÁO CỦA FILE BÁO CÁO VÀ TÊN FILE BÁO CÁO NGÀY

In [2]:
import pandas as pd
import numpy as np
import datetime
import openpyxl
billion=1000000000

# File Báo Cáo ngày sheet KM chứa data của file BCQT 20 xuất từ MIS
q=pd.read_excel(info_filename,sheet_name='KM')

#Lấy dữ liệu của ngày báo cáo trong file
NgayBaoCao=q.iloc[0,0].strftime("%d/%m/%Y")
ThangBaoCao=q.iloc[0,0].strftime("%#m")

# Chuyển đổi NgayBaoCao từ stirng sang datetime object
NgayBaoCao_dt = datetime.datetime.strptime(NgayBaoCao, "%d/%m/%Y")
date_str=NgayBaoCao_dt.strftime("%d.%m.%Y")
# Tên file mới theo ngày báo cáo
new_filename = f'Báo cáo ngày {date_str}.xlsx'

# tính số cho vay của Muadee
CVMuadee=q[q.CN=="'668"]['CVNGAYBC'].iloc[0]/billion
BK2=CVMuadee

# QUY ĐỔI BẢNG VỀ ĐƠN VỊ TỶ ĐỒNG:
# chọn những cột có chứa dữ liệu dạng số
numeric_cols=q.select_dtypes(include=['int64','float64']).columns
# chia những giá trị của cột này với 1 tỷ (ngoại trừ cột CN_GOC)
q.CN_GOC=q.CN_GOC*billion
q[numeric_cols]=q[numeric_cols]/billion
# Chuyển các cột dữ liệu thành dạng số thập phân
q.iloc[:,4:len(q.columns)+1]=q.iloc[:,4:len(q.columns)+1].astype(float)
# fill NaN values with zero in-place
q.fillna(value=0, inplace=True)
# bỏ các cột dữ liệu ko cần thiết
q=q.drop(['NGAY','CN_GOC','TENCN'],axis=1)

print(f'Ngày số liệu báo cáo của huy động trên KM là: {date_str}')

Ngày số liệu báo cáo của huy động trên KM là: 09.06.2023


In [3]:
# SỐ LIỆU TỔNG TÀI SẢN CỦA TOÀN HÀNG THEO FILE B02
# Đọc file B02
B02_data=pd.read_excel(info_filename,sheet_name='B02')
# Lưu số liệu tổng tài sản mới của file B02 và Chuyển đơn vị từ đồng thành tỉ đồng
TongTaiSanCapNhap=B02_data.iloc[94,3]/billion #([94,3] hoặc [47,3])
TongTaiSanCapNhap

486050.23434692

In [4]:
import datetime
import calendar

# Assuming NgayBaoCao_dt is a datetime object representing the current date

# Calculate the previous month
previous_month = NgayBaoCao_dt.replace(day=1) - datetime.timedelta(days=1)

# Get the last day of the previous month
last_day = calendar.monthrange(previous_month.year, previous_month.month)[1]

# Set the final date of the month for the last_month
last_month = previous_month.replace(day=last_day)

# Format the last_month as a string in the format "%d%m%y"
last_month_str = last_month.strftime("%d.%m.%Y")
month_of_last_month = last_month.strftime("%m.%Y")
print(f'Số liệu thực hiện tháng trước là: {last_month_str}')

Số liệu thực hiện tháng trước là: 31.05.2023


# A. CẬP NHẬP HUY ĐỘNG

In [5]:
# bỏ các cột tổng huy động
q=q.drop(['HTHANGTRC','HNGAYTRC','HNGAYBC','HSME','HSMEVND','HCIB','HCIBVND','HCN','HCNVND'],axis=1)

## GHI NHẬN CÁC SỐ LIỆU HUY ĐỘNG CỘT HPC108 CỦA CN VÀO SME

In [6]:
# tạo list các cột cần cập nhập
HPC_col=['HPC108VNDKKH','HPC108VNDNH','HPC108VNDTDH','HPC108SJC','HPC108USD','HPC108EUR','HPC108KHAC','HPC108KKH','HPC108NH','HPC108TDH']
HSME_col=['HSMEVNDKKH','HSMEVNDNH','HSMEVNDTDH','HSMESJC','HSMEUSD','HSMEEUR','HSMEKHAC','HSMEKKH','HSMENH','HSMETDH']
HCN_col=['HCNVNDKKH','HCNVNDNH','HCNVNDTDH','HCNSJC','HCNUSD','HCNEUR','HCNKHAC','HCNKKH','HCNNH','HCNTDH']

# lấy giá trị các cột HPC108 + tương ứng ở các cột HSME và - tương ứng ở các cột HCN
for i, j in zip(HSME_col, HPC_col):
    q[i]+=q[j]

for i,j in zip(HCN_col,HPC_col):
    q[i]-=q[j]

## CỘNG SỐ LIỆU ỦY THÁC TƯƠNG ỨNG THEO TỪNG MÃ ĐƠN VỊ

In [7]:
# đọc dữ liệu về Ủy Thác
UT=pd.read_excel(info_filename,sheet_name='UT')
UT=UT[UT['Vốn cấp 2'].notnull()]

# chuyển định dạng đúng của mã CN
if UT.iloc[:,0].dtypes == 'object':
    UT.iloc[:,0] = "'" + UT.iloc[:,0].str.zfill(3)
elif np.issubdtype(UT.iloc[:,0].dtypes, np.integer):
    UT.iloc[:,0] = "'" + UT.iloc[:,0].astype(str).str.zfill(3)
elif np.issubdtype(UT.iloc[:,0].dtypes, np.floating):
    UT.iloc[:,0] = "'" + UT.iloc[:,0].round(0).astype(int).astype(str).str.zfill(3)

# đổi các tên cột
UT=UT.rename(columns={
    'Vốn cấp 2':'Mã','CMB':'UTSME','Retail':"UTCN","CIB":"UTCIB"}).replace(' -   ', np.nan).fillna(0)
# các mã đơn vị cần cập nhập số liệu ủy thác
UT[(UT.UTSME!=0)|(UT.UTCN!=0)|(UT.UTCIB!=0)][['Mã','UTSME','UTCN','UTCIB']]

,Mã,UTSME,UTCN,UTCIB
8,'009,120.0,0.0,0.0
17,'020,0.0,0.0,250.0
27,'030,60.0,0.0,0.0
55,'059,550.0,0.0,0.0
58,'062,130.0,0.0,0.0
64,'068,0.0,0.0,10.0
156,'166,0.0,0.0,70.0
158,'168,200.0,0.0,0.0
317,'688,0.0,0.0,480.0


In [8]:
# các mã đơn vị sẽ update từ file Ủy thác
update_list=UT[(UT.UTSME!=0)|(UT.UTCN!=0)|(UT.UTCIB!=0)]['Mã'].tolist()
# số liệu ban đầu của các đơn vị cần update
q[q.CN.isin(update_list)][['CN','HSMEVNDTDH','HSMETDH','HCIBVNDTDH','HCIBTDH','HCNVNDTDH','HCNTDH']]

,CN,HSMEVNDTDH,HSMETDH,HCIBVNDTDH,HCIBTDH,HCNVNDTDH,HCNTDH
1,'068,3477.049767,3477.049767,0.000,0.000,1187.263342,1188.300683
9,'688,0.000000,0.000000,1219.000,1219.000,0.000000,0.000000
49,'059,272.027975,272.027975,173.000,173.000,740.211356,740.746147
53,'009,688.474937,688.474937,2.500,2.500,613.618779,613.618779
84,'062,299.044933,299.044933,0.000,0.000,593.965098,593.995182
98,'020,83.325553,83.325553,1384.000,1384.000,615.079464,615.148537
130,'030,0.000000,0.000000,0.000,0.000,610.500651,610.792127
206,'168,1006.187275,1006.187275,0.000,0.000,630.160172,630.196013
363,'166,0.000000,0.000000,1642.099,1642.099,0.000000,0.000000


In [9]:
# nối bảng UT với file KM theo từng mã CN
q=q.merge(UT,how='left',left_on='CN',right_on='Mã').fillna(0)

# cộng số Ủy thác theo từng mã đơn vị của từng khối vào VNDTDH và TDH
q['HSMEVNDTDH']+=q['UTSME']
q['HSMETDH']+=q['UTSME']
q['HCIBVNDTDH']+=q['UTCIB']
q['HCIBTDH']+=q['UTCIB']
q['HCNVNDTDH']+=q['UTCN']
q['HCNTDH']+=q['UTCN']

# xóa các cột của bảng UT
q.drop(UT.columns.tolist(),axis=1,inplace=True)

In [10]:
# các số liệu lúc sau của các đơn vị cần update
q[q.CN.isin(update_list)][['CN','HSMEVNDTDH','HSMETDH','HCIBVNDTDH','HCIBTDH','HCNVNDTDH','HCNTDH']]

,CN,HSMEVNDTDH,HSMETDH,HCIBVNDTDH,HCIBTDH,HCNVNDTDH,HCNTDH
1,'068,3477.049767,3477.049767,10.000,10.000,1187.263342,1188.300683
9,'688,0.000000,0.000000,1699.000,1699.000,0.000000,0.000000
49,'059,822.027975,822.027975,173.000,173.000,740.211356,740.746147
53,'009,808.474937,808.474937,2.500,2.500,613.618779,613.618779
84,'062,429.044933,429.044933,0.000,0.000,593.965098,593.995182
98,'020,83.325553,83.325553,1634.000,1634.000,615.079464,615.148537
130,'030,60.000000,60.000000,0.000,0.000,610.500651,610.792127
206,'168,1206.187275,1206.187275,0.000,0.000,630.160172,630.196013
363,'166,0.000000,0.000000,1712.099,1712.099,0.000000,0.000000


# CHIA SẺ SỐ HUY ĐỘNG LẠI GIỮA CÁC ĐƠN VỊ

In [11]:
# lưu dữ liệu vào dataframe
CIBSME_data=pd.read_excel(info_filename,sheet_name='CIB SME')

In [12]:
# backup dữ liệu vào CIB SME
df=CIBSME_data.copy()
date_string = df.iloc[0,0].date().strftime("%d/%m/%Y")
print(f'Ngày số liệu báo cáo của huy động CIB SME là: {date_string}')

Ngày số liệu báo cáo của huy động CIB SME là: 23/08/2023


In [13]:
selected_acct_no=['009704060001776','009704060001777','009704060001778','009704060001779',
                  '045704060003805','045704060003785','045704060003661','045704060003809']
# ngày đáo hạn của các món:
df[df['ACCT_NO'].isin(selected_acct_no)]['NGAY_DAOHAN']
# ngày đáo hạn sớm nhất của các món
print('Ngày đáo hạn sớm nhất các món chia sẻ huy động này: ')
df[df['ACCT_NO'].isin(selected_acct_no)]['NGAY_DAOHAN'].min()

Ngày đáo hạn sớm nhất các món chia sẻ huy động này: 


Timestamp('2023-08-24 00:00:00')

## SỐ DƯ HUY ĐỘNG 4 TK 0097 - TRỪ RA KHỎI MÃ CN '009 - CN HÀ NỘI

In [14]:
# CLIENT_NO = 02618199 (THUỘC CN HÀ NỘI)
# lọc data theo 4 số tài khoản "009704060001776", "009704060001777", "009704060001778", "009704060001779"
TK0097_NganHan = ['009704060001776','009704060001777'] # Ngan han
TK0097_TrungHan = ['009704060001778','009704060001779'] # Trung han
# Tổng số dư huy động quy đổi của các tài khoản 0097 ngắn hạn và 0097 trung hạn
# Nhân 80% số dư qd đổi này
# Quy đổi đơn vị đồng thành tỷ động
SoDuHuyDongNganHan_TK0097=CIBSME_data[CIBSME_data['ACCT_NO'].isin(TK0097_NganHan)]['SODU_QD'].sum()*0.8/billion
SoDuHuyDongTrungHan_TK0097=CIBSME_data[CIBSME_data['ACCT_NO'].isin(TK0097_TrungHan)]['SODU_QD'].sum()*0.8/billion
# các số liệu cần cập nhập
print('Tổng số dư huy động của 4 tài khoản 0097 theo ngắn hạn và dài hạn')
pd.DataFrame({'NH':[SoDuHuyDongNganHan_TK0097],'DH':[SoDuHuyDongTrungHan_TK0097]})

Tổng số dư huy động của 4 tài khoản 0097 theo ngắn hạn và dài hạn


,NH,DH
0,143.418521,380.0


In [15]:
# số liệu TRƯỚC khi cập nhập huy động ở 009
print('Số dư huy động CN 009 trước khi cập nhập')
q.loc[q.CN=="'009",['CN','HSMEVNDNH','HSMEVNDTDH','HSMENH','HSMETDH']]

Số dư huy động CN 009 trước khi cập nhập


,CN,HSMEVNDNH,HSMEVNDTDH,HSMENH,HSMETDH
53,'009,538.9967,808.474937,538.9967,808.474937


In [16]:
# Trừ vào ô HSMEVNDNH trong file bcqt của CN ='009
q.loc[q.CN=="'009",'HSMEVNDNH']-= SoDuHuyDongNganHan_TK0097
# Trừ vào ô HSMEVNDTDH trong file bcqt của CN ='009
q.loc[q.CN=="'009",'HSMEVNDTDH']-= SoDuHuyDongTrungHan_TK0097
# Trừ vào ô HSMENH trong file bcqt của CN ='009
q.loc[q.CN=="'009",'HSMENH']-= SoDuHuyDongNganHan_TK0097
# Trừ vào ô HSMETDH trong file bcqt của CN ='009
q.loc[q.CN=="'009",'HSMETDH']-= SoDuHuyDongTrungHan_TK0097

In [17]:
# số liệu SAU khi cập nhập huy động ở 009
print('Số dư huy động CN 009 sau khi cập nhập')
q.loc[q.CN=="'009",['CN','HSMEVNDNH','HSMEVNDTDH','HSMENH','HSMETDH']]

Số dư huy động CN 009 sau khi cập nhập


,CN,HSMEVNDNH,HSMEVNDTDH,HSMENH,HSMETDH
53,'009,395.57818,428.474937,395.57818,428.474937


## SỐ DƯ HUY ĐỘNG 4 TK 0457 - TRỪ Ở MÃ CN '045 - CN SÀI GÒN 

In [18]:
# CLIENT_NO 00006278 (THUỘC CN SÀI GÒN)
# lọc data theo 4 số tài khoản "045704060003443", "045704060003353", "045704060003440", "045704060003661"
TK0457_TrungHan =['045704060003805','045704060003785',] # TRUNG HAN
TK0457_NganHan =['045704060003661','045704060003809'] # NGAN HAN
# Tổng số dư huy động quy đổi của các tài khoản 0457 ngắn hạn và 0457 trung hạn
# Quy đổi đơn vị đồng thành tỷ động
SoDuHuyDongTrungHan_TK0457=CIBSME_data[CIBSME_data['ACCT_NO'].isin(TK0457_TrungHan)]['SODU_QD'].sum()/billion
SoDuHuyDongNganHan_TK0457=CIBSME_data[CIBSME_data['ACCT_NO'].isin(TK0457_NganHan)]['SODU_QD'].sum()/billion
print('Tổng số dư huy động của 4 tài khoản 0457 theo ngắn hạn và dài hạn')
pd.DataFrame({'NH':[SoDuHuyDongNganHan_TK0457],'DH':[SoDuHuyDongTrungHan_TK0457]})

Tổng số dư huy động của 4 tài khoản 0457 theo ngắn hạn và dài hạn


,NH,DH
0,400.0,300.0


In [19]:
# số liệu TRƯỚC khi cập nhập huy động ở 045
print('Số dư huy động CN 045 trước khi cập nhập')
q.loc[q.CN=="'045",['CN','HSMEVNDNH','HSMEVNDTDH','HSMENH','HSMETDH']]

Số dư huy động CN 045 trước khi cập nhập


,CN,HSMEVNDNH,HSMEVNDTDH,HSMENH,HSMETDH
129,'045,2355.759905,1321.427404,2355.759905,1321.427404


In [20]:
# CLIENT_NO 00006278 (THUỘC CN SÀI GÒN)
# Trừ vào ô HSMEVNDNH trong file bcqt của CN ='045
q.loc[q.CN=="'045",'HSMEVNDNH']-= SoDuHuyDongNganHan_TK0457
# Trừ vào ô HSMEVNDTDH trong file bcqt của CN ='045
q.loc[q.CN=="'045",'HSMEVNDTDH']-= SoDuHuyDongTrungHan_TK0457
# Trừ vào ô HSMENH trong file bcqt của CN ='045
q.loc[q.CN=="'045",'HSMENH']-= SoDuHuyDongNganHan_TK0457
# Trừ vào ô HSMETDH trong file bcqt của CN ='045
q.loc[q.CN=="'045",'HSMETDH']-= SoDuHuyDongTrungHan_TK0457

In [21]:
# số liệu SAU khi cập nhập huy động ở 045
print('Số dư huy động CN 045 sau khi cập nhập')
q.loc[q.CN=="'045",['CN','HSMEVNDNH','HSMEVNDTDH','HSMENH','HSMETDH']]

Số dư huy động CN 045 sau khi cập nhập


,CN,HSMEVNDNH,HSMEVNDTDH,HSMENH,HSMETDH
129,'045,1955.759905,1021.427404,1955.759905,1021.427404


## CỘNG SDHD 2 TK TRÊN VÀO MÃ CN '068 - TRUNG TÂM KINH DOANH

In [22]:
# số liệu TRƯỚC khi cập nhập huy động ở 068
print('Số dư huy động CN 068 trước khi cập nhập')
q.loc[q.CN=="'068",['CN','HSMEVNDNH','HSMEVNDTDH','HSMENH','HSMETDH']]

Số dư huy động CN 068 trước khi cập nhập


,CN,HSMEVNDNH,HSMEVNDTDH,HSMENH,HSMETDH
1,'068,4155.674924,3477.049767,4155.674924,3477.049767


In [23]:
# Trừ vào ô HSMEVNDNH trong file bcqt của CN ='068
q.loc[q.CN=="'068",'HSMEVNDNH']+= (SoDuHuyDongNganHan_TK0457+SoDuHuyDongNganHan_TK0097)
# Trừ vào ô HSMEVNDTDH trong file bcqt của CN ='068
q.loc[q.CN=="'068",'HSMEVNDTDH']+= (SoDuHuyDongTrungHan_TK0457+SoDuHuyDongTrungHan_TK0097)
# Trừ vào ô HSMENH trong file bcqt của CN ='068
q.loc[q.CN=="'068",'HSMENH']+= (SoDuHuyDongNganHan_TK0457+SoDuHuyDongNganHan_TK0097)
# Trừ vào ô HSMETDH trong file bcqt của CN ='068
q.loc[q.CN=="'068",'HSMETDH']+= (SoDuHuyDongTrungHan_TK0457+SoDuHuyDongTrungHan_TK0097)

In [24]:
# số liệu SAU khi cập nhập huy động ở 068
print('Số dư huy động CN 068 sau khi cập nhập')
q.loc[q.CN=="'068",['CN','HSMEVNDNH','HSMEVNDTDH','HSMENH','HSMETDH']]

Số dư huy động CN 068 sau khi cập nhập


,CN,HSMEVNDNH,HSMEVNDTDH,HSMENH,HSMETDH
1,'068,4699.093445,4157.049767,4699.093445,4157.049767


## LOẠI SỐ LIỆU TỪ FILE TÀI KHOẢN 43 44 (CHỨNG TỪ CÓ GIÁ)

In [25]:
# đọc dữ liệu file "43 44"
tk4344_df=pd.read_excel(info_filename,sheet_name='TK 43 44')
df=tk4344_df
# chuyển định dạng đúng của mã CN
if df.CN.dtypes == 'object':
    df.CN = "'" + df.CN.str.zfill(3)
elif np.issubdtype(df.CN.dtypes, np.integer):
    df.CN = "'" + df.CN.astype(str).str.zfill(3)
elif np.issubdtype(df.CN.dtypes, np.floating):
    df.CN = "'" + df.CN.round(0).astype(int).astype(str).str.zfill(3)

date_string = df.iloc[0,0].date().strftime("%d/%m/%Y")
print(f'Ngày số liệu báo cáo của huy động ở tk 43 44 là: {date_string}')

Ngày số liệu báo cáo của huy động ở tk 43 44 là: 06/09/2023


## GL_CODE 243 CUẢ CN 001 TRỪ ĐI VÀO PHẦN HUY ĐỘNG CỦA CN 001

In [26]:
# lọc GL_Code bắt đầu bằng 243 và CN 001
df=df[df.GL_CODE.apply(str).str.startswith('243')]
df=df[df.CN=="'001"] # lọc dữ liệu cột CN bằng 1

# Tính tỔng số dư quy đổi theo LOAI_KYHAN và LOẠI_KHACHHANG
df=df.groupby(['CN','LOAI_KHACHHANG','LOAI_TIEN','LOAI_KYHAN']).SODU_QD.sum().reset_index()
# Chuyển thành đơn vị tỷ đồng
df.SODU_QD=df.SODU_QD/billion
df

,CN,LOAI_KHACHHANG,LOAI_TIEN,LOAI_KYHAN,SODU_QD
0,'001,CIB,VND,DAI HAN,1380.0
1,'001,SME,USD,DAI HAN,7761.0
2,'001,SME,VND,DAI HAN,6440.0
3,'001,SME,VND,NGAN HAN,12500.0
4,'001,SME,VND,TRUNG HAN,16300.0


In [27]:
# sắp xếp các món huy động theo loại kỳ hạn tương ứng trong file KM
CN=[]
HSMENH=[]
HSMETDH=[]
HCIBNH=[]
HCIBTDH=[]
for i in range(len(df)):
    if df.iloc[i,1]=='SME' and df.iloc[i,3] == 'NGAN HAN':
        CN.append(df.iloc[i,0])
        HSMENH.append(df.iloc[i,4])
        HSMETDH.append(0)
        HCIBNH.append(0)
        HCIBTDH.append(0)
    if df.iloc[i,1]=='CIB' and df.iloc[i,3]=='NGAN HAN':
        CN.append(df.iloc[i,0])
        HSMENH.append(0)
        HSMETDH.append(0)
        HCIBNH.append(df.iloc[i,4])
        HCIBTDH.append(0)
    if df.iloc[i,1]=='SME' and df.iloc[i,3]!='NGAN HAN':
        CN.append(df.iloc[i,0])
        HSMENH.append(0)
        HSMETDH.append(df.iloc[i,4])
        HCIBNH.append(0)
        HCIBTDH.append(0)
    if df.iloc[i,1]=='CIB' and df.iloc[i,3]!='NGAN HAN':
        CN.append(df.iloc[i,0])
        HSMENH.append(0)
        HSMETDH.append(0)
        HCIBNH.append(0)
        HCIBTDH.append(df.iloc[i,4])
# bảng sắp xếp các món huy động theo NH VÀ TDH CỦA TỪNG KHỐI
data=pd.DataFrame({'CN':CN,'HSMENH':HSMENH,'HSMETDH':HSMETDH,'HCIBNH':HCIBNH,'HCIBTDH':HCIBTDH}).groupby('CN').sum().reset_index()
# chia các món huy động theo từng khồi
CIB_df=df[df.LOAI_KHACHHANG=='CIB'].drop(columns={'LOAI_KHACHHANG'})
SME_df=df[df.LOAI_KHACHHANG=='SME'].drop(columns={'LOAI_KHACHHANG'})

# chuyển từng món tiền vào từng ô với SME tương ứng trong file KM
CN=[]
HSMEVNDNH=[]
HSMEVNDTDH=[]
HSMESJC=[]
HSMEUSD=[]
HSMEEUR=[]
HSMEKHAC=[]

for i in range(len(SME_df)):
    if SME_df.iloc[i,1] == 'USD':
        CN.append(SME_df.iloc[i,0])
        HSMEVNDNH.append(0)
        HSMEVNDTDH.append(0)
        HSMESJC.append(0)
        HSMEUSD.append(SME_df.iloc[i,3])
        HSMEEUR.append(0)
        HSMEKHAC.append(0)
    if SME_df.iloc[i,1] == 'EUR':
        CN.append(SME_df.iloc[i,0])
        HSMEVNDNH.append(0)
        HSMEVNDTDH.append(0)
        HSMESJC.append(0)
        HSMEUSD.append(0)
        HSMEEUR.append(SME_df.iloc[i,3])
        HSMEKHAC.append(0)
    if SME_df.iloc[i,1] == 'SJC':
        CN.append(SME_df.iloc[i,0])
        HSMEVNDNH.append(0)
        HSMEVNDTDH.append(0)
        HSMESJC.append(SME_df.iloc[i,3])
        HSMEUSD.append(0)
        HSMEEUR.append(0)
        HSMEKHAC.append(0)
    if SME_df.iloc[i,1] != 'VND' and SME_df.iloc[i,1] != 'USD' and SME_df.iloc[i,1] != 'EUR'and SME_df.iloc[i,1] != 'SJC':
        CN.append(SME_df.iloc[i,0])
        HSMEVNDNH.append(0)
        HSMEVNDTDH.append(0)
        HSMESJC.append(0)
        HSMEUSD.append(0)
        HSMEEUR.append(0)
        HSMEKHAC.append(SME_df.iloc[i,3])
    if SME_df.iloc[i,1] == 'VND' and SME_df.iloc[i,2] == 'NGAN HAN':
        CN.append(SME_df.iloc[i,0])
        HSMEVNDNH.append(SME_df.iloc[i,3])
        HSMEVNDTDH.append(0)
        HSMESJC.append(0)
        HSMEUSD.append(0)
        HSMEEUR.append(0)
        HSMEKHAC.append(0)
    if SME_df.iloc[i,1] == 'VND' and SME_df.iloc[i,2] != 'NGAN HAN':
        CN.append(SME_df.iloc[i,0])
        HSMEVNDNH.append(0)
        HSMEVNDTDH.append(SME_df.iloc[i,3])
        HSMESJC.append(0)
        HSMEUSD.append(0)
        HSMEEUR.append(0)
        HSMEKHAC.append(0)
# tạo dataframe theo dạng file KM, tổng hợp lại theo từng mã CN (groupby)
SME_updated_df=pd.DataFrame({'CN':CN,'HSMEVNDNH':HSMEVNDNH,'HSMEVNDTDH':HSMEVNDTDH,'HSMESJC':HSMESJC,'HSMEUSD':HSMEUSD,
                             'HSMEEUR':HSMEEUR,'HSMEKHAC':HSMEKHAC}).groupby('CN').sum().reset_index()

# chuyển từng món tiền vào từng ô với CIB tương ứng trong file KM
CN=[]
HCIBVNDNH=[]
HCIBVNDTDH=[]
HCIBSJC=[]
HCIBUSD=[]
HCIBEUR=[]
HCIBKHAC=[]

for i in range(len(CIB_df)):
    if CIB_df.iloc[i,1] == 'USD':
        CN.append(CIB_df.iloc[i,0])
        HCIBVNDNH.append(0)
        HCIBVNDTDH.append(0)
        HCIBSJC.append(0)
        HCIBUSD.append(CIB_df.iloc[i,3])
        HCIBEUR.append(0)
        HCIBKHAC.append(0)
    if CIB_df.iloc[i,1] == 'EUR':
        CN.append(CIB_df.iloc[i,0])
        HCIBVNDNH.append(0)
        HCIBVNDTDH.append(0)
        HCIBSJC.append(0)
        HCIBUSD.append(0)
        HCIBEUR.append(CIB_df.iloc[i,3])
        HCIBKHAC.append(0)
    if CIB_df.iloc[i,1] == 'SJC':
        CN.append(CIB_df.iloc[i,0])
        HCIBVNDNH.append(0)
        HCIBVNDTDH.append(0)
        HCIBSJC.append(CIB_df.iloc[i,3])
        HCIBUSD.append(0)
        HCIBEUR.append(0)
        HCIBKHAC.append(0)
    if CIB_df.iloc[i,1] != 'VND' and CIB_df.iloc[i,1] != 'USD' and CIB_df.iloc[i,1] != 'EUR'and CIB_df.iloc[i,1] != 'SJC':
        CN.append(CIB_df.iloc[i,0])
        HCIBVNDNH.append(0)
        HCIBVNDTDH.append(0)
        HCIBSJC.append(0)
        HCIBUSD.append(0)
        HCIBEUR.append(0)
        HCIBKHAC.append(CIB_df.iloc[i,3])
    if CIB_df.iloc[i,1] == 'VND' and CIB_df.iloc[i,2] == 'NGAN HAN':
        CN.append(CIB_df.iloc[i,0])
        HCIBVNDNH.append(CIB_df.iloc[i,3])
        HCIBVNDTDH.append(0)
        HCIBSJC.append(0)
        HCIBUSD.append(0)
        HCIBEUR.append(0)
        HCIBKHAC.append(0)
    if CIB_df.iloc[i,1] == 'VND' and CIB_df.iloc[i,2] != 'NGAN HAN':
        CN.append(CIB_df.iloc[i,0])
        HCIBVNDNH.append(0)
        HCIBVNDTDH.append(CIB_df.iloc[i,3])
        HCIBSJC.append(0)
        HCIBUSD.append(0)
        HCIBEUR.append(0)
        HCIBKHAC.append(0)
# tạo dataframe theo dạng file KM, tổng hợp lại theo từng mã CN (groupby)
CIB_updated_df=pd.DataFrame({'CN':CN,'HCIBVNDNH':HCIBVNDNH,'HCIBVNDTDH':HCIBVNDTDH,'HCIBSJC':HCIBSJC,'HCIBUSD':HCIBUSD,
                             'HCIBEUR':HCIBEUR,'HCIBKHAC':HCIBKHAC}).groupby('CN').sum().reset_index()

# gộp các cột dữ dữ liệu về huy động như file KM
df=CIB_updated_df.merge(SME_updated_df,how='outer').merge(data,how='outer')

In [28]:
# số liệu TRƯỚC khi cập nhập
q[q.CN=="'001"][['CN', 'HSMEVNDNH', 'HSMEVNDTDH', 'HSMESJC', 'HSMEUSD', 'HSMEEUR', 'HSMEKHAC', 'HSMENH', 'HSMETDH', 
                 'HCIBVNDNH', 'HCIBVNDTDH', 'HCIBSJC', 'HCIBUSD', 'HCIBEUR', 'HCIBKHAC','HCIBNH', 'HCIBTDH']]

,CN,HSMEVNDNH,HSMEVNDTDH,HSMESJC,HSMEUSD,HSMEEUR,HSMEKHAC,HSMENH,HSMETDH,HCIBVNDNH,HCIBVNDTDH,HCIBSJC,HCIBUSD,HCIBEUR,HCIBKHAC,HCIBNH,HCIBTDH
0,'001,12500.0,22740.0,0.0,7762.259649,0.0,0.0,12500.0,30501.0,0.0,1380.0,0.0,0.0,0.0,0.0,0.0,1380.0


In [29]:
# trừ các số liệu cho vào các ô tương ứng trong file BCQT 20
q.merge(df,how='left',on='CN',suffixes=('', '_df2')).fillna(0)
updated_col=['HCIBVNDNH', 'HCIBVNDTDH', 'HCIBSJC', 'HCIBUSD', 'HCIBEUR',
       'HCIBKHAC', 'HSMEVNDNH', 'HSMEVNDTDH', 'HSMESJC', 'HSMEUSD', 'HSMEEUR',
       'HSMEKHAC', 'HSMENH', 'HSMETDH', 'HCIBNH', 'HCIBTDH']
# nối tương ứng bảng giá trị cần cập nhập theo từng mã CN
q = q.merge(df, on='CN', how='left', suffixes=('', '_df2')).fillna(0)
# cộng các số dư huy động vào file KM theo từng cột
for i in updated_col:
    q[f'{i}'] -= q[f'{i}_df2']
# xóa các cột được nối vào
for i in updated_col:
    q.drop(f'{i}_df2', axis=1, inplace=True)

In [30]:
# số liệu SAU khi cập nhập
q[q.CN=="'001"][['CN', 'HSMEVNDNH', 'HSMEVNDTDH', 'HSMESJC', 'HSMEUSD', 'HSMEEUR', 'HSMEKHAC', 'HSMENH', 'HSMETDH', 
                 'HCIBVNDNH', 'HCIBVNDTDH', 'HCIBSJC', 'HCIBUSD', 'HCIBEUR', 'HCIBKHAC','HCIBNH', 'HCIBTDH']]

,CN,HSMEVNDNH,HSMEVNDTDH,HSMESJC,HSMEUSD,HSMEEUR,HSMEKHAC,HSMENH,HSMETDH,HCIBVNDNH,HCIBVNDTDH,HCIBSJC,HCIBUSD,HCIBEUR,HCIBKHAC,HCIBNH,HCIBTDH
0,'001,0.0,0.0,0.0,1.259649,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## GL_CODE ĐẦU 244 GHI NHẬN CHO CÁC MÃ ĐƠN VỊ KHÁC

## lưu ý: kiểm tra GL_CODE

In [31]:
# đọc dữ liệu file "43 44"
df=tk4344_df
# lọc GL_Code bắt đầu bằng 244
df=df[df.GL_CODE.apply(str).str.startswith('244')] # chuyển cột "GL_CODE" sang string và tìm giá trị bắt đầu bằng 244

# Tính tỔng số dư quy đổi theo LOAI_KYHAN và LOẠI_KHACHHANG
df=df.groupby(['CN','LOAI_KHACHHANG','LOAI_TIEN','LOAI_KYHAN']).SODU_QD.sum().reset_index()
# Chuyển thành đơn vị tỷ đồng
df.SODU_QD=df.SODU_QD/billion
# LOAI_KHACHHANG là TCTD thì xếp vào ô CIB hay SME
df

,CN,LOAI_KHACHHANG,LOAI_TIEN,LOAI_KYHAN,SODU_QD
0,'001,TCTD,USD,DAI HAN,0.736620
1,'135,TCTD,JPY,DAI HAN,53.440442
2,'688,CIB,USD,DAI HAN,2778.763636


In [32]:
# sắp xếp các món huy động vào file KM theo loại kỳ hạn
CN=[]
HSMENH=[]
HSMETDH=[]
HCIBNH=[]
HCIBTDH=[]
for i in range(len(df)):
    if df.iloc[i,1]=='SME' and df.iloc[i,3] == 'NGAN HAN':
        CN.append(df.iloc[i,0])
        HSMENH.append(df.iloc[i,4])
        HSMETDH.append(0)
        HCIBNH.append(0)
        HCIBTDH.append(0)
    if df.iloc[i,1]!='SME' and df.iloc[i,3]=='NGAN HAN':
        CN.append(df.iloc[i,0])
        HSMENH.append(0)
        HSMETDH.append(0)
        HCIBNH.append(df.iloc[i,4])
        HCIBTDH.append(0)
    if df.iloc[i,1]=='SME' and df.iloc[i,3]!='NGAN HAN':
        CN.append(df.iloc[i,0])
        HSMENH.append(0)
        HSMETDH.append(df.iloc[i,4])
        HCIBNH.append(0)
        HCIBTDH.append(0)
    if df.iloc[i,1]!='SME' and df.iloc[i,3]!='NGAN HAN':
        CN.append(df.iloc[i,0])
        HSMENH.append(0)
        HSMETDH.append(0)
        HCIBNH.append(0)
        HCIBTDH.append(df.iloc[i,4])
data=pd.DataFrame({'CN':CN,'HSMENH':HSMENH,'HSMETDH':HSMETDH,'HCIBNH':HCIBNH,'HCIBTDH':HCIBTDH}).groupby('CN').sum().reset_index()

SME_df=df[df.LOAI_KHACHHANG=='SME'].drop(columns={'LOAI_KHACHHANG'})
# chuyển từng món tiền vào từng ô với SME tương ứng trong file KM
CN=[]
HSMEVNDNH=[]
HSMEVNDTDH=[]
HSMESJC=[]
HSMEUSD=[]
HSMEEUR=[]
HSMEKHAC=[]

for i in range(len(SME_df)):
    if SME_df.iloc[i,1] == 'USD':
        CN.append(SME_df.iloc[i,0])
        HSMEVNDNH.append(0)
        HSMEVNDTDH.append(0)
        HSMESJC.append(0)
        HSMEUSD.append(SME_df.iloc[i,3])
        HSMEEUR.append(0)
        HSMEKHAC.append(0)
    if SME_df.iloc[i,1] == 'EUR':
        CN.append(SME_df.iloc[i,0])
        HSMEVNDNH.append(0)
        HSMEVNDTDH.append(0)
        HSMESJC.append(0)
        HSMEUSD.append(0)
        HSMEEUR.append(SME_df.iloc[i,3])
        HSMEKHAC.append(0)
    if SME_df.iloc[i,1] == 'SJC':
        CN.append(SME_df.iloc[i,0])
        HSMEVNDNH.append(0)
        HSMEVNDTDH.append(0)
        HSMESJC.append(SME_df.iloc[i,3])
        HSMEUSD.append(0)
        HSMEEUR.append(0)
        HSMEKHAC.append(0)
    if SME_df.iloc[i,1] != 'VND' and SME_df.iloc[i,1] != 'USD' and SME_df.iloc[i,1] != 'EUR'and SME_df.iloc[i,1] != 'SJC':
        CN.append(SME_df.iloc[i,0])
        HSMEVNDNH.append(0)
        HSMEVNDTDH.append(0)
        HSMESJC.append(0)
        HSMEUSD.append(0)
        HSMEEUR.append(0)
        HSMEKHAC.append(SME_df.iloc[i,3])
    if SME_df.iloc[i,1] == 'VND' and SME_df.iloc[i,2] == 'NGAN HAN':
        CN.append(SME_df.iloc[i,0])
        HSMEVNDNH.append(SME_df.iloc[i,3])
        HSMEVNDTDH.append(0)
        HSMESJC.append(0)
        HSMEUSD.append(0)
        HSMEEUR.append(0)
        HSMEKHAC.append(0)
    if SME_df.iloc[i,1] == 'VND' and SME_df.iloc[i,2] != 'NGAN HAN':
        CN.append(SME_df.iloc[i,0])
        HSMEVNDNH.append(0)
        HSMEVNDTDH.append(SME_df.iloc[i,3])
        HSMESJC.append(0)
        HSMEUSD.append(0)
        HSMEEUR.append(0)
        HSMEKHAC.append(0)
# tạo dataframe theo dạng file KM, tổng hợp lại theo từng mã CN (groupby)
SME_updated_df=pd.DataFrame({'CN':CN,'HSMEVNDNH':HSMEVNDNH,'HSMEVNDTDH':HSMEVNDTDH,'HSMESJC':HSMESJC,'HSMEUSD':HSMEUSD,
                             'HSMEEUR':HSMEEUR,'HSMEKHAC':HSMEKHAC}).groupby('CN').sum().reset_index()

CIB_df=df[df.LOAI_KHACHHANG!='SME'].drop(columns={'LOAI_KHACHHANG'})
# chuyển từng món tiền vào từng ô với CIB tương ứng trong file KM
CN=[]
HCIBVNDNH=[]
HCIBVNDTDH=[]
HCIBSJC=[]
HCIBUSD=[]
HCIBEUR=[]
HCIBKHAC=[]

for i in range(len(CIB_df)):
    if CIB_df.iloc[i,1] == 'USD':
        CN.append(CIB_df.iloc[i,0])
        HCIBVNDNH.append(0)
        HCIBVNDTDH.append(0)
        HCIBSJC.append(0)
        HCIBUSD.append(CIB_df.iloc[i,3])
        HCIBEUR.append(0)
        HCIBKHAC.append(0)
    if CIB_df.iloc[i,1] == 'EUR':
        CN.append(CIB_df.iloc[i,0])
        HCIBVNDNH.append(0)
        HCIBVNDTDH.append(0)
        HCIBSJC.append(0)
        HCIBUSD.append(0)
        HCIBEUR.append(CIB_df.iloc[i,3])
        HCIBKHAC.append(0)
    if CIB_df.iloc[i,1] == 'SJC':
        CN.append(CIB_df.iloc[i,0])
        HCIBVNDNH.append(0)
        HCIBVNDTDH.append(0)
        HCIBSJC.append(CIB_df.iloc[i,3])
        HCIBUSD.append(0)
        HCIBEUR.append(0)
        HCIBKHAC.append(0)
    if CIB_df.iloc[i,1] != 'VND' and CIB_df.iloc[i,1] != 'USD' and CIB_df.iloc[i,1] != 'EUR'and CIB_df.iloc[i,1] != 'SJC':
        CN.append(CIB_df.iloc[i,0])
        HCIBVNDNH.append(0)
        HCIBVNDTDH.append(0)
        HCIBSJC.append(0)
        HCIBUSD.append(0)
        HCIBEUR.append(0)
        HCIBKHAC.append(CIB_df.iloc[i,3])
    if CIB_df.iloc[i,1] == 'VND' and CIB_df.iloc[i,2] == 'NGAN HAN':
        CN.append(CIB_df.iloc[i,0])
        HCIBVNDNH.append(CIB_df.iloc[i,3])
        HCIBVNDTDH.append(0)
        HCIBSJC.append(0)
        HCIBUSD.append(0)
        HCIBEUR.append(0)
        HCIBKHAC.append(0)
    if CIB_df.iloc[i,1] == 'VND' and CIB_df.iloc[i,2] != 'NGAN HAN':
        CN.append(CIB_df.iloc[i,0])
        HCIBVNDNH.append(0)
        HCIBVNDTDH.append(CIB_df.iloc[i,3])
        HCIBSJC.append(0)
        HCIBUSD.append(0)
        HCIBEUR.append(0)
        HCIBKHAC.append(0)
# tạo dataframe theo dạng file KM, tổng hợp lại theo từng mã CN (groupby)
CIB_updated_df=pd.DataFrame({'CN':CN,'HCIBVNDNH':HCIBVNDNH,'HCIBVNDTDH':HCIBVNDTDH,'HCIBSJC':HCIBSJC,'HCIBUSD':HCIBUSD,
                             'HCIBEUR':HCIBEUR,'HCIBKHAC':HCIBKHAC}).groupby('CN').sum().reset_index()


In [33]:
# gộp các cột dữ dữ liệu về huy động như file KM
df=CIB_updated_df.merge(SME_updated_df,how='outer').merge(data,how='outer')
# điền các giá trị trống bằng 0
df=df.fillna(0)
# số liệu huy động ghi nhận cho các đơn vị khác
df

,CN,HCIBVNDNH,HCIBVNDTDH,HCIBSJC,HCIBUSD,HCIBEUR,HCIBKHAC,HSMEVNDNH,HSMEVNDTDH,HSMESJC,HSMEUSD,HSMEEUR,HSMEKHAC,HSMENH,HSMETDH,HCIBNH,HCIBTDH
0,'001,0,0,0,0.736620,0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.736620
1,'135,0,0,0,0.000000,0,53.440442,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,53.440442
2,'688,0,0,0,2778.763636,0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,2778.763636


In [34]:
# gộp số liệu của CN '001 với CN '135
updated_col=['HCIBVNDNH', 'HCIBVNDTDH', 'HCIBSJC', 'HCIBUSD', 'HCIBEUR',
       'HCIBKHAC', 'HSMEVNDNH', 'HSMEVNDTDH', 'HSMESJC', 'HSMEUSD', 'HSMEEUR',
       'HSMEKHAC', 'HSMENH', 'HSMETDH', 'HCIBNH', 'HCIBTDH']
for i in updated_col:
    df.loc[df.CN=="'001",i]+=df.loc[df.CN=="'135",i].values[0]

df=df[df.CN!="'135"].fillna(0)

updated_col=['HCIBVNDNH', 'HCIBVNDTDH', 'HCIBSJC', 'HCIBUSD', 'HCIBEUR',
       'HCIBKHAC', 'HSMEVNDNH', 'HSMEVNDTDH', 'HSMESJC', 'HSMEUSD', 'HSMEEUR',
       'HSMEKHAC', 'HSMENH', 'HSMETDH', 'HCIBNH', 'HCIBTDH']
# nối tương ứng bảng giá trị cần cập nhập theo từng mã CN
q = q.merge(df, on='CN', how='left', suffixes=('', '_df2')).fillna(0)
# cộng các số dư huy động vào file KM theo từng cột
for i in updated_col:
    q[f'{i}'] += q[f'{i}_df2']
# xóa các cột được nối vào
for i in updated_col:
    q.drop(f'{i}_df2', axis=1, inplace=True)

## CẬP NHẬP HUY ĐỘNG TỪ TRÁI PHIẾU

In [35]:
# chọn đường dẫn file trái phiếu mới
file_path=TraiPhieuPath

# lập danh sách các tên sheet của file excel
xls = pd.ExcelFile(file_path)
sheet_names = xls.sheet_names

# Lấy tên file cuối cùng
last_sheet_name = sheet_names[-1]

# Đọc sheet cuối cùng của file excel
TraiPhieu_df = pd.read_excel(file_path, skiprows=1,sheet_name=last_sheet_name)
TraiPhieu_df=TraiPhieu_df[TraiPhieu_df['Tên TP'].notnull()]
# Chuyển các cột giá trị ghi nhận trái phiếu thành dạng thập phân
TraiPhieu_df['Mệnh giá (Ghi nhận TPDN)']=TraiPhieu_df['Mệnh giá (Ghi nhận TPDN)'].astype(float)
TraiPhieu_df['Số tiền bán (Ghi nhận HĐ)']=TraiPhieu_df['Số tiền bán (Ghi nhận HĐ)'].astype(float)
selected_cols=['Mã ĐV CV', 'Mã ĐV HĐ', 'Mệnh giá (Ghi nhận TPDN)', 'Số tiền bán (Ghi nhận HĐ)',
              'Ghi nhận cho khối và \nĐVKD về trái phiếu DN', 'Ghi nhận cho khối và \nĐVKD về huy động',
               'Tên TP', 'Tên KH mua', 'Ngày bán', 'Ngày mua lại','Đã trình ghi nhận CV',
       'Đã trình ghi nhận HĐ']
TraiPhieu_df=TraiPhieu_df[selected_cols]

In [36]:
df=TraiPhieu_df.copy()
# loại bỏ các dòng cuối của file excel
# lọc đã trình ghi nhận Huy động
df=df[df['Đã trình ghi nhận HĐ'].notnull()]
# lọc ngày mua lại trống
df=df[~df['Ngày mua lại'].notnull()]
# Group by theo từng mã đơn vị (Mã DV HĐ) và từng khối ("Ghi nhận cho khối và ĐVKD về huy động") ... tính tổng cột "Số tiền bán (Ghi nhận HĐ)"
df=df.groupby(['Mã ĐV HĐ','Ghi nhận cho khối và \nĐVKD về huy động'])['Số tiền bán (Ghi nhận HĐ)'].sum().reset_index()
df['Mã ĐV HĐ']="'"+df['Mã ĐV HĐ'].astype(int).astype(str).str.zfill(3)
df['Số tiền bán (Ghi nhận HĐ)']=df['Số tiền bán (Ghi nhận HĐ)'].astype(float)
TraiPhieuHuyDong=df['Số tiền bán (Ghi nhận HĐ)'].sum()*billion

In [37]:
SME_df=df[df['Ghi nhận cho khối và \nĐVKD về huy động']=='CKH CMB']
SME_df=SME_df.groupby('Mã ĐV HĐ')['Số tiền bán (Ghi nhận HĐ)'].sum().reset_index()
CIB_df=df[df['Ghi nhận cho khối và \nĐVKD về huy động']=='CKH CIB']
CIB_df=CIB_df.groupby('Mã ĐV HĐ')['Số tiền bán (Ghi nhận HĐ)'].sum().reset_index()

In [38]:
updated_col=['HSMEVNDNH','HSMENH']
# nối tương ứng bảng giá trị cần cập nhập theo từng mã CN
q = q.merge(SME_df, left_on='CN',right_on='Mã ĐV HĐ', how='left', suffixes=('', '_df2')).fillna(0)
# chuyển các cột cần update thành dạng float
q[updated_col]=q[updated_col].astype(float)

# cộng các số dư huy động vào file KM theo từng cột
for i in updated_col:
    q[i] += q['Số tiền bán (Ghi nhận HĐ)']
# xóa các cột được nối vào
for i in SME_df.columns.tolist():
    q.drop(i, axis=1, inplace=True)

updated_col=['HCIBVNDNH','HCIBNH']
# nối tương ứng bảng giá trị cần cập nhập theo từng mã CN
q = q.merge(CIB_df, left_on='CN',right_on='Mã ĐV HĐ', how='left', suffixes=('', '_df2')).fillna(0)
# chuyển các cột cần update thành dạng float
q[updated_col]=q[updated_col].astype(float)

# cộng các số dư huy động vào file KM theo từng cột
for i in updated_col:
    q[i] += q['Số tiền bán (Ghi nhận HĐ)']
# xóa các cột được nối vào
for i in CIB_df.columns.tolist():
    q.drop(i, axis=1, inplace=True)

## THÊM CÁC CỘT GIÁ TRỊ TỔNG CỦA HUY ĐỘNG

In [39]:
# chuyển các cột saut hành số thập phân
q[['HSMEVNDKKH', 'HSMEVNDNH', 'HSMEVNDTDH']]=q[['HSMEVNDKKH', 'HSMEVNDNH', 'HSMEVNDTDH']].astype(float)

# các cột cần phải thêm vào sau cùng: HSMEVND, HSME, HCIBVND, HCIB, HCNVND, HCN, HNGAYBC
# HSMEVND = HSMEVNDKKH,HSMEVNDNH,HSMEVNDTDH
# HSME = HSMEVND+HSMESJC+HSMEUSD+HSMEEUR+HSMEKHAC
# HNGAYBC = HSME+ HCIB + HCN
q['HSMEVND']=q['HSMEVNDKKH']+q['HSMEVNDNH']+q['HSMEVNDTDH']
q['HSME']=q['HSMEVND']+q['HSMESJC']+q['HSMEUSD']+q['HSMEEUR']+q['HSMEKHAC']
q['HCIBVND']=q['HCIBVNDKKH']+q['HCIBVNDNH']+q['HCIBVNDTDH']
q['HCIB']=q['HCIBVND']+q['HCIBSJC']+q['HCIBUSD']+q['HCIBEUR']+q['HCIBKHAC']
q['HCNVND']=q['HCNVNDKKH']+q['HCNVNDNH']+q['HCNVNDTDH']
q['HCN']=q['HCNVND']+q['HCNSJC']+q['HCNUSD']+q['HCNEUR']+q['HCNKHAC']
q['HNGAYBC']=q['HSME']+q['HCIB']+q['HCN']

# chuyển các cột số thành dạng số thập phân
q.iloc[:,1:]=q.iloc[:,1:].astype(float)

# ĐỐI CHIẾU HUY ĐỘNG VS B02

In [40]:
# kiểm tra dựa trên file BO2
check_df=pd.read_excel(info_filename,sheet_name='B02')
TienGuiKhachHang=check_df.iloc[110,3] # TK 42 tiền gửi khách hàng
CTCG=check_df.iloc[116,3] # TK 43 phát hành chứng từ có giá
VonTaiTroUyThac=check_df.iloc[114,3] # TK 44 vốn tài trợ ủy thác
print(f"TK 42 (Tiền gửi của khách hàng): {TienGuiKhachHang}")
print(f"TK 43 (Phát hành GTCG): {CTCG}")
print(f"TK 44 (Vốn TTUT): {VonTaiTroUyThac}")
print(f"Trái Phiếu cộng tay ghi nhận HĐ: {TraiPhieuHuyDong}")
# tổng tiền gửi khách hàng, CTCG, tài trợ ủy thác và trái phiếu cộng tay huy động
print(f'Tổng: {TienGuiKhachHang+CTCG+VonTaiTroUyThac+TraiPhieuHuyDong}')
f1=TienGuiKhachHang+CTCG+VonTaiTroUyThac+TraiPhieuHuyDong

TK 42 (Tiền gửi của khách hàng): 334496512019146.0
TK 43 (Phát hành GTCG): 46844941117598.0
TK 44 (Vốn TTUT): 2832940698597.0
Trái Phiếu cộng tay ghi nhận HĐ: 227268149730.97897
Tổng: 384401661985072.0


In [41]:
# tìm các cột có SBV Code chứa số 43 ở đầu trong dữ liệu
# lưu file vào biến alm
gl101=pd.read_excel(info_filename,sheet_name='gl101')
df=gl101[gl101['SBV_CODE'].astype(str).str.startswith('43')]
# Tổng cột CR_CLOS trừ cho tổng cột DR_CLOS và quy đổi thành đơn vị tỷ đồng
a=(df.CR_CLOS.sum()-df.DR_CLOS.sum())/billion

# lưu file vào biến alm
# đọc dữ liệu file "43 44"
df=pd.read_excel(info_filename,sheet_name='TK 43 44')
# tìm các cột có SBV Code chứa số 243 ở đầu trong dữ liệu
df=df[df['GL_CODE'].astype(str).str.startswith('243')]
# cột CN bỏ "001"
df=df[df.CN!=1]
# Quy đổi thành tỷ đồng
b=df.SODU_QD.sum()/billion

# Số liệu vốn Ủy thác tổng 3 cột Ủy thác của 3 khối trong file Ủy thác
c=UT.UTSME.sum()+UT.UTCN.sum()+UT.UTCIB.sum()
# Tổng số Chứng từ có giá:
CTCG=a-b-c
# tổng CTCG và huy động sau cập nhập
f2=(CTCG+q['HNGAYBC'].sum())*billion
print('Tổng huy động của báo cáo ngày: ')
f2

Tổng huy động của báo cáo ngày: 


384401661985072.0

In [42]:
# độ lệch của huy động báo cáo ngày và B02 check
print('Độ lệch của Huy động của báo cáo ngày so với B02: (đồng)')
f1-f2

Độ lệch của Huy động của báo cáo ngày so với B02: (đồng)


0.0

# B. CẬP NHẬP CHO VAY

In [43]:
# xóa các cột tổng về cho vay
q.drop(['CVTHANGTRC','CVNGAYTRC','CVNGAYBC','CVSME','CVSMEVND','CVCIB','CVCIBVND','CVCN','CVCNVND'],axis=1,inplace=True)

In [44]:
# THÊM 3 CỘT MỚI 30% UPAS LC CHO TỪNG KHỐI về tổng cho vay và 30% LC UPAS
q['UPAS_LC_SME_30']=q['UPAS_LC_SME']*0.3
q['UPAS_LC_CIB_30']=q['UPAS_LC_CIB']*0.3
q['UPAS_LC_CN_30']=q['UPAS_LC_CN']*0.3


## Cộng thêm số liệu từ file ỦY THÁC

In [45]:
df=pd.read_excel(info_filename,sheet_name='UT')
df=df[df['Vốn cấp 2'].notnull()].rename(columns={'Vốn cấp 2':'Mã'})

# chuyển thành dạng 3 kí tự với dấu phẩy đầu
df['Mã'] = "'"+df.Mã.astype(int).astype(str).str.zfill(3)

UyThac1=(df['von uy thac CMB'].sum()+df['von uy thac retail'].sum()+df['von uy thac CIB'].sum())*billion
UyThac1

3848752311632.0

In [46]:
df=df[['Mã','von uy thac CMB','von uy thac retail','von uy thac CIB']]
# chuyển các cột vốn ủy thác của từng khối thành dạng số thập phân
df[['von uy thac CMB','von uy thac retail','von uy thac CIB']]=df[['von uy thac CMB','von uy thac retail','von uy thac CIB']].astype(float)

# thêm 3 cột vào file KM tương ứng theo từng mã CN
q=q.merge(df, how='left',left_on='CN',right_on='Mã').fillna(0)

# cập nhập cho vay VNDDH và CVDH với với các vốn ủy thác theo từng khối
q['CVSMEVNDDH']+=q['von uy thac CMB']
q['CVSMEDH']+=q['von uy thac CMB']
q['CVCIBVNDDH']+=q['von uy thac CIB']
q['CVCIBDH']+=q['von uy thac CIB']
q['CVCNVNDDH']+=q['von uy thac retail']
q['CVCNDH']+=q['von uy thac retail']

# xóa các cột được nối vào
for i in df.columns.tolist():
    q.drop(i, axis=1, inplace=True)

# Xuất file ủy thác (số phát sinh) trên KM để tính ra số lũy kế

In [47]:
UyThacPhatSinh_df=pd.read_excel(info_filename,sheet_name='ủy thác phát sinh')
df=UyThacPhatSinh_df.copy()
UyThac2=df.PHATSINH_NO.sum()-df.PHATSINH_CO.sum()
df.PHATSINH_CO=df.PHATSINH_CO/billion
df.PHATSINH_NO=df.PHATSINH_NO/billion
print(f'Tong uy thac phat sinh them: {UyThac2}')

Tong uy thac phat sinh them: -15991970666


In [48]:
#PROFIT_CENTRE : 1 là SME, 2 là CN, 8 là CIB
SME_df=df[df.PROFIT_CENTRE.str.startswith("'1")]
CN_df=df[df.PROFIT_CENTRE.str.startswith("'2")]
CIB_df=df[df.PROFIT_CENTRE.str.startswith("'8")]

In [49]:
# tổng phát sinh có và phát sinh nợ theo Branch
SME_df=SME_df.groupby('MA_CN')[['PHATSINH_NO','PHATSINH_CO']].sum().reset_index()
# tổng phát sinh có và phát sinh nợ theo Branch của CIB
CIB_df=CIB_df.groupby('MA_CN')[['PHATSINH_NO','PHATSINH_CO']].sum().reset_index()
# tổng phát sinh có và phát sinh nợ theo Branch của CN
CN_df=CN_df.groupby('MA_CN')[['PHATSINH_NO','PHATSINH_CO']].sum().reset_index()

In [50]:
#PHATSINH_NO là cộng thêm vô ... PHATSINH_CO là giảm đi
#cộng vào cột CV VND DH và CV DH (của phần cho vay) theo từng mã đơn vị của từng khối
q = q.merge(SME_df,left_on='CN',right_on='MA_CN',how='left').fillna(0)
q['CVSMEVNDDH']+=q['PHATSINH_NO']
q['CVSMEDH']+=q['PHATSINH_NO']
q['CVSMEVNDDH']-=q['PHATSINH_CO']
q['CVSMEDH']-=q['PHATSINH_CO']
# xóa các cột đã thêm vào
q.drop(SME_df.columns.tolist(),axis=1,inplace=True)

In [51]:
#PHATSINH_NO là cộng thêm vô ... PHATSINH_CO là giảm đi
#cộng vào cột CV VND DH và CV DH (của phần cho vay) theo từng mã đơn vị của từng khối
q = q.merge(CIB_df,left_on='CN',right_on='MA_CN',how='left').fillna(0)
q['CVCIBVNDDH']+=q['PHATSINH_NO']
q['CVCIBDH']+=q['PHATSINH_NO']
q['CVCIBVNDDH']-=q['PHATSINH_CO']
q['CVCIBDH']-=q['PHATSINH_CO']
# xóa các cột đã thêm vào
q.drop(CIB_df.columns.tolist(),axis=1,inplace=True)

In [52]:
#PHATSINH_NO là cộng thêm vô ... PHATSINH_CO là giảm đi
#cộng vào cột CV VND DH và CV DH (của phần cho vay) theo từng mã đơn vị của từng khối
q = q.merge(CN_df,left_on='CN',right_on='MA_CN',how='left').fillna(0)
q['CVCNVNDDH']+=q['PHATSINH_NO']
q['CVCNDH']+=q['PHATSINH_NO']
q['CVCNVNDDH']-=q['PHATSINH_CO']
q['CVCNDH']-=q['PHATSINH_CO']
# xóa các cột đã thêm vào
q.drop(CN_df.columns.tolist(),axis=1,inplace=True)

# ghi nhận CHO VAY từ TRÁI PHIẾU

In [53]:
# tính tổng trái phiểu hệ thống
TPDN=q['DTCK'].sum()*billion
print(f'Trái phiếu số hệ thống: {TPDN}')

Trái phiếu số hệ thống: 6493797662702.0


In [54]:
# là cột DTCK ... cập nhập tay từ file trái phiếu cộng tay
# mở file trái phiế ... CỘT "đã trình ghi nhận CV ko trống ... ngày mua lại = blank ... 
df=TraiPhieu_df.copy()
# chuyển giá trị cột CN thành dạng 3 kí tự 1 => 001
df['Mã ĐV CV']="'"+df['Mã ĐV CV'].astype(int).astype(str).str.zfill(3)

# # CỘT "đã trình ghi nhận CV ko trống ... ngày mua lại = blank ... 
df=df[df['Đã trình ghi nhận CV'].notnull()]
df=df[~df['Ngày mua lại'].notnull()]


In [55]:
TraiPhieuChoVay=df['Mệnh giá (Ghi nhận TPDN)'].sum()
print(f'Trái phiếu cộng tay thêm: {TraiPhieuChoVay}')

Trái phiếu cộng tay thêm: 228.0


In [56]:
# trái phiếu phòng đầu tư
TraiPhieuPDT=df.loc[df['Ghi nhận cho khối và \nĐVKD về trái phiếu DN']=='P.ĐT','Mệnh giá (Ghi nhận TPDN)'].sum()
print(f'Trái phiếu phòng Đầu tư: {TraiPhieuPDT}')

print(f'Trái phiếu cộng tay ghi nhận vào cột trái phiếu hệ thống: {TraiPhieuChoVay-TraiPhieuPDT}')

Trái phiếu phòng Đầu tư: 119.6
Trái phiếu cộng tay ghi nhận vào cột trái phiếu hệ thống: 108.4


In [57]:
# "GHI NHẬN CHO KHỐI VÀ ĐVKD về trái phiếu DN"
# loại bỏ những trái phiếu của phòng đầu tư
df=df[df['Ghi nhận cho khối và \nĐVKD về trái phiếu DN']!='P.ĐT']

# tạo file data trái phiếu ghi nhận cho từng khối
SME_df=df[df['Ghi nhận cho khối và \nĐVKD về trái phiếu DN']=='CMB']
CIB_df=df[df['Ghi nhận cho khối và \nĐVKD về trái phiếu DN']=='CIB']

# tính tổng mệnh giá ghi nhận TPDN cho từng mã CN
SME_df=SME_df.groupby('Mã ĐV CV')['Mệnh giá (Ghi nhận TPDN)'].sum().reset_index()
CIB_df=CIB_df.groupby('Mã ĐV CV')['Mệnh giá (Ghi nhận TPDN)'].sum().reset_index()

In [58]:
# cập nhập vào cột tổng cho vay của khối SME
q=q.merge(SME_df,how='left',left_on='CN',right_on='Mã ĐV CV').fillna(0)

# cột thêm các trái phiếu cộng tay này vào cột DTCK trong file KM
q['DTCK'] += q['Mệnh giá (Ghi nhận TPDN)']

# xóa các cột thêm vào
for i in SME_df.columns.tolist():
    q.drop(i, axis=1, inplace=True)
    

# cập nhập vào cột tổng cho vay của khối CIB
q=q.merge(CIB_df,how='left',left_on='CN',right_on='Mã ĐV CV').fillna(0)
q['Mệnh giá (Ghi nhận TPDN)']

# cột thêm các trái phiếu cộng tay này vào cột DTCK trong file KM
q['DTCK'] += q['Mệnh giá (Ghi nhận TPDN)']

# xóa các cột thêm vào
for i in CIB_df.columns.tolist():
    q.drop(i, axis=1, inplace=True)

In [59]:
q['DTCK'].sum()*billion

6602197662702.0

# CỘNG 300 TỶ VÀO CHO VAY DH VÀ CHO VAY VND DH CHO CN 217 - CN HUẾ

In [60]:
# Loại bỏ 300 tỷ chiết khấu SME của CN 217 
q.loc[q.CN=="'217","SME_CK_DL"]+=-300
# CỘNG THÊM 300 TỶ VÀO SỐ CHO VAY VNDDH VÀ CHO VAY DH CỦA SME CỦA CN 217(CTCP QUAN LY VA PHA TRIEN BDS CONASI)
q.loc[q.CN=="'217","CVSMEDH"]+=300
q.loc[q.CN=="'217","CVSMEVNDDH"]+=300

# THÊM CỘT CÁC CỘT TỔNG CHO VAY

In [61]:
# thêm cột trái phiếu ghi nhận cho khối SME... chỉ trừ mã 166
q['SME_DTCK']=q['DTCK']
q.loc[q.CN=="'166",'SME_DTCK']=0
# thêm cột trái phiếu ghi nhận cho khối CIB ... chỉ lấy mã 166
q['CIB_DTCK']=0
q.loc[q.CN=="'166",'CIB_DTCK']=q.loc[q.CN=="'166",'DTCK'].values[0]

In [62]:
# loại bỏ 45 tỷ trái phiếu của CN 001 của khối SME 
q.loc[q.CN=="'001",'SME_DTCK']=0

In [63]:
# thêm các cột về tổng cho vay VND cho từng khối
q['CVSMEVND']=q['CVSMEVNDNH']+q['CVSMEVNDTH']+q['CVSMEVNDDH']
q['CVCIBVND']=q['CVCIBVNDNH']+q['CVCIBVNDTH']+q['CVCIBVNDDH']
q['CVCNVND']=q['CVCNVNDNH']+q['CVCNVNDTH']+q['CVCNVNDDH']


In [64]:
#  cho vay SME đã bao gồm Trái phiếu
q['CVSME_TP']=q['CVSMEVND']+q['CVSMESJC']+q['CVSMEUSD']+q['CVSMEEUR']+q['CVSMEKHAC']+q['SME_DTCK']
# cho vay CIB đã bao gồm trái phiếu
q['CVCIB_TP']=q['CVCIBVND']+q['CVCIBSJC']+q['CVCIBUSD']+q['CVCIBEUR']+q['CVCIBKHAC']+q['CIB_DTCK']
# cho vay CN
q['CVCN_TP']=q['CVCNVND']+q['CVCNSJC']+q['CVCNUSD']+q['CVCNEUR']+q['CVCNKHAC']


In [65]:
# Cho vay SME đã bao gồm trái phiếu + 30% UPAS LC
q['CVSME_UPDATED']=q['CVSME_TP']+q['UPAS_LC_SME_30']
# Cho vay CIB đã bao gồm trái phiếu + 30% UPAS LC
q['CVCIB_UPDATED']=q['CVCIB_TP']+q['UPAS_LC_CIB_30']
# Cho vay CN + 30% UPAS LC
q['CVCN_UPDATED']=q['CVCN_TP']+q['UPAS_LC_CN_30']

In [66]:
# thêm các cột dư nợ thẻ (tổng từ ba khối tương ứng loại tiền và loại kỳ hạn)
q['THE_CVVNDNH']=q['THE_CVSMEVNDNH'] + q['THE_CVCIBVNDNH'] + q['THE_CVCNVNDNH']
q['THE_CVVNDTH']=q['THE_CVSMEVNDTH'] + q['THE_CVCIBVNDTH'] + q['THE_CVCNVNDTH']
q['THE_CVVNDDH']=q['THE_CVSMEVNDDH'] + q['THE_CVCIBVNDDH'] + q['THE_CVCNVNDDH']

q['THE_CVSJC']=q['THE_CVSMESJC'] + q['THE_CVCIBSJC'] + q['THE_CVCNSJC']
q['THE_CVUSD']=q['THE_CVSMEUSD'] + q['THE_CVCIBUSD'] + q['THE_CVCNUSD']
q['THE_CVEUR']=q['THE_CVSMEEUR'] + q['THE_CVCIBEUR'] + q['THE_CVCNEUR']
q['THE_CVKHAC']=q['THE_CVSMEKHAC'] + q['THE_CVCIBKHAC'] + q['THE_CVCNKHAC']

q['THE_CVVND']=q['THE_CVVNDNH']+q['THE_CVVNDTH']+q['THE_CVVNDDH']
q['THE_CVNH']=q['THE_CVSMENH']+q['THE_CVCIBNH']+q['THE_CVCNNH']
q['THE_CVTH']=q['THE_CVSMETH']+q['THE_CVCIBTH']+q['THE_CVCNTH']
q['THE_CVDH']=q['THE_CVSMEDH']+q['THE_CVCIBDH']+q['THE_CVCNDH']
q['THE_CV']=q['THE_CVVND']+q['THE_CVSJC']+q['THE_CVUSD']+q['THE_CVEUR']+q['THE_CVKHAC']



In [67]:
# CVNGAYBC = "Cho vay bao gồm CC STK (bao gồm TPDN)" của 3 khối khác và cột dư nợ thẻ ngày báo cáo
q['CVNGAYBC_TP']=q['CVSME_TP']+q['CVCIB_TP']+q['CVCN_TP']+q['THE_CVNGAYBC']
# CVNGAYBC_UPDATED = "Cho vay bao gồm CC STK (gồm TPDN)+30%UPAS_LC"
q['CVNGAYBC_UPDATED']=q['CVSME_UPDATED']+q['CVCIB_UPDATED']+q['CVCN_UPDATED']+q['THE_CV']


In [68]:
# thêm các cột về chiết khấu = chiết khẩu của từng khối
q['CK_DL']=q['SME_CK_DL']+q['CIB_CK_DL']+q['CN_CK_DL']

# tạo thêm các cột tổng dư thẻ theo từng nhóm nợ
q['THE_N1_DL']=q['THE_SME_N1_DL']+q['THE_CIB_N1_DL']+q['THE_CN_N1_DL']
q['THE_N2_DL']=q['THE_SME_N2_DL']+q['THE_CIB_N2_DL']+q['THE_CN_N2_DL']
q['THE_N3_DL']=q['THE_SME_N3_DL']+q['THE_CIB_N3_DL']+q['THE_CN_N3_DL']
q['THE_N4_DL']=q['THE_SME_N4_DL']+q['THE_CIB_N4_DL']+q['THE_CN_N4_DL']
q['THE_N5_DL']=q['THE_SME_N5_DL']+q['THE_CIB_N5_DL']+q['THE_CN_N5_DL']


In [69]:
# thêm các cột tổng nợ theo từng nhóm (bằng nợ 3 khối cộng lại)
q['N1_DL']=q['SME_N1_DL']+q['CIB_N1_DL']+q['CN_N1_DL']
q['N2_DL']=q['SME_N2_DL']+q['CIB_N2_DL']+q['CN_N2_DL']
q['N3_DL']=q['SME_N3_DL']+q['CIB_N3_DL']+q['CN_N3_DL']
q['N4_DL']=q['SME_N4_DL']+q['CIB_N4_DL']+q['CN_N4_DL']
q['N5_DL']=q['SME_N5_DL']+q['CIB_N5_DL']+q['CN_N5_DL']


In [70]:
# TRÁI PHIẾU DOANH NGHIỆP CỦA PHÒNG ĐẦU TƯ
# Trái phiếu của phòng đầu tư từ file trái phiếu ghi nhận tay
a=TraiPhieuPDT

# TRÁI PHIẾU PHÒNG ĐẦU TƯ TRÊN HỆ THỐNG
# lưu file vào biến alm
gl101=pd.read_excel(info_filename,sheet_name='gl101')
# chuyển cột SBV_CODE thành dạng object
gl101.SBV_CODE=gl101.SBV_CODE.astype(str)
# lọc Branch_no '379, SBV_CODE startswith 153 163 1413,
df=gl101[(gl101.BRANCH_NO=="'379")&(gl101.SBV_CODE.str.startswith(('153','163','1413')))]
# con số thứ 2 = tổng DR_CLOS - tổng CR_CLOS
b=df.DR_CLOS.sum()-df.CR_CLOS.sum()
b=b/billion

# con số thứ 3 lọc Branch_no '001, SBV_CODE startswith 153 163 1413, tổng DR_CLOS - tổng CR_CLOS
# lọc Branch_no '001, SBV_CODE startswith 153 163 1413
df=gl101[(gl101.BRANCH_NO=="'001")&(gl101.SBV_CODE.str.startswith(('153','163','1413')))]
# tổng DR_CLOS - tổng CR_CLOS
c=df.DR_CLOS.sum()-df.CR_CLOS.sum()
c=c/billion

# giá trị ô BK 1 (trái phiếu phòng đầu tư)
BK1=a+b+c
print(f'Trái phiếu phòng Đầu tư từ trái phiếu cộng tay: {a} \nTrái phiếu phòng Đầu tư trên hệ thống: {b} \nTrái phiều ở phòng nguồn vốn: {c} \nTổng trái phiếu phòng đầu tư: {BK1}')

Trái phiếu phòng Đầu tư từ trái phiếu cộng tay: 119.6 
Trái phiếu phòng Đầu tư trên hệ thống: 913.422025979 
Trái phiều ở phòng nguồn vốn: 45.0 
Tổng trái phiếu phòng đầu tư: 1078.0220259789999


In [71]:
# 45 tỷ trái phiếu không cập nhập vào cột trái phiếu SME của từng đơn vị ... (45 tỷ trái phiếu nguồn vốn)
q.loc[q.CN=="'001",'SME_DTCK']=0
# cập nhập 1594 trái phiếu phòng đầu tư và 45 tỷ trái phiếu của nguồn vốn vào cho vay của CN 001
q.loc[q.CN=="'001",'CVNGAYBC_TP']+=(BK1+BK2)
# Cho vay bao gồm CC STK (gồm TPDN)+30%UPAS_LC: + số cho vay của Muadee vào CN 001
q.loc[q.CN=="'001",'CVNGAYBC_UPDATED']+=(BK1+BK2)

In [72]:
# thêm cột Cho vay (không bao gồm chiết khấu)
q['Cho vay (không bao gồm chiết khấu)']=q['CVSME_TP']+q['CVCIB_TP']+q['CVCN_TP']+q['THE_CVNGAYBC']


In [73]:
# cập nhập cho vay không bao gồm chiết khấu của CN 001 thêm số của ô BK 1
q.loc[q.CN=="'001",'Cho vay (không bao gồm chiết khấu)']+=(BK1)

# ĐỐI CHIẾU TỔNG CHO VAY VS B02

In [74]:
ChoVayKH=check_df.iloc[24,3] # cho vay khách hàng TK2
TraiPhieuChoVay # trái phiếu doanh nghiệp cộng tay
TPDN # trái phiếu hệ thống
UpasLc30=(q['UPAS_LC_SME_30'].sum()+q['UPAS_LC_CIB_30'].sum()+q['UPAS_LC_CN_30'].sum())*billion
TongUyThac=UyThac1+UyThac2
print(f'TK2 (cho vay khách hàng): {ChoVayKH}\nTPDN (hệ thống): {TPDN}\n30% UPAS LC: {UpasLc30}\nTrái Phiếu Cho Vay Cộng tay: {TraiPhieuChoVay}\nỦy thác: {TongUyThac}')

TK2 (cho vay khách hàng): 274543019196154.0
TPDN (hệ thống): 6493797662702.0
30% UPAS LC: 10773899281391.639
Trái Phiếu Cho Vay Cộng tay: 228.0
Ủy thác: 3832760340966.0


In [75]:
f1=ChoVayKH+TPDN+UpasLc30+TraiPhieuChoVay+TongUyThac
print(f'Tổng: {f1}')

Tổng: 295643476481441.6


In [76]:
list=["'379","'668"] # loại bỏ số cho vay của Muadee và của Phòng Đầu tư
ChoVayBCNgay=q.loc[~q.CN.isin(list),'CVNGAYBC_UPDATED'].sum()*billion
f2=ChoVayBCNgay
print(f'Tổng Cho Vay trên Báo cáo ngày: {ChoVayBCNgay}')

Tổng Cho Vay trên Báo cáo ngày: 296171476481200.7


In [77]:
print('Độ lệch của Cho vay của báo cáo ngày so với B02: (đồng)')
f1-f2

Độ lệch của Cho vay của báo cáo ngày so với B02: (đồng)


-527999999759.0625

# CẬP NHẬP MUA BÁN NỢ VÀ THÊM SỐ LIỆU NĂM 2022

In [78]:
MBN_df=pd.read_excel(info_filename,sheet_name='MBN')
df=MBN_df.copy()
# loại bỏ file không ghi nhận trong danh sách, ở cột ghi chú
df=df[~df['ghi chu'].notnull()]
# chuyển đơn vị thành tỷ đồng
df['DU NO CON LAI']=df['DU NO CON LAI']/billion
# loại bỏ các dấu "'" trong cột loại KH 208
df['Loại KH 208']=df['Loại KH 208'].str.replace("'","")

In [79]:
CN_df=df[df['Loại KH 208']=="DIN"].groupby('ma don vi')['DU NO CON LAI'].sum().reset_index().rename(columns={'DU NO CON LAI':'BanNo_CN'})
CMB_df=df[df['Loại KH 208']=="DCO"].groupby('ma don vi')['DU NO CON LAI'].sum().reset_index().rename(columns={'DU NO CON LAI':'BanNo_SME'})

In [80]:
q=q.merge(CMB_df,how='left',left_on='CN',right_on='ma don vi').fillna(0).drop(columns='ma don vi') # nối cột bán nợ CMB vào 
q=q.merge(CN_df,how='left',left_on='CN',right_on='ma don vi').fillna(0).drop(columns='ma don vi') # nối cột bán nợ CN vào
q['BanNo_CIB']=0
q['BanNo_The']=0

In [81]:
df=MBN_df.copy()
# loại bỏ file không ghi nhận trong danh sách, ở cột ghi chú
# df=df[~df['ghi chu'].notnull()]
# chuyển đơn vị thành tỷ đồng
df['DU NO CON LAI']=df['DU NO CON LAI']/billion
# loại bỏ các dấu "'" trong cột loại KH 208
df['Loại KH 208']=df['Loại KH 208'].str.replace("'","")
df=df.groupby('ma don vi')['DU NO CON LAI'].sum().reset_index().rename(columns={'DU NO CON LAI':'BanNo'})

In [82]:
df=MBN_df.copy()
# loại bỏ file không ghi nhận trong danh sách, ở cột ghi chú
# df=df[~df['ghi chu'].notnull()]
# chuyển đơn vị thành tỷ đồng
df['DU NO CON LAI']=df['DU NO CON LAI']/billion
# loại bỏ các dấu "'" trong cột loại KH 208
df['Loại KH 208']=df['Loại KH 208'].str.replace("'","")
df=df.groupby('ma don vi')['DU NO CON LAI'].sum().reset_index().rename(columns={'DU NO CON LAI':'BanNo'})
q=q.merge(df,how='left',left_on='CN',right_on='ma don vi').fillna(0).drop(columns='ma don vi')


In [83]:
# THÊM CỘT SỐ LIỆU 2022
# đọc file info sheet 2022
df=pd.read_excel(info_filename,sheet_name='2022')
# bỏ cột STT
df=df.drop(columns=('STT'))
# thêm các số liệu 2022 theo từng mã đơn vị
q=q.merge(df,left_on='CN',right_on='MDV',how='left').fillna(0)
# bỏ cột MDV 
q=q.drop(columns=('MDV'))

In [84]:
# Nợ 345 của SME đã bao gồm bán nợ
q['SME_N345_DL_UPDATED']=q['SME_N3_DL']+q['SME_N4_DL']+q['SME_N5_DL']+q['BanNo_SME']
q['CIB_N345_DL_UPDATED']=q['CIB_N3_DL']+q['CIB_N4_DL']+q['CIB_N5_DL']+q['BanNo_CIB']
q['CN_N345_DL_UPDATED']=q['CN_N3_DL']+q['CN_N4_DL']+q['CN_N5_DL']+q['BanNo_CN']

In [85]:
# Nợ 345 của SME đã bao gồm bán nợ
q['SME_N345_DL_UPDATED']=q['SME_N3_DL']+q['SME_N4_DL']+q['SME_N5_DL']+q['BanNo_SME']
q['CIB_N345_DL_UPDATED']=q['CIB_N3_DL']+q['CIB_N4_DL']+q['CIB_N5_DL']+q['BanNo_CIB']
q['CN_N345_DL_UPDATED']=q['CN_N3_DL']+q['CN_N4_DL']+q['CN_N5_DL']+q['BanNo_CN']

# thể tổng N345
q['THE_N345_DL_UPDATED']=q['THE_N3_DL'] + q['THE_N4_DL'] + q['THE_N5_DL']+q['BanNo_The']

# cột tổng nợ nhóm 345 bao gồm bán nợ 
q['SME_N345_DL_UPDATED']=q['SME_N3_DL']+q['SME_N4_DL']+q['SME_N5_DL']+q['BanNo_SME']
q['CIB_N345_DL_UPDATED']=q['CIB_N3_DL']+q['CIB_N4_DL']+q['CIB_N5_DL']+q['BanNo_CIB']
q['CN_N345_DL_UPDATED']=q['CN_N3_DL']+q['CN_N4_DL']+q['CN_N5_DL']+q['BanNo_CN']

# Tổng nợ nhóm 345
q['N345_DL']=q['SME_N345_DL_UPDATED']-q['BanNo_SME']+q['CIB_N345_DL_UPDATED']+q['CN_N345_DL_UPDATED']-q['BanNo_CN']+q['THE_N345_DL_UPDATED']

# Tổng nợ 345 sau khi chỉnh các món điều chuyển 
q['N345_SauDieuChuyen']=q['N345_DL']+q['N_DieuChuyen']+q['BanNo']


# CẬP NHẬP LỢI NHUẬN CỦA CN 001

In [86]:
# LỢI NHUẬN LŨY KẾ TẠM TÍNH CỦA 001 TRỤ SỞ CHÍNH = TỔNG 001+179+322+323+324+379+666+668
q.loc[q.CN=="'001",'LNTRONGTHANG']+=(
    q.loc[q.CN=="'179",'LNTRONGTHANG'].values[0]+
    q.loc[q.CN=="'322",'LNTRONGTHANG'].values[0]+
    q.loc[q.CN=="'323",'LNTRONGTHANG'].values[0]+
    q.loc[q.CN=="'379",'LNTRONGTHANG'].values[0]+
    q.loc[q.CN=="'666",'LNTRONGTHANG'].values[0]+
    q.loc[q.CN=="'668",'LNTRONGTHANG'].values[0])
# LỢI NHUẬN LŨY KẾ TRONG THÁNG CỦA 001 TSC = TỔNG 001+179+322+323+324+666+668
q.loc[q.CN=="'001",'LNLUYKE']+=(
    q.loc[q.CN=="'179",'LNLUYKE'].values[0]+
    q.loc[q.CN=="'322",'LNLUYKE'].values[0]+
    q.loc[q.CN=="'323",'LNLUYKE'].values[0]+
    q.loc[q.CN=="'379",'LNLUYKE'].values[0]+
    q.loc[q.CN=="'666",'LNLUYKE'].values[0]+
    q.loc[q.CN=="'668",'LNLUYKE'].values[0])


In [87]:
# xóa các dòng dữ liệu bị trùng
q=q.drop_duplicates()


# THÊM CÁC SỐ LIỆU KẾ HOẠCH VÀ THỰC HIỆN

In [88]:
# lấy số liệu KẾ HOẠCH từ sheet KH file info
df1=pd.read_excel(info_filename,sheet_name='KH')
# find the index of the cell containing "Mã"
row_index, col_index = df1.eq('Mã').stack().idxmax()
df1=pd.read_excel(info_filename,sheet_name='KH',skiprows=row_index+1)
# loại bỏ các cột "unamed"


In [89]:
# lấy số liệu KẾ HOẠCH từ sheet KH file info
df1=df1[(df1['Mã']!='000')&df1['STT'].notnull()]
# chuyển thành dạng 3 kí tự
df1['Mã'] ="'"+df1['Mã'].astype(int).astype(str).str.zfill(3)

# loại bỏ các cột "unamed"
unnamed_cols = [col for col in df1.columns if 'Unnamed' in col]
df1 = df1.drop(columns=unnamed_cols)

# loại bỏ 2 cột stt và tên đơn vị
df1=df1.drop(columns=(['STT','Tên đơn vị']))

In [90]:
# lấy số liệu THỰC HIỆN từ sheet TH file info
df2=pd.read_excel(info_filename,sheet_name='TH')
# find the index of the cell containing "Mã"
row_index, col_index = df2.eq('Mã').stack().idxmax()

df2=pd.read_excel(info_filename,sheet_name='TH',skiprows=row_index+1)

# đổi mã thành dạng 3 kí tự + "'"
df2['Mã'] ="'"+df2['Mã'].astype(int).astype(str).str.zfill(3)

# loại bỏ các cột "unamed"
unnamed_cols = [col for col in df2.columns if 'Unnamed' in col]
df2 = df2.drop(columns=unnamed_cols)
df2=df2[df2.Mã!="'000"]
# loại bỏ 2 cột stt và tên đơn vị
df2=df2.drop(columns=(['STT','Tên đơn vị']))

In [91]:
q1=q.merge(pd.merge(df1,df2,how='outer').fillna(0),left_on='CN',right_on='Mã')
q1=q1.drop(columns='Mã')

In [92]:
# ĐIỀU CHỈNH TAY LỢI NHUẬN Kế hoạch
q1.loc[q1.CN=="'001","LN kế hoạch"]=(-521.708558377528)
q1.loc[q1.CN=="'001","LN Tạm tính kế hoạch"]=95.6579242947089

# ĐIỀU CHỈNH TAY CHỈ TIÊU PHỤ

In [93]:
# LỢI NHUẬN LŨY KẾ TẠM TÍNH
q1['LN_LK_TT']=q1['LNTRONGTHANG'] + q1['LN Lũy kế']

In [94]:
# ĐIỀU CHỈNH TAY LỢI NHUẬN
q1.loc[q1.CN=="'001","LN_LK_TT"]+=(-101.2656*0.85-1.3375*0.7)

In [95]:
# thêm cột tổng huy động kế hoạch tháng
q1['H_KH']=q1['HĐ - CMB kế hoạch']+q1['HĐ - CIB kế hoạch']+q1['HĐ - CN kế hoạch']
# thêm cột tổng cho vay theo kế hoạch tháng
q1['CV_KH']=q1['CV - CMB kế hoạch']+q1['CV - CIB kế hoạch']+q1['CV - CN kế hoạch']+q1['CV - THẺ kế hoạch']

In [96]:
# thêm cột HĐ CN  Tiền gửi thanh toán và cột HĐ CN % thực hiện kế hoạch
q1[f'HĐ CN (Tiền gửi thanh toán) {date_str}']=q1['HCNKKH']

In [97]:
# THÊM CỘT THU DỊCH VỤ
q1['Tổng thu dịch vụ kế hoạch tháng']=q1['Thu DV - CMB kế hoạch']+q1['Thu DV - CN kế hoạch']+q1['Thu DV - CIB kế hoạch']+q1['Thu DV - THẺ kế hoạch']-q1['DS BHNT - THẺ kế hoạch']

# THÊM CỘT THU KINH DOANH NGOẠI TỆ
q1['Tổng thu KDNT kế hoạch tháng']=q1['Thu KDNT - CMB kế hoạch']+q1['Thu KDNT - CN kế hoạch']+q1['Thu KDNT - CIB kế hoạch']

# THÊM CỘT DOANH SỐ MUA BÁN NGOẠI TỆ
q1['Tổng DS MBNT kế hoạch tháng']=q1['DS MBNT KHDN 2022 - CMB kế hoạch']+q1['DS MBNT - CIB kế hoạch']+q1['DS MBNT - CN kế hoạch']

# THÊM CỘT BẢO HIỂM NHÂN THỌ
q1['Tổng BHNT kế hoạch']=q1['BHNT - KHDN 2022 - CMB kế hoạch']+q1['DS BHNT - CIB kế hoạch']+q1['DS BHNT - CN kế hoạch']

# THÊM CỘT DOANH SỐ THANH TOÁN QUỐC TẾ
q1['DS TTQT - TH']=q1['TTQT - CMB']+q1['TTQT - CIB']
q1['Tổng DS TTQT - kế hoạch']=q1['TTQT KHDN 2022 - CMB kế hoạch']+q1['DS TTQT - CIB kế hoạch']

# THÊM CỘT THU HỘ TRÁI PHIẾU
q1['THU HỘ TP - CN']=q1['THU HỘ TP - CN']-q1['THU HỘ TP - CMB']-q1['THU HỘ TP - CIB']
q1['Tổng Thu Hộ TP - TH']=q1['THU HỘ TP - CMB']+q1['THU HỘ TP - CIB']+q1['THU HỘ TP - CN']
q1['Tổng Thu Hộ TP - kế hoạch']=q1['THU HỘ TRÁI PHIẾU - CMB kế hoạch']+q1['THU HỘ TRÁI PHIẾU - CN kế hoạch']+q1['THU HỘ TRÁI PHIẾU - CIB kế hoạch']


In [98]:
# ĐIỀU CHỈNH TAY CHỈ TIÊU PHỤ
q1.loc[q1.CN=="'001",'Tổng thu Dịch Vụ năm 2022']+=(-761.074079002)
q1.loc[q1.CN=="'001",'Thu DV - TH']+=(-557.06582473)
q1.loc[q1.CN=="'001",'Thu Dịch Vụ CMB năm 2022']+=(-650.529988676)
q1.loc[q1.CN=="'001",'Thu DV - CMB']+=(-480.976191529)
q1.loc[q1.CN=="'001",'Thu Dịch Vụ CIB năm 2022']+=(-104.990077503)
q1.loc[q1.CN=="'001",'Thu DV - CIB']+=(-73.120084614)
q1.loc[q1.CN=="'001",'Thu Dịch Vụ CN năm 2022']+=(-5.554012823)
q1.loc[q1.CN=="'001",'Thu DV - CN']+=(-2.969548587)


In [99]:
q1=q1.drop_duplicates()

# TÍNH SỐ LIỆU KHU VỰC LOẠI 1, KHU VỰC LOẠI 2, KHU VỰC LOẠI 3 VÀ CỤM ĐƠN VỊ

In [100]:
df=pd.read_excel(info_filename,sheet_name='DS Cum Va Khu Vuc')

# MÃ CỤM VÀ MÃ ĐƠN VỊ
df[df.CUM.notnull()][['MDV','CUM']]
# gộp tổng số liệu các nhóm đơn vị theo từng cụm
CUM=df[df.CUM.notnull()][['MDV','CUM']].merge(
    q1,how='outer',left_on='MDV',right_on='CN').groupby('CUM').sum().reset_index().rename(columns={'CUM':'CN'})

# MÃ ĐƠN VỊ VÀ MÃ KHU VỰC LOẠI 1
df[df['KV TYPE 1'].notnull()][['MDV','KV TYPE 1']]
# gộp tổng số liệu các nhóm đơn vị theo từng khu vực loại 1
KV_TYPE_1=df[df['KV TYPE 1'].notnull()][['MDV','KV TYPE 1']].merge(
    q1,how='outer',left_on='MDV',right_on='CN').groupby(['KV TYPE 1']).sum().reset_index().rename(columns={'KV TYPE 1':'CN'})

# MÃ ĐƠN VỊ VÀ MÃ KHU VỰC LOẠI 2
df[df['KV TYPE 2'].notnull()][['MDV','KV TYPE 2']]
# gộp tổng số liệu các nhóm đơn vị theo từng khu vực loại 2
KV_TYPE_2=df[df['KV TYPE 2'].notnull()][['MDV','KV TYPE 2']].merge(
    q1,how='outer',left_on='MDV',right_on='CN').groupby(['KV TYPE 2']).sum().reset_index().rename(columns={'KV TYPE 2':'CN'})

# MÃ ĐƠN VỊ VÀ MÃ KHU VỰC LOẠI 3
df[df['KV TYPE 3'].notnull()][['MDV','KV TYPE 3']]
# gộp tổng số liệu các nhóm đơn vị theo từng khu vực loại 3
KV_TYPE_3=df[df['KV TYPE 3'].notnull()][['MDV','KV TYPE 3']].merge(
    q1,how='outer',left_on='MDV',right_on='CN').groupby(['KV TYPE 3']).sum().reset_index().rename(columns={'KV TYPE 3':'CN'})

# MÃ ĐƠN VỊ : MÃ TOÀN HÀNG
df[df['TOÀN HÀNG'].notnull()][['MDV','TOÀN HÀNG']]
# gộp tổng số liệu các nhóm đơn vị theo toàn hàng
TOAN_HANG=df[df['TOÀN HÀNG'].notnull()][['MDV','TOÀN HÀNG']].merge(
    q1,how='outer',left_on='MDV',right_on='CN').groupby(['TOÀN HÀNG']).sum().reset_index().rename(columns={'TOÀN HÀNG':'CN'})

# gộp 4 bảng dữ liệu lại với nhau
BC_df=pd.concat([TOAN_HANG,q1,CUM,KV_TYPE_1,KV_TYPE_2,KV_TYPE_3])

C:\Users\phatdd3\AppData\Local\Temp\ipykernel_18656\1424451507.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  q1,how='outer',left_on='MDV',right_on='CN').groupby('CUM').sum().reset_index().rename(columns={'CUM':'CN'})
C:\Users\phatdd3\AppData\Local\Temp\ipykernel_18656\1424451507.py:13: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  q1,how='outer',left_on='MDV',right_on='CN').groupby(['KV TYPE 1']).sum().reset_index().rename(columns={'KV TYPE 1':'CN'})
C:\Users\phatdd3\AppData\Local\Temp\ipykernel_18656\1424451507.py:19: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecate

In [101]:

# CẬP NHẬP LẠI TỔNG TÀI SẢN CỦA TOÀN HÀNG THEO FILE B02
# cập nhập lại số liệu tổng tài sản của toàn hàng
BC_df.loc[BC_df.CN=="'000",'TS_CO']=TongTaiSanCapNhap

# MÃ VÀ SỐ THỨ TỰ HIỂN THỊ VÀ TÊN ĐƠN VỊ

In [102]:
df=pd.read_excel(info_filename,sheet_name='DS Cum Va Khu Vuc')

# MÃ ĐƠN VỊ VÀ TÊN HIỂN THỊ 
df[['STT','MDV','Tên đơn vị']].rename(columns={'MDV':"CN"})

# thêm cột tên đơn vị theo từng mã đơn vị tương ứng
BC_df=df[['STT','MDV','Tên đơn vị']].rename(columns={'MDV':"CN"}).merge(BC_df).sort_values(by='STT')


In [103]:
# xóa các dòng bị lặp lại và sắp xếp lại theo stt
BaoCaoNgay_df=BC_df.drop_duplicates().sort_values(by='STT')

# ĐIỀU CHỈNH TAY CHỈ TIÊU

In [104]:
# ĐIỀU CHỈNH TAY LỢI NHUẬN

# ĐIỀU CHỈNH LỢI NHUẬN NĂM 2022 (cột FY)
A=+145.003*0.85+(70-15)*0.85+30*0.85+70*0.85+65*0.85+100*0.85-15*0.85
B=+2*0.7
BaoCaoNgay_df.loc[BaoCaoNgay_df.CN=="'000","LN_2022"]+=(A+B)
BaoCaoNgay_df.loc[BaoCaoNgay_df.CN=="A","LN_2022"]+=(A)
BaoCaoNgay_df.loc[BaoCaoNgay_df.CN=="B","LN_2022"]+=(B)
BaoCaoNgay_df.loc[BaoCaoNgay_df.CN=="KV1","LN_2022"]+=(A)
BaoCaoNgay_df.loc[BaoCaoNgay_df.CN=="KV4","LN_2022"]+=(B)

# ĐIỀU CHỈNH LỢI NHUẬN CỘT lN Phát sinh (cột GB)
A=(-1.3375*0.7)
BaoCaoNgay_df.loc[BaoCaoNgay_df.CN=="A","LN Phát sinh"]+=(A+0.9)
BaoCaoNgay_df.loc[BaoCaoNgay_df.CN=="B","LN Phát sinh"]+=(-A-0.9)
BaoCaoNgay_df.loc[BaoCaoNgay_df.CN=="KV4","LN Phát sinh"]+=(-A-0.9)
BaoCaoNgay_df.loc[BaoCaoNgay_df.CN=="'001","LN Phát sinh"]+=(A+0.9)


# ĐIỀU CHỈNH LỢI NHUẬN CỘT LN lũy kế (cột GC)
A=101.2655*0.85
B=1.3375*0.7
BaoCaoNgay_df.loc[BaoCaoNgay_df.CN=="A","LN Lũy kế"]-=(B+0.9+0.9-26.7-24.9)
BaoCaoNgay_df.loc[BaoCaoNgay_df.CN=="B","LN Lũy kế"]+=(B-0.9+0.9)
BaoCaoNgay_df.loc[BaoCaoNgay_df.CN=="KV1","LN Lũy kế"]+=(A)
BaoCaoNgay_df.loc[BaoCaoNgay_df.CN=="KV4","LN Lũy kế"]+=(B-0.9+0.9)
BaoCaoNgay_df.loc[BaoCaoNgay_df.CN=="'001","LN Lũy kế"]-=(A+B+0.9+0.9-26.7-24.9)

# cập nhập lại lợi nhuận kế hoạch tháng của CN 001 (cột GD)
A=194/12*4
BaoCaoNgay_df.loc[BaoCaoNgay_df.CN=="'000","LN kế hoạch"]+=(A-104.6+120.8-124.1)
BaoCaoNgay_df.loc[BaoCaoNgay_df.CN=="A","LN kế hoạch"]+=(A-104.6+120.8-124.1)
BaoCaoNgay_df.loc[BaoCaoNgay_df.CN=="KV1","LN kế hoạch"]+=(A+16.2+16.1)
BaoCaoNgay_df.loc[BaoCaoNgay_df.CN=="'001","LN kế hoạch"]+=(-124.1-16.1)

# cập nhập lại lợi nhuận tạm tính kế hoạch (cột GH)
BaoCaoNgay_df.loc[BaoCaoNgay_df.CN=="'000","LN Tạm tính kế hoạch"]+=0.2
BaoCaoNgay_df.loc[BaoCaoNgay_df.CN=="A","LN Tạm tính kế hoạch"]+=0
BaoCaoNgay_df.loc[BaoCaoNgay_df.CN=="'689","LN Tạm tính kế hoạch"]+=0.2

# cập nhập lợi nhuận LN tạm tính lũy kế từ đầu năm đến (cột GG)
BaoCaoNgay_df.loc[BaoCaoNgay_df.CN=="'000","LN_LK_TT"]+=(0)
BaoCaoNgay_df.loc[BaoCaoNgay_df.CN=="A","LN_LK_TT"]+=(0)
BaoCaoNgay_df.loc[BaoCaoNgay_df.CN=="'001","LN_LK_TT"]+=(0)

# điều chỉnh tay
# cập nhập lại tổng thu dịch vụ năm 2022 (cột GW) 
A=+145.003*0.85+(70-15)*0.85+30*0.85+70*0.85+65*0.85+100*0.85-15*0.85
B=2*0.7
BaoCaoNgay_df.loc[BaoCaoNgay_df.CN=="'000",'Tổng thu Dịch Vụ năm 2022']+=(A+B)
BaoCaoNgay_df.loc[BaoCaoNgay_df.CN=="A",'Tổng thu Dịch Vụ năm 2022']+=(A)
BaoCaoNgay_df.loc[BaoCaoNgay_df.CN=="B",'Tổng thu Dịch Vụ năm 2022']+=(B)
BaoCaoNgay_df.loc[BaoCaoNgay_df.CN=="KV1",'Tổng thu Dịch Vụ năm 2022']+=(A)
BaoCaoNgay_df.loc[BaoCaoNgay_df.CN=="KV4",'Tổng thu Dịch Vụ năm 2022']+=(B)

# điều chỉnh tay
# cập nhập lại tổng thu dịch vụ thực hiện (cột GX)
A=101.2655*0.85-158.8
B=1.3375*0.7
C=-158.8
BaoCaoNgay_df.loc[BaoCaoNgay_df.CN=="'000",'Thu DV - TH']+=(A+B-164.3-161.3)
BaoCaoNgay_df.loc[BaoCaoNgay_df.CN=="A",'Thu DV - TH']+=(A-164.3)
BaoCaoNgay_df.loc[BaoCaoNgay_df.CN=="B",'Thu DV - TH']+=(B)
BaoCaoNgay_df.loc[BaoCaoNgay_df.CN=="KV1",'Thu DV - TH']+=(A-C)
BaoCaoNgay_df.loc[BaoCaoNgay_df.CN=="KV4",'Thu DV - TH']+=(B)
BaoCaoNgay_df.loc[BaoCaoNgay_df.CN=="'001",'Thu DV - TH']+=(C-164.3-161.3)

# cập nhập lại tổng thu dịch vụ kế hoạch tháng (cột GY):
A=194/12*4+16.2+16.1
BaoCaoNgay_df.loc[BaoCaoNgay_df.CN=="'000",'Tổng thu dịch vụ kế hoạch tháng']+=(A)
BaoCaoNgay_df.loc[BaoCaoNgay_df.CN=="A",'Tổng thu dịch vụ kế hoạch tháng']+=(A)
BaoCaoNgay_df.loc[BaoCaoNgay_df.CN=="KV1",'Tổng thu dịch vụ kế hoạch tháng']+=(A)

# cập nhập lại tổng thu dịch vụ SME năm 2022 (cột HA):
A=+145.003*0.85+(70-15)*0.85+30*0.85+70*0.85+65*0.85+100*0.85-15*0.85
B=+2*0.7
BaoCaoNgay_df.loc[BaoCaoNgay_df.CN=="'000",'Thu Dịch Vụ CMB năm 2022']+=(A+B)
BaoCaoNgay_df.loc[BaoCaoNgay_df.CN=="A",'Thu Dịch Vụ CMB năm 2022']+=(A)
BaoCaoNgay_df.loc[BaoCaoNgay_df.CN=="B",'Thu Dịch Vụ CMB năm 2022']+=(B)
BaoCaoNgay_df.loc[BaoCaoNgay_df.CN=="KV1",'Thu Dịch Vụ CMB năm 2022']+=(A)
BaoCaoNgay_df.loc[BaoCaoNgay_df.CN=="KV4",'Thu Dịch Vụ CMB năm 2022']+=(B)

# cập nhập lại thu dịch vụ SME (cột HB):
A=+101.2656*0.85-136.5
B=+1.3375*0.7
BaoCaoNgay_df.loc[BaoCaoNgay_df.CN=="'000",'Thu DV - CMB']+=(A+B-143.6-135.9)
BaoCaoNgay_df.loc[BaoCaoNgay_df.CN=="A",'Thu DV - CMB']+=(A-143.6)
BaoCaoNgay_df.loc[BaoCaoNgay_df.CN=="B",'Thu DV - CMB']+=(B)
BaoCaoNgay_df.loc[BaoCaoNgay_df.CN=="KV1",'Thu DV - CMB']+=(A+158.8-22.3)
BaoCaoNgay_df.loc[BaoCaoNgay_df.CN=="KV4",'Thu DV - CMB']+=(B)
BaoCaoNgay_df.loc[BaoCaoNgay_df.CN=="'001",'Thu DV - CMB']+=(-158.8+22.3-143.6-135.9)

# cập nhập lại thu dịch vụ CIB (cột HF):
A=-22.3-20.7-25.4
list=["'000","A","'001"]
for i in list:
    BaoCaoNgay_df.loc[BaoCaoNgay_df.CN==i,'Thu DV - CIB']+=(A)

# cập nhập lại thu dịch vụ SME kế hoạch tháng (cột HC):
A=+194/12*4+16.2+16.1
BaoCaoNgay_df.loc[BaoCaoNgay_df.CN=="'000",'Thu DV - CMB kế hoạch']+=A
BaoCaoNgay_df.loc[BaoCaoNgay_df.CN=="A",'Thu DV - CMB kế hoạch']+=(A)
BaoCaoNgay_df.loc[BaoCaoNgay_df.CN=="KV1",'Thu DV - CMB kế hoạch']+=(A)

# cập nhập lại thu dịch vụ SME kế hoạch tháng (cột HC):
A=115.3+16.2
list=["'000","A","KV6","'166"]
for i in list:
    BaoCaoNgay_df.loc[BaoCaoNgay_df.CN==i,'THU HỘ TP - CN']+=A
A=16.1
list=["'000","A","KV1"]
for i in list:
    BaoCaoNgay_df.loc[BaoCaoNgay_df.CN==i,'THU HỘ TP - CN']+=A

# cập nhập lại thu hộ TP thực hiện (cột KI):
A=115.3-115.3
list=["'000","A","KV6"]
for i in list:
    BaoCaoNgay_df.loc[BaoCaoNgay_df.CN==i,'Tổng Thu Hộ TP - TH']+=A

# comeback
# cập nhập lại thu hộ TP - CN thực hiện (cột KQ):
A=115.3-115.3
list=["'000","A","KV6"]
for i in list:
    BaoCaoNgay_df.loc[BaoCaoNgay_df.CN==i,'THU HỘ TP - CN']+=A
A=-131.5
list=["'166","A","KV6","'000"]
for i in list:
    BaoCaoNgay_df.loc[BaoCaoNgay_df.CN==i,'THU HỘ TP - CN']+=A
A=-16.1
list=["'000","A","KV1"]
for i in list:
    BaoCaoNgay_df.loc[BaoCaoNgay_df.CN==i,'THU HỘ TP - CN']+=A


# THÊM CÁC CỘT SỐ LIỆU TỈ LỆ %

In [105]:
# tỉ lệ phần trăm thực hiện kế hoạch
BaoCaoNgay_df['HNGAYBC_PR'] = ((BaoCaoNgay_df['HNGAYBC'] / BaoCaoNgay_df['H_KH']).fillna(0)).round(2)

# tỉ lệ phần trăm thực hiện kế hoạch
BaoCaoNgay_df['CVNGAYBC_KH_PR'] = ((BaoCaoNgay_df['CVNGAYBC_UPDATED'] / BaoCaoNgay_df['CV_KH']).fillna(0)).round(2)
# % thực hiện kế hoạch thẻ
BaoCaoNgay_df['THE_CV_PR'] = ((BaoCaoNgay_df['THE_CVNGAYBC'] / BaoCaoNgay_df['CV - THẺ kế hoạch']).fillna(0)).round(2)
# thêm các cột % thực hiện kế hoạch cho từng khối của huy động (Plan Ratio/Progress Ratio)
BaoCaoNgay_df['HSME_PR'] = ((BaoCaoNgay_df['HSME'] / BaoCaoNgay_df['HĐ - CMB kế hoạch']).fillna(0)).round(2)
BaoCaoNgay_df['HCIB_PR'] = ((BaoCaoNgay_df['HCIB'] / BaoCaoNgay_df['HĐ - CIB kế hoạch']).fillna(0)).round(2)
BaoCaoNgay_df['HCN_PR'] = ((BaoCaoNgay_df['HCN'] / BaoCaoNgay_df['HĐ - CN kế hoạch']).fillna(0)).round(2)
# thêm các cột % thực hiện kế hoạch cho từng khối của cho vay (Plan Ratio/ProgressRatio)
BaoCaoNgay_df['CVSME_PR'] = ((BaoCaoNgay_df['CVSME_UPDATED'] / BaoCaoNgay_df['CV - CMB kế hoạch']).fillna(0)).round(2)
BaoCaoNgay_df['CVCIB_PR'] = ((BaoCaoNgay_df['CVCIB_UPDATED'] / BaoCaoNgay_df['CV - CIB kế hoạch']).fillna(0)).round(2)
BaoCaoNgay_df['CVCN_PR'] = ((BaoCaoNgay_df['CVCN_UPDATED'] / BaoCaoNgay_df['CV - CN kế hoạch']).fillna(0)).round(2)

# thêm cột HĐ CN  Tiền gửi thanh toán và cột HĐ CN % thực hiện kế hoạch
BaoCaoNgay_df['HĐ CN (Tiền gửi thanh toán) % thực hiện so kế hoạch']=BaoCaoNgay_df[f'HĐ CN (Tiền gửi thanh toán) {date_str}']/BaoCaoNgay_df['HĐ KKH - CN kế hoạch']
# Thẻ tỷ lệ N345/DN
BaoCaoNgay_df['N345_CV_PR_THE'] = ((BaoCaoNgay_df['THE_N345_DL_UPDATED'] / BaoCaoNgay_df['THE_CVNGAYBC']).fillna(0)).round(2)
# tỷ lệ nợ/ChoVay
BaoCaoNgay_df['N345_CV_PR']=((BaoCaoNgay_df['N345_DL']+BaoCaoNgay_df['N_DieuChuyen']+BaoCaoNgay_df['BanNo'])/(BaoCaoNgay_df['CVNGAYBC_UPDATED']+BaoCaoNgay_df['N_DieuChuyen']+BaoCaoNgay_df['BanNo']).fillna(0)).round(2)


# TỶ LỆ NỢ TRÊN CHO VAY THEO TỪNG KHỐI = NỢ 345 BAO GỒM BÁN NỢ / CHO VAY KHỐI BAO GỒM TRÁI PHIẾU
BaoCaoNgay_df['N345_CV_PR_SME']=(BaoCaoNgay_df['SME_N345_DL_UPDATED']/BaoCaoNgay_df['CVSME_TP']).fillna(0).round(2)
BaoCaoNgay_df['N345_CV_PR_CIB']=(BaoCaoNgay_df['CIB_N345_DL_UPDATED']/BaoCaoNgay_df['CVCIB_TP']).fillna(0).round(2)
BaoCaoNgay_df['N345_CV_PR_CN']=(BaoCaoNgay_df['CN_N345_DL_UPDATED']/BaoCaoNgay_df['CVCN_TP']).fillna(0).round(2)



In [106]:
LN_PR=[]
num=[]
# CÁCH TÍNH CỘT PHẦN TRĂM THỰC HIỆN KẾ HOẠCH LỢI NHUẬN:
for i,j in zip(BaoCaoNgay_df['LN Lũy kế'].values, BaoCaoNgay_df['LN kế hoạch'].values):
    if i<0 and j<0 and i<j and (i/j-1)<1:
        k=1+(-(i/j-1))
        num.append(k)
    if i<0 and j<0 and i<j and (i/j-1>=1):
        k=-(i/j-1)
        num.append(k)
    if i<0 and j<0 and i>j:
        k=-((i-j)/j-1)
        num.append(k)
    if  i>=0 and j<0:
        k=(i-j)/abs(j)+1
        num.append(k)
    if i<0 and j>0:
        k=(i-j)/j
        num.append(k)
    if i>=0 and j>0:
        k=i/j
        num.append(k)

for l in num:
    if l>-1:
        LN_PR.append(l)
    else:
        LN_PR.append(-1)

In [107]:
LN_PR=[]
num=[]
# CÁCH TÍNH CỘT PHẦN TRĂM THỰC HIỆN KẾ HOẠCH LỢI NHUẬN:
for i,j in zip(BaoCaoNgay_df['LN Lũy kế'].values, BaoCaoNgay_df['LN kế hoạch'].values):
    if i<0 and j<0 and i<j and (i/j-1)<1:
        k=1+(-(i/j-1))
        num.append(k)
    if i<0 and j<0 and i<j and (i/j-1>=1):
        k=-(i/j-1)
        num.append(k)
    if i<0 and j<0 and i>j:
        k=-((i-j)/j-1)
        num.append(k)
    if  i>=0 and j<0:
        k=(i-j)/abs(j)+1
        num.append(k)
    if i<0 and j>0:
        k=(i-j)/j
        num.append(k)
    if i>=0 and j>0:
        k=i/j
        num.append(k)

for l in num:
    if l>-1:
        LN_PR.append(l)
    else:
        LN_PR.append(-1)
# thêm cột lợi nhuận kế hoạch
BaoCaoNgay_df['LN_PR']=LN_PR
BaoCaoNgay_df['LN_PR']=(BaoCaoNgay_df['LN_PR']).fillna(0).round(2)

In [108]:
LN_LK_TT_PR=[]
num=[]
# CÁCH TÍNH CỘT PHẦN TRĂM THỰC HIỆN KẾ HOẠCH LỢI NHUẬN LŨY KẾ TẠM TÍNH:
for i,j in zip(BaoCaoNgay_df['LN_LK_TT'].values, BaoCaoNgay_df['LN Tạm tính kế hoạch'].values):
    if i<0 and j<0 and i<j and (i/j-1)<1:
        k=1+(-(i/j-1))
        num.append(k)
    if i<0 and j<0 and i<j and (i/j-1>=1):
        k=-(i/j-1)
        num.append(k)
    if i<0 and j<0 and i>j:
        k=-((i-j)/j-1)
        num.append(k)
    if  i>=0 and j<0:
        k=(i-j)/abs(j)+1
        num.append(k)
    if i<0 and j>0:
        k=(i-j)/j
        num.append(k)
    if i>=0 and j>0:
        k=i/j
        num.append(k)

for l in num:
    if l>-1:
        LN_LK_TT_PR.append(l)
    else:
        LN_LK_TT_PR.append(-1)
        
# thêm cột lợi nhuận kế hoạch
BaoCaoNgay_df['LN_LK_TT_PR']=LN_LK_TT_PR
BaoCaoNgay_df['LN_LK_TT_PR']=(BaoCaoNgay_df['LN_LK_TT_PR']).fillna(0).round(2)

In [109]:
# Get all columns of type 'object'
object_columns = BaoCaoNgay_df.select_dtypes(include='object').columns.tolist()

# Exclude 'CN' and 'Tên đơn vị'
selected_columns = [col for col in object_columns if col not in object_columns]

# chuyển các cột trừ 'CN' và 'Tên đơn vị' thành dạng số thập phân
BaoCaoNgay_df[selected_columns]=BaoCaoNgay_df[selected_columns].astype(float)

In [110]:
# THÊM CỘT TỶ LỆ IB_MB/TONG SLGD
BaoCaoNgay_df['TỶ LỆ % IB_MB/TONG SLGD']=BaoCaoNgay_df['IB_MB năm 2022']/BaoCaoNgay_df['TONG GD năm 2022']
BaoCaoNgay_df['TỶ LỆ % IB_MB/TONG SLGD - CN tháng báo cáo']=BaoCaoNgay_df['IB_MB - tháng báo cáo']/BaoCaoNgay_df['TONG GD - tháng báo cáo']
# ĐIỀU CHỈNH CỘT TỶ LỆ GIAO DỊCH QUA KÊNH SỐ VÀ TỶ LỆ IB_MB TRONG TỔNG SLGD
BaoCaoNgay_df['TỶ LỆ % GIAO DỊCH QUA KÊNH SỐ - CN Năm 2022']=BaoCaoNgay_df['IB_MB năm 2022']/BaoCaoNgay_df['TONG GD năm 2022']


In [111]:
# THÊM CỘT % THU DỊCH VỤ
BaoCaoNgay_df['Tổng thu dịch vụ - % TH so với KH']=(BaoCaoNgay_df['Thu DV - TH']/BaoCaoNgay_df['Tổng thu dịch vụ kế hoạch tháng']).fillna(0)
BaoCaoNgay_df['Thu dịch vụ CMB - % TH so với KH']=(BaoCaoNgay_df['Thu DV - CMB']/BaoCaoNgay_df['Thu DV - CMB kế hoạch']).fillna(0)
BaoCaoNgay_df['Thu dịch vụ CIB - % TH so với KH']=(BaoCaoNgay_df['Thu DV - CIB']/BaoCaoNgay_df['Thu DV - CIB kế hoạch']).fillna(0)
BaoCaoNgay_df['Thu dịch vụ CN - % TH so với KH']=(BaoCaoNgay_df['Thu DV - CN']/BaoCaoNgay_df['Thu DV - CN kế hoạch']).fillna(0)
BaoCaoNgay_df['Thu dịch vụ THẺ - % TH so với KH']=(BaoCaoNgay_df['Thu DV - Thẻ']/BaoCaoNgay_df['Thu DV - THẺ kế hoạch']).fillna(0)
# THÊM CỘT % TỔNG THU KINH DOANH NGOẠI TỆ
BaoCaoNgay_df['Tổng thu KDNT - % TH so với KH']=(BaoCaoNgay_df['Thu KDNT - TH']/BaoCaoNgay_df['Tổng thu KDNT kế hoạch tháng']).fillna(0)
BaoCaoNgay_df['Thu KDNT CMB - % TH so với KH']=(BaoCaoNgay_df['Thu KDNT - CMB']/BaoCaoNgay_df['Thu KDNT - CMB kế hoạch']).fillna(0)
BaoCaoNgay_df['Thu KDNT CIB - % TH so với KH']=(BaoCaoNgay_df['Thu KDNT - CIB']/BaoCaoNgay_df['Thu KDNT - CIB kế hoạch']).fillna(0)
BaoCaoNgay_df['Thu KDNT CN - % TH so với KH']=(BaoCaoNgay_df['Thu KDNT - CN']/BaoCaoNgay_df['Thu KDNT - CN kế hoạch']).fillna(0)
# THÊM CỘT % DOANH SỐ MUA BÁN NGOẠI TỆ
BaoCaoNgay_df['Tổng DS MBNT - % TH so với KH']=(BaoCaoNgay_df['DS MBNT - TH']/BaoCaoNgay_df['Tổng DS MBNT kế hoạch tháng']).fillna(0)
BaoCaoNgay_df['DS MBNT CMB - % TH so với KH']=(BaoCaoNgay_df['DS MBNT - CMB']/BaoCaoNgay_df['DS MBNT KHDN 2022 - CMB kế hoạch']).fillna(0)
BaoCaoNgay_df['DS MBNT CIB - % TH so với KH']=(BaoCaoNgay_df['DS MBNT - CIB']/BaoCaoNgay_df['DS MBNT - CIB kế hoạch']).fillna(0)
BaoCaoNgay_df['DS MBNT CN - % TH so với KH']=(BaoCaoNgay_df['DS MBNT - CN']/BaoCaoNgay_df['DS MBNT - CN kế hoạch']).fillna(0)
# THÊM CỘT % BẢO HIỂM NHÂN THỌ
BaoCaoNgay_df['Tổng BHNT - % TH so với KH']=(BaoCaoNgay_df['BHNT - TH']/BaoCaoNgay_df['Tổng BHNT kế hoạch']).fillna(0)
BaoCaoNgay_df['BHNT CMB - % TH so với KH']=(BaoCaoNgay_df['BHNT - CMB']/BaoCaoNgay_df['BHNT - KHDN 2022 - CMB kế hoạch']).fillna(0)
BaoCaoNgay_df['BHNT CIB - % TH so với KH']=(BaoCaoNgay_df['BHNT - CIB']/BaoCaoNgay_df['DS BHNT - CIB kế hoạch']).fillna(0)
BaoCaoNgay_df['BHNT CN - % TH so với KH']=(BaoCaoNgay_df['BHNT - CN']/BaoCaoNgay_df['DS BHNT - CN kế hoạch']).fillna(0)
BaoCaoNgay_df['BHNT Thẻ - % TH so với KH']=(BaoCaoNgay_df['BHNT - Thẻ']/BaoCaoNgay_df['DS BHNT - THẺ kế hoạch']).fillna(0)
# THÊM CỘT % DOANH SỐ THANH TOÁN QUỐC TẾ
BaoCaoNgay_df['Tổng DS TTQT - % TH so với KH']=(BaoCaoNgay_df['DS TTQT - TH']/BaoCaoNgay_df['Tổng DS TTQT - kế hoạch']).fillna(0)
BaoCaoNgay_df['DS TTQT CMB - % TH so với KH']=(BaoCaoNgay_df['TTQT - CMB']/BaoCaoNgay_df['TTQT KHDN 2022 - CMB kế hoạch']).fillna(0)
BaoCaoNgay_df['DS TTQT CIB - % TH so với KH']=(BaoCaoNgay_df['TTQT - CIB']/BaoCaoNgay_df['DS TTQT - CIB kế hoạch']).fillna(0)
# THÊM CỘT % THU HỘ TRÁI PHIẾU
BaoCaoNgay_df['Tổng Thu Hộ TP - % TH so với KH']=(BaoCaoNgay_df['Tổng Thu Hộ TP - TH']/BaoCaoNgay_df['Tổng Thu Hộ TP - kế hoạch']).fillna(0)
BaoCaoNgay_df['Thu Hộ TP CMB - % TH so với KH']=(BaoCaoNgay_df['THU HỘ TP - CMB']/BaoCaoNgay_df['THU HỘ TRÁI PHIẾU - CMB kế hoạch']).fillna(0)
BaoCaoNgay_df['Thu Hộ TP CIB - % TH so với KH']=(BaoCaoNgay_df['THU HỘ TP - CIB']/BaoCaoNgay_df['THU HỘ TRÁI PHIẾU - CIB kế hoạch']).fillna(0)
BaoCaoNgay_df['Thu Hộ TP CN - % TH so với KH']=(BaoCaoNgay_df['THU HỘ TP - CN']/BaoCaoNgay_df['THU HỘ TRÁI PHIẾU - CN kế hoạch']).fillna(0)
# THÊM CÁC CỘT KHDN
BaoCaoNgay_df['SL KHDN LŨY KẾ CMB - % TH so với KH']=(BaoCaoNgay_df['SL KHDN LŨY KẾ - CMB']/BaoCaoNgay_df['SL KH - CMB kế hoạch']).fillna(0)

BaoCaoNgay_df['QUẢN LÝ TIỀN MẶT_LŨY KẾ CMB - % TH so với KH']=(BaoCaoNgay_df['QUẢN LÝ TIỀN MẶT_LŨY KẾ - CMB']/BaoCaoNgay_df['SL KHDN sd QLTM 2022 - CMB kế hoạch']).fillna(0)
BaoCaoNgay_df['SL KHDN CHUỖI CMB - % TH so với KH']=(BaoCaoNgay_df['SL KHDN CHUỖI - CMB']/BaoCaoNgay_df['SLKHDN Chuỗi 2022 - CMB kế hoạch']).fillna(0)
BaoCaoNgay_df['IBK CMB - % TH so với KH']=(BaoCaoNgay_df['IBK - CMB']/BaoCaoNgay_df['IBK - CMB kế hoạch']).fillna(0)
BaoCaoNgay_df['DƯ NỢ TÍN DỤNG CHUỖI CMB - % TH so với KH']=(BaoCaoNgay_df['DƯ NỢ TÍN DỤNG CHUỖI - CMB']/BaoCaoNgay_df['Dư nợ tín dụng chuỗi 2022 - CMB kế hoạch']).fillna(0)
BaoCaoNgay_df['KHÁCH HÀNG TÍN DỤNG (KHDN) - CMB % TH so với KH']=(BaoCaoNgay_df['KHÁCH HÀNG TÍN DỤNG (KHDN) - CMB']/BaoCaoNgay_df['SL KHDN tín dụng 2022 - CMB kế hoạch']).fillna(0)
BaoCaoNgay_df['SL KHDN PAYROLL CMB - % TH so với KH']=(BaoCaoNgay_df['SL KHDN PAYROLL - CMB']/BaoCaoNgay_df['SL KHDN Payroll - CMB kế hoạch']).fillna(0)
BaoCaoNgay_df['SL TK NHẬN LƯƠNG PAYROLL - CMB % TH so với KH']=(BaoCaoNgay_df['SL TK NHẬN LƯƠNG PAYROLL - CMB']/BaoCaoNgay_df['SL TK nhận lương Payroll - CMB kế hoạch']).fillna(0)

BaoCaoNgay_df['SL KHCN - % TH so với KH']=(BaoCaoNgay_df['SỐ LƯỢNG KH CN']/BaoCaoNgay_df['SLKH - CN kế hoạch']).fillna(0)
BaoCaoNgay_df['KH CN BÁN CHÉO CHO KHDN - % TH so với KH']=(BaoCaoNgay_df['BÁN CHÉO KHDN - CN']/BaoCaoNgay_df['BÁN CHÉO - CN kế hoạch']).fillna(0)

BaoCaoNgay_df['SL Điểm lẻ chuỗi - % TH so với KH']=(BaoCaoNgay_df['SỐ LƯỢNG ĐIỂM LẺ CHUỖI - CN']/BaoCaoNgay_df['SL điểm lẻ chuỗi - CN kế hoạch']).fillna(0)
BaoCaoNgay_df['SL THẺ GHI NỢ - % TH so với KH']=(BaoCaoNgay_df['SL THẺ GHI NỢ_LŨY KẾ - THẺ']/BaoCaoNgay_df['SL THẺ DEBIT LK - THẺ kế hoạch']).fillna(0)
BaoCaoNgay_df['SL KHDN CIB - % TH so với KH']=(BaoCaoNgay_df['SL KHDN CIB']/BaoCaoNgay_df['SL KHDNL - CIB kế hoạch']).fillna(0)
BaoCaoNgay_df['SL người dùng kênh số CN - % TH so với KH']=(BaoCaoNgay_df['SỐ LƯỢNG NGƯỜI DÙNG KÊNH SỐ - CN']/BaoCaoNgay_df['SL người dùng kênh số - CN kế hoạch']).fillna(0)
BaoCaoNgay_df['SL THẺ TÍN DỤNG - % TH so với KH']=(BaoCaoNgay_df['SL THẺ TÍN DỤNG_LŨY KẾ - THẺ']/BaoCaoNgay_df['SL THẺ CREDIT LŨY KẾ - THẺ kế hoạch']).fillna(0)



# CẬP NHẬP TỶ LỆ GIAO DỊCH QUA KÊNH SỐ - KẾ HOẠCH

In [112]:
# lấy tỉ lệ giao dịch qua kênh số năm 2022
df=pd.read_excel(info_filename,sheet_name='2022')[['MDV','TỶ LỆ % GIAO DỊCH QUA KÊNH SỐ - CN Năm 2022']]

In [113]:
# thay thế cột giá trị tỷ lệ giao dịch qua kênh số cũ
BaoCaoNgay_df=BaoCaoNgay_df.drop(columns='TỶ LỆ % GIAO DỊCH QUA KÊNH SỐ - CN Năm 2022').merge(df,left_on='CN',right_on='MDV',how='left').fillna(0).drop(columns='MDV')


In [114]:
# Kế hoạch tỷ lệ GD qua kênh số năm 2023 là tất cả các đơn vị, cụm, khu vực, toàn hàng đạt 60% vào tháng 12 năm 2023
column_name = 'Tỷ lệ GD qua kênh số 2023 - 12'
default_value = 0.6

# Use the 'insert()' method to insert the column at position 1
df.insert(1, column_name, default_value)

# lấy 60% trừ cho tỷ lệ giao dịch kênh số hiện giờ chia cho 12:
# mỗi tháng tỉ lệ này phải tăng trưởng bằng số trên để tới tới tháng 12 năm 2023 đạt 2023:
for i in range(1,12):
    df[f'Tỷ lệ GD qua kênh số 2023 - {i}']=(df['Tỷ lệ GD qua kênh số 2023 - 12']-df['TỶ LỆ % GIAO DỊCH QUA KÊNH SỐ - CN Năm 2022'])/12+df.iloc[:,1+i]

In [115]:
# chọn cột kế hoạch tỷ lệ  giao dịch qua kênh số tương ứng với ngày báo cáo
if 1 <= NgayBaoCao_dt.month <= 11:
    BaoCaoNgay_df['Tỷ lệ % GD qua kênh số - CN kế hoạch'] = df.iloc[:, NgayBaoCao_dt.month + 2]
else:
    BaoCaoNgay_df['Tỷ lệ % GD qua kênh số - CN kế hoạch'] = df['Tỷ lệ GD qua kênh số 2023 - 12']


In [116]:
# do tỷ lệ giao dịch qua kênh số của '999 đã đạt 100% nên kế hoạch của khu vực này ko là 60% nên 100%
BaoCaoNgay_df.loc[BaoCaoNgay_df.CN=="'999",'Tỷ lệ % GD qua kênh số - CN kế hoạch']=1
# tỷ lệ giao dịch qua kênh số của '324 có kê hoạch = 0
BaoCaoNgay_df.loc[BaoCaoNgay_df.CN=="'324",'Tỷ lệ % GD qua kênh số - CN kế hoạch']=0
BaoCaoNgay_df.loc[BaoCaoNgay_df.CN=="'689",'Tỷ lệ % GD qua kênh số - CN kế hoạch']=0
BaoCaoNgay_df=BaoCaoNgay_df.drop_duplicates()

In [117]:
# tính lại tỷ lệ GD qua kênh số - % thực hiện so với KH
BaoCaoNgay_df['Tỷ lệ GD qua kênh số CN - % TH so với KH']=(BaoCaoNgay_df['TỶ LỆ % IB_MB/TONG SLGD - CN tháng báo cáo']/BaoCaoNgay_df['Tỷ lệ % GD qua kênh số - CN kế hoạch']).fillna(0)

In [118]:
# replace any inf values with NaN
BaoCaoNgay_df.replace([np.inf, -np.inf], np.nan, inplace=True)
# fill NaN values with 0 (or any other value you prefer)
BaoCaoNgay_df.fillna(0,inplace=True)

# CẬP NHẬP CÁC CỘT ngày thành lập và các cột dữ liệu trống khác

In [119]:
df=pd.read_excel(info_filename,sheet_name='DS Cum Va Khu Vuc')
df=df.iloc[:,[0,-1,-2,-3,-4,-5,-6,-7,-8,-9,-10]]

In [120]:
# Assuming your dataframe is named 'df' and the datetime column is named 'datetime_column'
df['Ngày thành lập'] = pd.to_datetime(df['Ngày thành lập']).dt.date
df['Ngày thành lập mới'] = pd.to_datetime(df['Ngày thành lập mới']).dt.date

In [121]:
BaoCaoNgay_df=BaoCaoNgay_df.merge(df)

# TẠO BÁO CÁO THÀNH NHIỀU SHEET KHÁC NHAU ĐẶT TÊN SHEET TƯƠNG ỨNG

In [122]:
# sắp xếp thứ tự các cột
ordered_col=['STT','CN','Tên đơn vị','HNGAYBC','H_2022','HNGAYBC','H_KH','HNGAYBC_PR',
             'HSME_2022','HSME','HSMEVND','HSMEVNDKKH','HSMEVNDNH','HSMEVNDTDH','HSMESJC','HSMEUSD','HSMEEUR','HSMEKHAC','HSMEKKH','HSMENH','HSMETDH','HĐ - CMB kế hoạch','HSME_PR',
             'HCIB_2022','HCIB','HCIBVND','HCIBVNDKKH','HCIBVNDNH','HCIBVNDTDH','HCIBSJC','HCIBUSD','HCIBEUR','HCIBKHAC','HCIBKKH','HCIBNH','HCIBTDH','HĐ - CIB kế hoạch','HCIB_PR',
             'HCN','HCN_2022','HCN','HCNVND','HCNVNDKKH','HCNVNDNH','HCNVNDTDH','HCNSJC','HCNUSD','HCNEUR','HCNKHAC','HCNKKH','HCNNH','HCNTDH','Trái phiếu T12','HĐ - CN kế hoạch','HCN_PR',
             'HĐ CN (Tiền gửi thanh toán) 31.12.2022',f'HĐ CN (Tiền gửi thanh toán) {date_str}','HĐ KKH - CN kế hoạch','HĐ CN (Tiền gửi thanh toán) % thực hiện so kế hoạch',
             'Cho vay (không bao gồm chiết khấu)','CV_UPDATED_2022','CVNGAYBC_UPDATED','CVNGAYBC_TP','CV_KH','CVNGAYBC_KH_PR',
             'CVSME_UPDATED_2022','CVSME_UPDATED','CVSME_TP','CVSMEVND','CVSMEVNDNH','CVSMEVNDTH','CVSMEVNDDH','CVSMESJC','CVSMEUSD','CVSMEEUR','CVSMEKHAC','CVSMENH','CVSMETH','CVSMEDH','SME_DTCK','UPAS_LC_SME_30','CV - CMB kế hoạch','CVSME_PR',
             'CVCIB_UPDATED_2022','CVCIB_UPDATED','CVCIB_TP','CVCIBVND','CVCIBVNDNH','CVCIBVNDTH','CVCIBVNDDH','CVCIBSJC','CVCIBUSD','CVCIBEUR','CVCIBKHAC','CVCIBNH','CVCIBTH','CVCIBDH','CIB_DTCK','UPAS_LC_CIB_30','CV - CIB kế hoạch','CVCIB_PR',
             'CVCN_UPDATED','CVCN_UPDATED_2022','CVCN_UPDATED','CVCN_TP','CVCNVND','CVCNVNDNH','CVCNVNDTH','CVCNVNDDH','CVCNSJC','CVCNUSD','CVCNEUR','CVCNKHAC','CVCNNH','CVCNTH','CVCNDH','UPAS_LC_CN_30','CV - CN kế hoạch','CVCN_PR',
             'THE_CV_2022','THE_CV','THE_CVVND','THE_CVVNDNH','THE_CVVNDTH','THE_CVVNDDH','THE_CVSJC','THE_CVUSD','THE_CVEUR','THE_CVKHAC','THE_CVNH','THE_CVTH','THE_CVDH','CV - THẺ kế hoạch','THE_CV_PR',
             'CK_DL','SME_CK_DL','CIB_CK_DL','CN_CK_DL','N1_DL','N2_DL','N3_DL','N4_DL','N5_DL','N345_DL','N_DieuChuyen','BanNo','N345_SauDieuChuyen','N345_CV_PR',
             'SME_N1_DL','SME_N2_DL','SME_N3_DL','SME_N4_DL','SME_N5_DL','BanNo_SME','SME_N345_DL_UPDATED','N345_CV_PR_SME',
             'CIB_N1_DL','CIB_N2_DL','CIB_N3_DL','CIB_N4_DL','CIB_N5_DL','BanNo_CIB','CIB_N345_DL','N345_CV_PR_CIB',
             'CN_N1_DL','CN_N2_DL','CN_N3_DL','CN_N4_DL','CN_N5_DL','BanNo_CN','CN_N345_DL_UPDATED','N345_CV_PR_CN',
             'THE_N1_DL','THE_N2_DL','THE_N3_DL','THE_N4_DL','THE_N5_DL','BanNo_The','THE_N345_DL_UPDATED','N345_CV_PR_THE',
             'LN_2022','LNTRONGTHANG','LNLUYKE','LN Phát sinh','LN Lũy kế','LN kế hoạch','LN_PR',
             'LNTRONGTHANG','LN_LK_TT','LN Tạm tính kế hoạch','LN_LK_TT_PR','TS_CO',
             'Huy động ứng trước TK 023','Cho vay ứng trước TK 023','Bán nợ 31/3 còn lại sau đáo hạn','Đáo hạn','Bán nợ đến 31/3',
             'Bán nợ trong tháng 4 chưa trừ huy động','Bán nợ từ 1/4 đến hiện nay','Ngày thành lập','Ngày thành lập mới','HPC108','Cho vay khác',
             'Tổng thu Dịch Vụ năm 2022','Thu DV - TH','Tổng thu dịch vụ kế hoạch tháng','Tổng thu dịch vụ - % TH so với KH',
             'Thu Dịch Vụ CMB năm 2022','Thu DV - CMB','Thu DV - CMB kế hoạch','Thu dịch vụ CMB - % TH so với KH',
             'Thu Dịch Vụ CIB năm 2022','Thu DV - CIB','Thu DV - CIB kế hoạch','Thu dịch vụ CIB - % TH so với KH',
             'Thu Dịch Vụ CN năm 2022','Thu DV - CN','Thu DV - CN kế hoạch','Thu dịch vụ CN - % TH so với KH',
             'Thu Dịch Vụ THẺ năm 2022','Thu DV - Thẻ','Thu DV - THẺ kế hoạch','Thu dịch vụ THẺ - % TH so với KH',
             'Tổng KDNT năm 2022','Thu KDNT - TH','Tổng thu KDNT kế hoạch tháng','Tổng thu KDNT - % TH so với KH',
             'Thu KDNT CMB năm 2022','Thu KDNT - CMB','Thu KDNT - CMB kế hoạch','Thu KDNT CMB - % TH so với KH',
             'Thu KDNT CIB năm 2022','Thu KDNT - CIB','Thu KDNT - CIB kế hoạch','Thu KDNT CIB - % TH so với KH',
             'Thu KDNT CN năm 2022','Thu KDNT - CN','Thu KDNT - CN kế hoạch','Thu KDNT CN - % TH so với KH',
             'Tổng DS MBNT năm 2022','DS MBNT - TH','Tổng DS MBNT kế hoạch tháng','Tổng DS MBNT - % TH so với KH',
             'DS MBNT CMB năm 2022','DS MBNT - CMB','DS MBNT KHDN 2022 - CMB kế hoạch','DS MBNT CMB - % TH so với KH',
             'DS MBNT CIB năm 2022','DS MBNT - CIB','DS MBNT - CIB kế hoạch','DS MBNT CIB - % TH so với KH',
             'DS MBNT CN năm 2022','DS MBNT - CN','DS MBNT - CN kế hoạch','DS MBNT CN - % TH so với KH',
             'Tổng BHNT năm 2022','BHNT - TH','Tổng BHNT kế hoạch','Tổng BHNT - % TH so với KH',
             'BHNT CMB năm 2022','BHNT - CMB','BHNT - KHDN 2022 - CMB kế hoạch','BHNT CMB - % TH so với KH',
             'BHNT CIB năm 2022','BHNT - CIB','DS BHNT - CIB kế hoạch','BHNT CIB - % TH so với KH',
             'BHNT CN năm 2022','BHNT - CN','DS BHNT - CN kế hoạch','BHNT CN - % TH so với KH',
             'BHNT Thẻ năm 2022','BHNT - Thẻ','DS BHNT - THẺ kế hoạch','BHNT Thẻ - % TH so với KH',
             'Tổng DS TTQT năm 2022','DS TTQT - TH','Tổng DS TTQT - kế hoạch','Tổng DS TTQT - % TH so với KH',
             'DS TTQT CMB năm 2022','TTQT - CMB','TTQT KHDN 2022 - CMB kế hoạch','DS TTQT CMB - % TH so với KH',
             'DS TTQT CIB năm 2022','TTQT - CIB','DS TTQT - CIB kế hoạch','DS TTQT CIB - % TH so với KH',
             'Tổng Thu Hộ TP - Năm 2022','Tổng Thu Hộ TP - TH','Tổng Thu Hộ TP - kế hoạch','Tổng Thu Hộ TP - % TH so với KH',
             'Thu Hộ TP CMB - Năm 2022','THU HỘ TP - CMB','THU HỘ TRÁI PHIẾU - CMB kế hoạch','Thu Hộ TP CMB - % TH so với KH',
             'Thu Hộ TP CN - Năm 2022','THU HỘ TP - CN','THU HỘ TRÁI PHIẾU - CN kế hoạch','Thu Hộ TP CN - % TH so với KH',
             'Thu Hộ TP CIB - Năm 2022','THU HỘ TP - CIB','THU HỘ TRÁI PHIẾU - CIB kế hoạch','Thu Hộ TP CIB - % TH so với KH',
             'SL KHDN LŨY KẾ - CMB Năm 2022','SL KHDN LŨY KẾ - CMB','SL KH - CMB kế hoạch','SL KHDN LŨY KẾ CMB - % TH so với KH',
             'QUẢN LÝ TIỀN MẶT_LŨY KẾ - CMB Năm 2022','QUẢN LÝ TIỀN MẶT_LŨY KẾ - CMB','SL KHDN sd QLTM 2022 - CMB kế hoạch','QUẢN LÝ TIỀN MẶT_LŨY KẾ CMB - % TH so với KH',
             'SL KHDN CHUỖI - CMB Năm 2022','SL KHDN CHUỖI - CMB','SLKHDN Chuỗi 2022 - CMB kế hoạch','SL KHDN CHUỖI CMB - % TH so với KH',
             'IBK - CMB Năm 2022','IBK - CMB','IBK - CMB kế hoạch','IBK CMB - % TH so với KH',
             'DƯ NỢ TÍN DỤNG CHUỖI - CMB Năm 2022','DƯ NỢ TÍN DỤNG CHUỖI - CMB','Dư nợ tín dụng chuỗi 2022 - CMB kế hoạch','DƯ NỢ TÍN DỤNG CHUỖI CMB - % TH so với KH',
             'KHÁCH HÀNG TÍN DỤNG (KHDN) - CMB Năm 2022','KHÁCH HÀNG TÍN DỤNG (KHDN) - CMB','SL KHDN tín dụng 2022 - CMB kế hoạch','KHÁCH HÀNG TÍN DỤNG (KHDN) - CMB % TH so với KH',
             'SL KHDN PAYROLL - CMB Năm 2022','SL KHDN PAYROLL - CMB','SL KHDN Payroll - CMB kế hoạch','SL KHDN PAYROLL CMB - % TH so với KH',
             'SL TK NHẬN LƯƠNG PAYROLL - CMB Năm 2022','SL TK NHẬN LƯƠNG PAYROLL - CMB','SL TK nhận lương Payroll - CMB kế hoạch','SL TK NHẬN LƯƠNG PAYROLL - CMB % TH so với KH',
             'SL KH CHO VAY CMB','SL KH D.NGHIỆP CIB - Năm 2022','SL KHDN CIB','SL KHDNL - CIB kế hoạch','SL KHDN CIB - % TH so với KH',
             'SL KH CN - Năm 2022','SỐ LƯỢNG KH CN','SLKH - CN kế hoạch','SL KHCN - % TH so với KH',
             'SL NGƯỜI DÙNG KÊNH SỐ - CN Năm 2022','SỐ LƯỢNG NGƯỜI DÙNG KÊNH SỐ - CN','SL người dùng kênh số - CN kế hoạch','SL người dùng kênh số CN - % TH so với KH',
             'KH CN BÁN CHÉO CHO KH DOANH NGHIỆP - Năm 2022','BÁN CHÉO KHDN - CN','BÁN CHÉO - CN kế hoạch','KH CN BÁN CHÉO CHO KHDN - % TH so với KH',
             'TỶ LỆ % GIAO DỊCH QUA KÊNH SỐ - CN Năm 2022','TỶ LỆ % IB_MB/TONG SLGD - CN tháng báo cáo','Tỷ lệ % GD qua kênh số - CN kế hoạch','Tỷ lệ GD qua kênh số CN - % TH so với KH',
             'SL ĐIỂM LẺ CHUỖI - CN Năm 2022','SỐ LƯỢNG ĐIỂM LẺ CHUỖI - CN','SL điểm lẻ chuỗi - CN kế hoạch','SL Điểm lẻ chuỗi - % TH so với KH',
             'SL THẺ GHI NỢ_LŨY KẾ - Năm 2022','SL THẺ GHI NỢ_LŨY KẾ - THẺ','SL THẺ DEBIT LK - THẺ kế hoạch','SL THẺ GHI NỢ - % TH so với KH',
             'SL THẺ TÍN DỤNG_LŨY KẾ - Năm 2022','SL THẺ TÍN DỤNG_LŨY KẾ - THẺ','SL THẺ CREDIT LŨY KẾ - THẺ kế hoạch','SL THẺ TÍN DỤNG - % TH so với KH'

]
ChiTiet_df=BaoCaoNgay_df[ordered_col].sort_values(by='STT')

In [123]:
# tạo ra danh sách các cột cần lấy cho từng khối
my_list=ChiTiet_df.columns.tolist()
# tìm các cột của khối SME
target_strings = ["CMB", "SME"]
CMB_list = [s for s in my_list if "SME" in s or "CMB" in s]
# tìm các cột của khối CIB
target_string = "CIB"
CIB_list = [s for s in my_list if target_string in s]
# tìm các cột của  khối CN
target_string = "CN"
CN_list = [s for s in my_list if target_string in s]
CN_list.remove('CN')
adding_columns=[ 'SL THẺ GHI NỢ_LŨY KẾ - Năm 2022', 'SL THẺ GHI NỢ_LŨY KẾ - THẺ', 'SL THẺ DEBIT LK - THẺ kế hoạch',
 'SL THẺ GHI NỢ - % TH so với KH', 'SL THẺ TÍN DỤNG_LŨY KẾ - Năm 2022', 'SL THẺ TÍN DỤNG_LŨY KẾ - THẺ',
 'SL THẺ CREDIT LŨY KẾ - THẺ kế hoạch', 'SL THẺ TÍN DỤNG - % TH so với KH']
for i in adding_columns:
    CN_list.append(i)
# TÌM CÁC CỘT CỦA KHỐI THẺ:
target_strings = ["thẻ", "the"]
THE_list = [s for s in my_list if any(target in s.lower() for target in target_strings)]


In [124]:
# TẠO DATAFRAME CỦA CÁC SHEET SME, CIB, CN và THE
CMB_df = pd.concat([BaoCaoNgay_df[['STT','CN','Tên đơn vị']], BaoCaoNgay_df[CMB_list]], axis=1) # SHEET SME
CIB_df = pd.concat([BaoCaoNgay_df[['STT','CN','Tên đơn vị']], BaoCaoNgay_df[CIB_list]], axis=1) # SHEET  CIB
CN_df = pd.concat([BaoCaoNgay_df[['STT','CN','Tên đơn vị']], BaoCaoNgay_df[CN_list]], axis=1) # SHEET CN
THE_df = pd.concat([BaoCaoNgay_df[['STT','CN','Tên đơn vị']], BaoCaoNgay_df[THE_list]], axis=1) # SHEET THE

In [125]:
# ĐỔI TÊN CỘT THÀNH TÊN HIỂN THỊ TRONG BÁO CÁO
rename_dict ={
    'Tỷ lệ % GD qua kênh số - CN kế hoạch':f'Tỷ lệ % GD qua kênh số - CN kế hoạch tháng {NgayBaoCao_dt.month}',
    'THU HỘ TP - CIB':f'THU HỘ TP - CIB thực hiện ngày {last_month_str}','THU HỘ TP - CN':f'THU HỘ TP - CN thực hiện ngày {last_month_str}',
    'THU HỘ TP - SME':f'THU HỘ TP - CMB thực hiện ngày {last_month_str}','TTQT - CIB':f'TTQT - CIB thực hiện ngày {last_month_str}',
    'TTQT - SME':f'TTQT - CMB thực hiện ngày {last_month_str}','BHNT - Thẻ':f'BHNT - Thẻ thực hiện ngày {last_month_str}',
    'BHNT - CN':f'BHNT - CN thực hiện ngày {last_month_str}','BHNT - CIB':f'BHNT - CIB thực hiện ngày {last_month_str}',
    'BHNT - SME':f'BHNT - CMB thực hiện ngày {last_month_str}','DS MBNT - CN':f'DS MBNT - CN thực hiện ngày {last_month_str}',
    'DS MBNT - CIB':f'DS MBNT - CIB thực hiện ngày {last_month_str}','DS MBNT - SME':f'DS MBNT - CMB thực hiện ngày {last_month_str}',
    'Thu KDNT - CN':f'Thu KDNT - CN thực hiện ngày {last_month_str}','Thu KDNT - CIB':f'Thu KDNT - CIB thực hiện ngày {last_month_str}',
    'Thu KDNT - SME':f'Thu KDNT - CMB thực hiện ngày {last_month_str}','SL THẺ GHI NỢ_LŨY KẾ - THẺ':f'SL THẺ GHI NỢ_LŨY KẾ - THẺ thực hiện ngày {last_month_str}',
    'SL THẺ TÍN DỤNG_LŨY KẾ - THẺ':f'SL THẺ TÍN DỤNG_LŨY KẾ - THẺ thực hiện ngày {last_month_str}',
    'TỶ LỆ % IB_MB/TONG SLGD - CN tháng báo cáo':f'TỶ LỆ % IB_MB/TONG SLGD - CN thực hiện ngày {last_month_str}','SỐ LƯỢNG ĐIỂM LẺ CHUỖI - CN':f'SỐ LƯỢNG ĐIỂM LẺ CHUỖI - CN thực hiện ngày {last_month_str}',
    'SỐ LƯỢNG KH CN':f'SỐ LƯỢNG KH CN thực hiện ngày {last_month_str}','SỐ LƯỢNG NGƯỜI DÙNG KÊNH SỐ - CN':f'SỐ LƯỢNG NGƯỜI DÙNG KÊNH SỐ - CN thực hiện ngày {last_month_str}','BÁN CHÉO KHDN - CN':f'BÁN CHÉO KHDN - CN thực hiện ngày {last_month_str}',
    'SL KHDN PAYROLL - SME':f'SL KHDN PAYROLL - CMB thực hiện ngày {last_month_str}','SL TK NHẬN LƯƠNG PAYROLL - SME':f'SL TK NHẬN LƯƠNG PAYROLL - CMB thực hiện ngày {last_month_str}','SL KH CHO VAY SME':f'SL KH CHO VAY CMB thực hiện ngày {last_month_str}','SL KHDN CIB':f'SL KHDN CIB thực hiện ngày {last_month_str}',
    'SL KHDN CHUỖI - SME':f'SL KHDN CHUỖI - CMB thực hiện ngày {last_month_str}','IBK - SME':f'IBK - CMB thực hiện ngày {last_month_str}','DƯ NỢ TÍN DỤNG CHUỖI - SME':f'DƯ NỢ TÍN DỤNG CHUỖI - CMB thực hiện ngày {last_month_str}',
    'SL KHDN LŨY KẾ - SME':f'SL KHDN LŨY KẾ - CMB thực hiện ngày {last_month_str}','QUẢN LÝ TIỀN MẶT_LŨY KẾ - SME':f'QUẢN LÝ TIỀN MẶT_LŨY KẾ - CMB thực hiện ngày {last_month_str}','KHÁCH HÀNG TÍN DỤNG (KHDN) - SME':f'KHÁCH HÀNG TÍN DỤNG (KHDN) - CMB thực hiện ngày {last_month_str}',
    'BHNT - TH':f'Tổng BHNT thực hiện ngày {last_month_str}','DS TTQT - TH':f'Tổng DS TTQT thực hiện ngày {last_month_str}','Tổng Thu Hộ TP - TH':f'Tổng Thu hộ TP thực hiện ngày {last_month_str}',
    'Thu KDNT - TH':f'Tổng thu KDNT thực hiện ngày {last_month_str}','DS MBNT - TH':f'Tổng DS MBNT thực hiện ngày {last_month_str}',
    'Thu DV - CN':f'Thu DV - CN thực hiện ngày {last_month_str}','Thu DV - Thẻ':f'Thu DV - Thẻ thực hiện ngày {last_month_str}','Thu DV - CIB':f'Thu DV - CIB thực hiện ngày {last_month_str}','Thu DV - CMB':f'Thu DV - CMB thực hiện ngày {last_month_str}','Thu DV - TH':f'Tổng thu Dịch vụ thực hiện ngày {last_month_str}',
    'HPC108':f'Huy động cá nhân chuyển sang doanh nghiệp {date_str}','TS_CO':'Tổng tài sản','LN_PR':'% thực hiện kế hoạch lợi nhuận','LN_KH':'Lợi nhuận kế hoạch tháng','HĐ KKH - CN kế hoạch':'HĐ CN (Tiền gửi thanh toán) kế hoạch tháng',
    'LN_LK_TT_PR':'% thực hiện kế hoạch lợi nhuận lũy kế tạm tính','LN_LK_TT':f'LN tạm tính lũy kế từ đầu năm đến {date_str}','LNTRONGTHANG':f'LN tạm tính từ 01.02-{date_str}','LN_KH':'Lợi nhuận kế hoạch tháng','LN Lũy kế':f'LN thực lũy kế từ đầu năm đến {last_month_str}',
    'LN Phát sinh':f'Lợi nhuận thực phát sinh {month_of_last_month}','LN_TH':f'Lợi nhuận thực tháng {last_month_str}','LNLUYKE':'LN KM lũy kế từ đầu năm 2022','LNTRONGTHANG':'LN KM phát sinh trong tháng 01/2022','LN_2022':'Lợi nhuận năm 2022','N345_CV_PR_THE':'Thẻ % tỷ lệ N345/DN','THE_N345_DL_UPDATED':'Thẻ tổng N345','BanNo_The':'Thẻ bán nợ','THE_N5_DL':'Thẻ N5','THE_N4_DL':'Thẻ N4','THE_N3_DL':'Thẻ N3','THE_N2_DL':'Thẻ N2','THE_N1_DL':'Thẻ N1','N345_CV_PR_CN':'CN % tỷ lệ N345/DN',
    'CIB_N345_DL_UPDATED':'CIB tổng N345 (bao gồm bán nợ)','BanNo_CN':'CN bán nợ','CN_N5_DL':'CN N5','CN_N4_DL':'CN N4','CN_N3_DL':'CN N3','CN_N2_DL':'CN N2','CN_N1_DL':'CN N1',
    'N345_CV_PR_CIB':'CIB % tỷ lệ N345/DN','CIB_N345_DL':'CIB tổng N345','BanNo_CIB':'CIB bán nợ','CIB_N5_DL':'CIB N5','CIB_N4_DL':'CIB N4','CIB_N3_DL':'CIB N3','CIB_N2_DL':'CIB N2','CIB_N1_DL':'CIB N1',
    'N345_CV_PR_SME':'CMB % tỷ lệ N345/DN','SME_N345_DL_UPDATED':'CMB tổng N345 (bao gồm bán nợ)','BanNo_SME':'CMB bán nợ','SME_N5_DL':'CMB N5','SME_N4_DL':'CMB N4','SME_N3_DL':'CMB N3','SME_N2_DL':'CMB N2','SME_N1_DL':'CMB N1',
    'CN_N345_DL_UPDATED':'CN tổng N345 (bao gồm bán nợ)','N345_CV_PR':'% Tỷ lệ Nợ 345/cho vay','N345_SauDieuChuyen':'Tổng nợ 345 sau khi chỉnh các món điều chuyển','BanNo':'Tổng bán nợ','N_DieuChuyen':'Các món nợ điều chuyển','N345_DL':'Tổng nợ nhóm 345','N5_DL':'Nợ N5','N4_DL':'Nợ N4','N3_DL':'Nợ N3','N2_DL':'Nợ N2','N1_DL':'Nợ N1',
    'CN_CK_DL':'Chiết khấu CN','CIB_CK_DL':'Chiết khấu CIB','SME_CK_DL':'Chiết khấu CMB','CK_DL':'Chiết khấu',
    'THE_CV_PR':'% thực hiện kế hoạch dư nợ thẻ','CV - THẺ kế hoạch':'Kế  hoạch tháng dư nợ thẻ','THE_CVDH':'Dư nợ thẻ DH','THE_CVTH':'Dư nợ thẻ TH','THE_CVNH':'Dư nợ thẻ NH','THE_CVKHAC':'Dư nợ thẻ Ngtệ khác','THE_CVEUR':'Dư nợ thẻ EUR','THE_CVUSD':'Dư nợ thẻ USD','THE_CVSJC':'Dư nợ thẻ SJC','THE_CVVNDDH':'Dư nợ thẻ VND DH','THE_CVVNDTH':'Dư nợ thẻ VND TH','THE_CVVNDNH':'Dư nợ thẻ VND NH','THE_CVVND':'Dư nợ thẻ VND','THE_CV':f'Dư nợ Thẻ {date_str}',
    'THE_CV_2022':'Dư nợ thẻ 31.12.2022','CVCN_PR':'% CN thực hiện kế hoạch CV','CVCN_KH':'Cho Vay CN kế hoạch tháng','UPAS_LC_CN_30':'30% UPAS LC - CN','CVCNDH':'Cho Vay CN DH','CVCNTH':'Cho Vay CN TH','CVCNNH':'Cho Vay CN NH','CVCNKHAC':'Cho Vay CN Ngtệ Khác','CVCNEUR':'Cho Vay CN EUR','CVCNUSD':'Cho Vay CN USD','CVCNSJC':'Cho Vay CN SJC','CVCNVNDDH':'Cho Vay CN VND DH','CVCNVNDTH':'Cho Vay CN VND TH','CVCNVNDNH':'Cho Vay CN VND NH','CVCNVND':'Cho Vay CN VND','CVCN_TP':f'Cho Vay CN (bao gồm TP) ngày {date_str}',
    'CVCN_UPDATED':f'Cho Vay CN (bao gồm TP) + 30%UPAS_LC ngày {date_str}','CVCN_UPDATED_2022':'Cho Vay CN (bao gồm TP) + 30%UPAS_LC ngày 31.12.2022','CVCIB_PR':'% CIB thực hiện kế hoạch CV','CVCIB_KH':'Cho Vay CIB kế hoạch tháng','UPAS_LC_CIB_30':'30% UPAS LC - CIB','CIB_DTCK':'Trái phiếu (ghi nhận CIB)','CVCIBDH':'Cho Vay CIB DH','CVCIBTH':'Cho Vay CIB TH','CVCIBNH':'Cho Vay CIB NH','CVCIBKHAC':'Cho Vay CIB Ngtệ Khác','CVCIBEUR':'Cho Vay CIB EUR','CVCIBUSD':'Cho Vay CIB USD','CVCIBSJC':'Cho Vay CIB SJC','CVCIBVNDDH':'Cho Vay CIB VND DH','CVCIBVNDTH':'Cho Vay CIB VND TH','CVCIBVNDNH':'Cho Vay CIB VND NH','CVCIBVND':'Cho Vay CIB VND','CVCIB_TP':f'Cho Vay CIB ngày (bao gồm TP) {date_str}','CVCIB_UPDATED':f'Cho Vay CIB (bao gồm TP) + 30%UPAS_LC ngày {date_str}','CVCIB_UPDATED_2022':'Cho Vay CIB (bao gồm TP) + 30%UPAS_LC ngày 31.12.2022',
    'CVSME_PR':'% CMB thực hiện kế hoạch CV','CVSME_KH':'Cho Vay CMB kế hoạch tháng','UPAS_LC_SME_30':'30% UPAS LC - CMB','SME_DTCK':'Trái phiếu (ghi nhận CMB)','CVSMEDH':'Cho Vay CMB DH','CVSMETH':'Cho Vay CMB TH','CVSMENH':'Cho Vay CMB NH','CVSMEKHAC':'Cho Vay CMB Ngtệ Khác','CVSMEEUR':'Cho Vay CMB EUR','CVSMEUSD':'Cho Vay CMB USD','CVSMESJC':'Cho Vay CMB SJC','CVSMEVNDDH':'Cho Vay CMB VND DH','CVSMEVNDTH':'Cho Vay CMB VND TH','CVSMEVNDNH':'Cho Vay CMB VND NH','CVSMEVND':'Cho Vay CMB VND','CVSME_TP':f'Cho Vay CMB (bao gồm TP) ngày {date_str}','CVSME_UPDATED':f'Cho Vay CMB (bao gồm TP) + 30%UPAS_LC ngày {date_str}','CVSME_UPDATED_2022':'Cho Vay CMB (bao gồm TP) + 30%UPAS_LC ngày 31.12.2022',
    'CV_UPDATED_2022':'Cho vay bao gồm CC STK (gồm TPDN)+30%UPAS_LC 31/12/2022','CVNGAYBC_KH_PR':'% thực hiện kế hoạch Cho Vay','CV_KH':'Cho vay kế hoạch tháng','CVNGAYBC_TP':f'Cho Vay bao gồm CC STK (gồm TPDN) ngày {date_str}','CVNGAYBC_UPDATED':f'Cho Vay bao gồm CC STK (gồm TPDN) + 30%UPAS_LC ngày {date_str}',
    'HCN_PR':'% CN thực hiện kế hoạch HĐ','HCN_KH':'Huy động CN kế hoạch','HCNTDH':'Huy Động CN TDH','HCNNH':'Huy Động CN NH','HCNKKH':'Huy Động CN KKH','HCNKHAC':'Huy Động CN Ngtệ Khác','HCNEUR':'Huy Động CN EUR','HCNUSD':'Huy Động CN USD','HCNSJC':'Huy Động CN SJC','HCNVNDTDH':'Huy Động CN VND TDH','HCNVNDNH':'Huy Động CN VND NH','HCNVNDKKH':'Huy Động CN VND KKH','HCNVND':'Huy Động CN VND','HCN':f'Huy Động CN ngày {date_str}','HCN_2022':'Huy Động CN 31.12.2022',
    'HCIB_PR':'% CIB thực hiện kế hoạch HĐ','HCIB_KH':'Huy động CIB kế hoạch','HCIBTDH':'Huy Động CIB TDH','HCIBNH':'Huy Động CIB NH','HCIBKKH':'Huy Động CIB KKH','HCIBKHAC':'Huy Động CIB Ngtệ Khác','HCIBEUR':'Huy Động CIB EUR','HCIBUSD':'Huy Động CIB USD','HCIBSJC':'Huy Động CIB SJC','HCIBVNDTDH':'Huy Động CIB VND TDH','HCIBVNDNH':'Huy Động CIB VND NH','HCIBVNDKKH':'Huy Động CIB VND KKH','HCIBVND':'Huy Động CIB VND','HCIB':f'Huy động CIB ngày {date_str}','HCIB_2022':'Huy Động CIB 31.12.2022',
    'HSME_PR':'% CMB thực hiện kế hoạch HĐ','HSME_KH':'Huy động CMB kế hoạch','HSMETDH':'Huy Động CMB TDH','HSMENH':'Huy Động CMB NH','HSMEKKH':'Huy Động CMB KKH','HSMEKHAC':'Huy Động CMB Ngtệ Khác','HSMEEUR':'Huy Động CMB EUR','HSMEUSD':'Huy Động CMB USD','HSMESJC':'Huy Động CMB SJC','HSMEVNDTDH':'Huy Động CMB VND TDH','HSMEVNDNH':'Huy Động CMB VND NH','HSMEVNDKKH':'Huy Động CMB VND KKH','HSMEVND':'Huy Động CMB VND','HSME':f'Huy động CMB ngày {date_str}','HSME_2022':'Huy Động CMB 31.12.2022',
    'HNGAYBC_PR':'% thực hiện kế hoạch Huy Động','H_KH':'Huy động TT1 kế hoạch tháng','HNGAYBC':f'Huy Động TT1 ngày {date_str}','H_2022':'Huy động TT1 tại ngày 31.12.2022','HNGAYBC':f'Huy Động TT1 ngày {date_str}',
    'Tên đơn vị':'Tên đơn vị','CN':'Mã','STT':'STT',
}
# Đổi tên cột, loại bỏ các dòng giống nhau và sắp xếp lại theo STT
ChiTiet_df = ChiTiet_df.rename(columns=rename_dict).drop_duplicates().sort_values(by='STT')
CMB_df = CMB_df.rename(columns=rename_dict).drop_duplicates().sort_values(by='STT')
CIB_df = CIB_df.rename(columns=rename_dict).drop_duplicates().sort_values(by='STT')
CN_df = CN_df.rename(columns=rename_dict).drop_duplicates().sort_values(by='STT')
THE_df = THE_df.rename(columns=rename_dict).drop_duplicates().sort_values(by='STT')

In [126]:
# Create an ExcelWriter object
writer = pd.ExcelWriter(new_filename, engine='xlsxwriter')

# Write each dataframe to a separate sheet in the same Excel file
ChiTiet_df.to_excel(writer, sheet_name='Chi Tiet', index=False,startrow=2)
CMB_df.to_excel(writer, sheet_name='CMB', index=False,startrow=2)
CIB_df.to_excel(writer, sheet_name='CIB', index=False,startrow=2)
CN_df.to_excel(writer, sheet_name='CN', index=False,startrow=2)
THE_df.to_excel(writer, sheet_name='THE', index=False,startrow=2)

# tạo các dataframe cho các sheet khu vực
df=pd.read_excel(info_filename)
df=df[df.sheet_name.notnull()]
df=df[['MDV','sheet_name']]

dfs={}
BaoCaoNgay_dfS ={}
for i in df['sheet_name'].unique():
    dfs[i]=df[df['sheet_name']==i] # lọc ra dataframe chứa danh sách đơn vị
    BaoCaoNgay_dfS[i]=dfs[i].merge(ChiTiet_df,how='left',left_on='MDV',right_on='Mã').sort_values(by='STT') # Lấy dataframe chi tiết tương ứng từng dataframe trên
    BaoCaoNgay_dfS[i]=BaoCaoNgay_dfS[i].drop(columns=dfs[i].columns).drop_duplicates() # bỏ các cột nối vào và bỏ các giá trị trùng lặp
    BaoCaoNgay_dfS[i].to_excel(writer,sheet_name=i,index=False,startrow=2) # lưu các báo cáo theo từng khu vực vào từng sheet vs tên sheet tương ứng cột sheet_name trong dataframe trên

# Save the Excel file
writer.save()

# Release the Excel file
writer.close()

C:\Users\phatdd3\AppData\Local\Temp\ipykernel_18656\1265481437.py:25: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
C:\Users\phatdd3\AppData\Local\anaconda3\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


# THIẾT KẾ GIAO DIỆN BÁO CÁO

In [127]:
from openpyxl import load_workbook
from openpyxl import Workbook
from openpyxl.styles import Font

# mở file excel Báo Cáo ngày
wb=load_workbook(new_filename)

# CHỈNH FONT VÀ FONT SIZE CHO CÁC SHEET CỦA FILE

In [128]:
# get all sheet names
sheet_names = wb.sheetnames

# loop through each sheet
for sheet_name in sheet_names:
    sheet = wb[sheet_name]

    # loop through each cell in the sheet
    for row in sheet.iter_rows():
        for cell in row:
            # set the font to Myriad Pro and size 10
            cell.font = Font(name='Myriad Pro', size=10)
# save the modified workbook
wb.save(new_filename)

# THÊM CÁC CỘT ĐỂ ĐIỀU CHỈNH CỘT VÀO ĐÚNG SỐ NHƯ FILE BÁO CÁO CŨ

In [129]:
from openpyxl.utils import column_index_from_string
# thêm các cột trống MỚI tại các vị trí bên dưới

# Define the sheet names to modify
sheet_names = []
for sheet in wb.sheetnames:
    if sheet not in ["CMB", "CIB", "CN", "THE"]:
        sheet_names.append(sheet)

# Define the column positions to insert a blank column
positions_to_insert = ['GV','HQ','IH','IY','JT','KG','KX','ME','MG','ML','NG']

# Loop through each sheet and insert a blank column at the specified positions
for sheet_name in sheet_names:
    ws = wb[sheet_name]
    for position in positions_to_insert:
        col_idx = openpyxl.utils.column_index_from_string(position)
        ws.insert_cols(col_idx)

# save the modified workbook
wb.save(new_filename)

# ẨN MỘT SỐ CỘT

In [130]:
# ẨN MỘT SỐ CỘT 

# Define a list of sheet names to apply changes to
for sheet in wb.sheetnames:
    if sheet not in ["CMB", "CIB", "CN", "THE"]:
        sheet_names.append(sheet)

# loop through the sheets
for sheet_name in sheet_names:
    # select the sheet
    ws = wb[sheet_name]
    for col in ['GK','GL','GM','GN','GO','GP','GQ']:
        ws.column_dimensions[col].hidden = True        

# loop through the sheets
for sheet_name in sheet_names:
    # select the sheet
    ws = wb[sheet_name]
    for col in ['GF', 'GG', 'GH', 'GI']:
        ws.column_dimensions[col].hidden = True

# loop through the sheets
for sheet_name in sheet_names:
    # select the sheet
    ws = wb[sheet_name]
    for col in ['FZ', 'GA']:
        ws.column_dimensions[col].hidden = True
        
# save the modified workbook
wb.save(new_filename)

# CÀI ĐẶT ĐỘ CAO, ĐỘ RỘNG, CANH GIỮA VÀ XUỐNG HÀNG CHO TẤT CẢ CÁC CỘT CỦA DÒNG 

In [131]:
from openpyxl.utils import get_column_letter
from openpyxl.styles import Alignment

# Iterate through all sheets
for sheet in wb.sheetnames:
    ws = wb[sheet]
    ws.sheet_view.zoomScale = 90 # set zoomSale 90% for all sheets
    max_column = ws.max_column    
    
    # Set the alignment for each cell in row 3
    for column in range(1, max_column + 1):
        cell = ws.cell(row=3, column=column)
        alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
        cell.alignment = alignment

        # Set the column height and width
        ws.row_dimensions[3].height = 95
        ws.column_dimensions[get_column_letter(column)].width = 11
    
    column_widths = {'A': 5,'B': 6,'C': 40}
    for column, width in column_widths.items():
        ws.column_dimensions[column].width = width

    sheet_to_modify = ['Chi Tiet','KV ĐNB+Tây Nguyên','KV Miền Trung','KV Phía Bắc','KV Hồ Chí Minh','KV Tây Nam Bộ']
    columns_to_modify = ['F','BJ','BK','BN','BO','CF','CG','CZ','MM','MN','MO','MQ','MR','MS','NI']
    
    for sheet_name in sheet_to_modify:
        ws = wb[sheet_name]
        for column in columns_to_modify:
            ws.column_dimensions[column].width = 13

# Save the modified workbook
wb.save(new_filename)


# THÊM CÁC CHỮ CHÚ THÍCH PHÍA TRÊN TÊN CỘT

In [132]:
# Xác định font Myriad Pro, in đậm và size 12
font= Font(name='Myriad Pro', bold=True, size=11, color="FF0000")
# Xác định các nội dung tương ứng từng Ô:
cell_data = [
    ('GW2', 'THU THUẦN TỪ DỊCH VỤ'), ('HR2', 'THU KINH DOANH NGOẠI TỆ'), ('II2', 'DOANH SỐ MUA BÁN NGOẠI TỆ(TRIỆU USD)'),
    ('IZ2', 'DOANH SỐ BẢO HIỂM NHÂN THỌ(TỶ ĐỒNG)'), ('JU2', 'DOANH SỐ TTQT'), ('KH2', 'THU HỘ TRÁI PHIẾU'),
    ('KY2', 'SỐ LƯỢNG KHDN LŨY KẾ'), ('LC2', 'QUẢN LÝ TIỀN MẶT LŨY KẾ'), ('LG2', 'SL KHDN CHUỖI'), ('LK2', 'IBK'),
    ('LO2', 'DƯ NỢ TÍN DỤNG CHUỖI'), ('LS2', 'KHÁCH HÀNG TÍN DỤNG (KHDN)'), ('LW2', 'SỐ LƯỢNG KHDN PAYROLL'),
    ('MA2', 'SL TK KHCN NHẬN LƯƠNG PAYROLL'), ('MF2', 'SỐ LƯỢNG KH CHO VAY SME'), ('MH2', 'SỐ LƯỢNG KH D NGHIỆP CIB'),
    ('MM2', 'SỐ LƯỢNG KH CÁ NHÂN'), ('MQ2', 'SỐ LƯỢNG NGƯỜI DÙNG KÊNH SỐ'), ('MU2', 'KH CN BÁN CHÉO CHO KH DOANH NGHIỆP'),
    ('MY2', 'TỶ LỆ GIAO DỊCH QUA KÊNH SỐ'), ('NC2', 'SỐ LƯỢNG ĐIỂM LẺ CHUỖI'), ('NH2', 'SL THẺ GHI NỢ_LŨY KẾ'), 
    ('NL2', 'SL THẺ TÍN DỤNG_LŨY KẾ')
]

# xác định các shet cần thay đổi
sheets_to_modify=['Chi Tiet', 'KV ĐNB+Tây Nguyên', 'KV Miền Trung', 'KV Phía Bắc', 'KV Hồ Chí Minh', 'KV Tây Nam Bộ']

for sheet in sheets_to_modify:
    ws=wb[sheet]
    # xóa các giá trị font cũ trong các ô
    for row in ws.iter_rows(min_row=2, max_row=2):
        for cell in row:
            cell.font = Font()

for sheet in sheets_to_modify:
    ws=wb[sheet]
    # Iterate over the cell data and set the values and font formatting
    for cell, value in cell_data:
        ws[cell].value = value
        ws[cell].font = font

# Save the changes to the Excel file
wb.save(new_filename)

In [133]:
# Xác định font Myriad Pro, in đậm và size 11
font= Font(name='Myriad Pro', bold=True, size=11, color="FF0000")

# select sheet:
ws=wb['CMB']
selected_cols=['S','T']
for col in selected_cols:
    ws.column_dimensions[col].width=13.6

# xóa các giá trị font cũ trong các ô
for row in ws.iter_rows(min_row=2, max_row=2):
    for cell in row:
        cell.font = Font()
cell_values = [
    ('AT2', 'THU THUẦN TỪ DỊCH VỤ CMB'), ('AX2', 'THU KINH DOANH NGOẠI TỆ CMB'), ('BB2', 'DOANH SỐ MUA/BÁN NGOẠI TỆ_TRIỆU USD CMB'),
    ('BF2', 'DOANH SỐ BẢO HIỂM NHÂN THỌ_TỶ ĐỒNG CMB'), ('BJ2', 'DOANH SỐ TTQT CMB'), ('BN2', 'THU HỘ TRÁI PHIẾU CMB'),
    ('BR2', 'SỐ LƯỢNG KH D.NGHIỆP CMB'), ('BV2', 'QUẢN LÝ TIỀN MẶT LŨY KẾ'), ('BZ2', 'SL KHDN CHUỖI'), ('CD2', 'IBK'),
    ('CH2', 'DỰ NỢ TÍN DỤNG CHUỖI'), ('CL2', 'KHÁCH HÀNG TÍN DỤNG (KHDN)'), ('CP2', 'SỐ LƯỢNG KHDN PAYROLL'),
    ('CT2', 'SL TK KHCN NHẬN LƯƠNG PAYROLL'), ('CX2', 'SỐ LƯỢNG KH')
]

for cell, value in cell_values:
    ws[cell].value = value
    ws[cell].font = font

# select sheet:
ws=wb['CIB']
selected_cols=['S','T']
for col in selected_cols:
    ws.column_dimensions[col].width=13.6

# xóa các giá trị font cũ trong các ô
for row in ws.iter_rows(min_row=2, max_row=2):
    for cell in row:
        cell.font = Font()
cell_values = [('AT2', 'THU THUẦN TỪ DỊCH VỤ CIB'), ('AX2', 'THU KINH DOANH NGOẠI TỆ CIB'), 
               ('BB2', 'DOANH SỐ MUA/BÁN NGOẠI TỆ_TRIỆU USD CIB'), ('BF2', 'DOANH SỐ BẢO HIỂM NHÂN THỌ_TỶ ĐỒNG CIB'),
               ('BJ2', 'DOANH SỐ TTQT CIB'), ('BN2', 'THU HỘ TRÁI PHIẾU CIB'), ('BR2', 'SỐ LƯỢNG KH D.NGHIỆP CIB')]

for cell, value in cell_values:
    ws[cell].value = value
    ws[cell].font = font

# select sheet:
ws=wb['THE']
# xóa các giá trị font cũ trong các ô
for row in ws.iter_rows(min_row=2, max_row=2):
    for cell in row:
        cell.font = Font()

cell_values = [('AA2', 'THU THUẦN TỪ DỊCH VỤ THẺ'), ('AE2', 'DOANH SỐ BẢO HIỂM NHÂN THỌ_TỶ ĐỒNG'),
    ('AI2', 'SL THẺ GHI NỢ_LŨY KẾ'), ('AM2', 'SL THẺ TÍN DỤNG_LŨY KẾ')]

for cell, value in cell_values:
    ws[cell].value = value
    ws[cell].font = font

# select sheet:
ws=wb['CN']
selected_cols=['X','Y','Z']
for col in selected_cols:
    ws.column_dimensions[col].width=13.6

# xóa các giá trị font cũ trong các ô
for row in ws.iter_rows(min_row=2, max_row=2):
    for cell in row:
        cell.font = Font()
cell_values = {
    'AY2': 'THU THUẦN TỪ DỊCH VỤ CN', 'BC2': 'THU KINH DOANH NGOẠI TỆ CIB',
    'BG2': 'DOANH SỐ MUA/BÁN NGOẠI TỆ_TRIỆU USD CIB', 'BK2': 'DOANH SỐ BẢO HIỂM NHÂN THỌ_TỶ ĐỒNG CN',
    'BO2': 'THU HỘ TRÁI PHIẾU CIB', 'BS2': 'SỐ LƯỢNG KH CÁ NHÂN', 'BW2': 'SỐ LƯỢNG NGƯỜI DÙNG KÊNH SỐ',
    'CA2': 'KH CN BÁN CHÉO CHO KH DOANH NGHIỆP', 'CE2': 'TỶ LỆ GIAO DỊCH QUA KÊNH SỐ', 'CI2': 'SỐ LƯỢNG ĐIỂM LẺ CHUỖI', 
    'CL1': 'CÁC CHỈ TIÊU KHÁC TTT', 'CL2': 'SL THẺ GHI NỢ_LŨY KẾ', 'CP2': 'SL THẺ TÍN DỤNG_LŨY KẾ'
}

for cell, value in cell_values.items():
    ws[cell].value = value
    ws[cell].font = font


# Save the changes to the Excel file
wb.save(new_filename)

## CẬP NHẬP CHỨNG TỪ CÓ GIÁ CỦA ALM VÀO Ô BK1 VÀ Ô E1

In [134]:
# tìm các cột có SBV Code chứa số 43 ở đầu trong dữ liệu
df=gl101[gl101['SBV_CODE'].astype(str).str.startswith('43')]
# Tổng cột CR_CLOS trừ cho tổng cột DR_CLOS và quy đổi thành đơn vị tỷ đồng
a=(df.CR_CLOS.sum()-df.DR_CLOS.sum())/billion

# lưu file vào biến alm
# đọc dữ liệu file "43 44"
df=pd.read_excel(info_filename,sheet_name='TK 43 44')
# tìm các cột có SBV Code chứa số 243 ở đầu trong dữ liệu
df=df[df['GL_CODE'].astype(str).str.startswith('243')]
# cột CN bỏ "001"
df=df[df.CN!=1]
# Tính tổng cột SODU_QD
df.SODU_QD.sum()
# Quy đổi thành tỷ đồng
b=df.SODU_QD.sum()/billion

# Số liệu vốn Ủy thác tổng 3 cột Ủy thác của 3 khối trong file Ủy thác
c=UT.UTSME.sum()+UT.UTCN.sum()+UT.UTCIB.sum()
# Tổng số Chứng từ có giá:
CTCG=a-b-c

# Xác định font in đậm
bold_font= Font(bold=True)
# Set the font size of cell A1 to 18
font = Font(name='Myriad Pro', size=18, bold=True)

# Loop through all sheets in the workbook
for sheet_name in wb.sheetnames:
    sheet = wb[sheet_name]
    sheet['A1'] = f'BÁO CÁO NGÀY {date_str}'
    sheet['A1'].font=bold_font
    sheet['A1'].font = font

# chọn sheet chi tiet để thêm các dữ liệu
ws=wb['Chi Tiet']
ws['G1'].value = 'GTCG của ALM'
ws['F1'].value = CTCG
ws['F1'].number_format='#,##0.0'

ws['E1'].value = CTCG
ws['E1'].number_format='#,##0.0'

ws['BL1'].value = 'TPDN phòng đầu tư'
ws['BK1'].value = BK1
ws['BK1'].number_format='#,##0.0'

# ô BK2 (dư nợ cho vay của Muadee)
ws['BL2'].value = 'Dư nợ cho vay của Muadee'
ws['BL2'].font=Font(name='Myriad Pro', size=10)
ws['BK2'].value = BK2
ws['BK2'].font=Font(name='Myriad Pro', size=10)
ws['BK2'].number_format='#,##0.0'
# In đậm ô A1 này trong file excel
ws['A1'].font=bold_font
# Set size 18 trong ô A1
ws['A1'].font = font

# Save the workbook
wb.save(new_filename)

# IN ĐẬM CỤM CN VÀ PGD ĐẶC THÙ CHO TẤT CẢ CÁC SHEET

In [135]:
# danh sách các đơn vị và cụm đơn vị: 
df=pd.read_excel(info_filename)
df[df.CUM.notnull()].CUM
# các string cần tìm trong list 
search_strings=df[df.CUM.notnull()].CUM.drop_duplicates().tolist()

In [136]:
# Define a list of search strings
search_strings = df[df.CUM.notnull()].CUM.drop_duplicates().tolist()

# Open the Excel file
wb = openpyxl.load_workbook(new_filename)

# Loop through each sheet in the workbook
for sheet in wb.sheetnames:
    # Select the current sheet
    ws = wb[sheet]

    # Create a Font object with bold=True
    bold_font = Font(bold=True)

    # Create an empty list to store the row numbers
    matching_rows = []

    # Iterate through the rows in the worksheet
    for row_number, row in enumerate(ws.iter_rows(min_row=2, min_col=2, values_only=True), start=2):
        # Check if any of the search strings appear in the current row
        if any(string in str(cell) for string in search_strings for cell in row):
            # If a match is found, append the row number to the list of matching rows
            matching_rows.append(row_number)

    # Apply the bold font to the matching rows
    for row_number in matching_rows:
        for cell in ws[row_number]:
            cell.font = bold_font

# Save the changes to the Excel file
wb.save(new_filename)


# TÔ MÀU CHO BÁO CÁO

In [137]:
# TÔ MÀU CHO CÁC Ô DÒNG THỨ 3 PHÂN LOẠI SME, CIB, CN, THE
from openpyxl.utils import get_column_letter
from openpyxl.styles import PatternFill
# mở file excel Báo Cáo ngày
wb=load_workbook(new_filename)

# Define colors
CMB_color = 'f9be8b'
CIB_color = '92cddc'
CN_color = 'b1a0c7'
Th_color = 'bfbfbf'

# Iterate over all sheets
for sheet in wb:
    # Define partial strings to search for
    partial_strings = {'CMB': CMB_color, 'CIB': CIB_color, 'CN': CN_color, 'Thẻ': Th_color}
    
    # Search for the partial strings in the 5th row of the sheet
    for cell in sheet[3]:
        if cell.value is not None:
            cell_value_lower = str(cell.value).lower()
            for partial_string, color in partial_strings.items():
                if partial_string.lower() in cell_value_lower:
                    col_letter = get_column_letter(cell.column)
                    cell_range = f'{col_letter}3'
                    sheet[cell_range].fill = PatternFill(start_color=color, end_color=color, fill_type='solid')

# Save the modified workbook
wb.save(new_filename)

In [138]:
# TÔ MÀU CHO CÁC Ô TỪ DÒNG 4 TỚI MAX ROW
import openpyxl
from openpyxl.styles import PatternFill
from openpyxl.utils import get_column_letter

# Load the Excel file
wb=load_workbook(new_filename)

# Define colors
CMB_color = 'FDE9D9'
CIB_color = 'DAEEF3'
CN_color = 'E4DFEC'
Th_color = 'F2F2F2'

# Iterate over all sheets
for sheet in wb:# Save the modified workbook
    # Define partial strings to search for in row 3
    partial_strings = {'CMB': CMB_color, 'CIB': CIB_color, 'CN': CN_color, 'Thẻ': Th_color}
    
    # Search for the partial strings in row 3 of the sheet
    for cell in sheet[3]:
        if cell.value is not None:
            cell_value_lower = str(cell.value).lower()
            for partial_string, color in partial_strings.items():
                if partial_string.lower() in cell_value_lower:
                    col_letter = get_column_letter(cell.column)
                    for row in sheet.iter_rows(min_row=4, max_col=sheet.max_column, max_row=sheet.max_row):
                        cell_range = f'{col_letter}{row[0].row}'
                        sheet[cell_range].fill = PatternFill(start_color=color, end_color=color, fill_type='solid')

# Save the modified workbook
wb.save(new_filename)

In [139]:
# TÔ MÀU CHO CÁC Ô TIÊU ĐỀ
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

# Define the fill colors and columns to fill
fill_color_dict = {
    'ffff99': ['F','H','BJ','BK','BM'],
    'ffffdd': ['FY','FZ','GA','GB','GC','GD','GE'],
    'e6b8b7': ['EN','EO','EP','EQ','ER'],
    'b8cce4': ['GW','GX','GY','GZ'],
    'd8e4bc': ['HR','HS','HT','HU','IZ','JA','JB','JC'],
    'd9d9d9': ['II','IJ','IK','IL','MM','MN', 'MO','MP'],
    'fcd5b4': ['JU','JV','JW','JX'],
    'b7dee8': ['KH','KI','KJ','KK'],
    'fde9d9': ['KY','KZ','LA','LB','LW','LX','LY','LZ'],
    'f2f2f2': ['LC','LD','LE','LF'],
    'dce6f1': ['LG','LH','LI','LJ','MH','MI','MJ','MK'],
    'daeef3': ['LK','LL','LM','LN','MU','MV','MW','MX'],
    'f2dcdb': ['LO','LP','LQ','LR'],
    'ddd9c4': ['LS','LT','LU','LV','NC','ND','NE','NF'],
    'e4dfec': ['MF','NH','NI','NJ','NK','MY','MZ','NA','NB'],
    'c5d9f1': ['MQ','MR','MS','MT']
}

sheets_to_modify=[]
for sheet in wb.sheetnames:
    if sheet not in ['CMB','CIB','THE','CN']:
        sheets_to_modify.append(sheet)

# Loop through each sheet in the workbook
for sheet in sheets_to_modify:
    ws=wb[sheet]
    # Loop through each fill color and column combination
    for fill_color, columns in fill_color_dict.items():
        # Create a PatternFill object with the current fill color
        fill = PatternFill(start_color=fill_color, end_color=fill_color, fill_type='solid')
        # Loop through each column in the current combination
        for column in columns:
            # Get the cell in row 3 of the current column
            cell = ws[column + '3']
            # Set the fill color of the cell
            cell.fill = fill

# Save the modified workbook
wb.save(new_filename)

In [140]:
# # TÔ MÀU CHO CÁC ĐƠN VỊ LỚN TRONG KHU VỰC
from openpyxl.utils import get_column_letter
from openpyxl.styles import PatternFill

# Read the dataframe
df=pd.read_excel(info_filename)
df=df[df['KV TYPE 2'].notnull()][['KV TYPE 2','CUM']]

# Define the fill colors
KV1_fill = PatternFill(start_color='e6b8b7', end_color='e6b8b7', fill_type='solid')
KV2_fill = PatternFill(start_color='c4d79b', end_color='c4d79b', fill_type='solid')
KV3_fill = PatternFill(start_color='ffff99', end_color='ffff99', fill_type='solid')
KV4_fill = PatternFill(start_color='ccc0da', end_color='ccc0da', fill_type='solid')
KV5_fill = PatternFill(start_color='b7dee8', end_color='b7dee8', fill_type='solid')

# Create a dictionary to store the list of strings to find for each KV TYPE 2 value
dfs={}
for i in df['KV TYPE 2'].unique():
    dfs[i]=df[df['KV TYPE 2']==i]['CUM'].tolist()

# Iterate through all sheets of the Excel file
for sheet in wb.worksheets:
    # Iterate through all rows of column C
    for row in range(1, sheet.max_row + 1):
        # Iterate through all KV TYPE 2 values
        for kv_type, strings_to_find in dfs.items():
            # Check if the cell value in column C is in the list of strings to find for this KV TYPE 2 value
            if sheet.cell(row=row, column=2).value in strings_to_find:
                # Set the fill color based on the KV TYPE 2 value
                if kv_type == 'KV1':
                    fill_color = KV1_fill
                elif kv_type == 'KV2':
                    fill_color = KV2_fill
                elif kv_type == 'KV3':
                    fill_color = KV3_fill
                elif kv_type == 'KV4':
                    fill_color = KV4_fill
                elif kv_type == 'KV5':
                    fill_color = KV5_fill
                # Set the fill color for the cell
                col_letter = get_column_letter(3)
                cell_range = f'{col_letter}{row}'
                sheet[cell_range].fill = fill_color

# Save the changes to the Excel file
wb.save(new_filename)

In [141]:
# TÔ MÀU VÀNG CHO DÒNG "TOÀN HÀNG" CHO TẤT CẢ CÁC SHEET
from openpyxl.styles import PatternFill

# Define the color
color = 'FFFFCC'
# Define the row number
row_num = 4
# Define the Bold font
font = Font(bold=True)

# Loop through each sheet name
for sheet_name in ["Chi Tiet","CMB","CIB","CN","THE"]:

    # Select the sheet
    sheet = wb[sheet_name]

    # Get the maximum column in the sheet
    max_col = sheet.max_column

    # Create a PatternFill object with the specified color
    fill = PatternFill(start_color=color, end_color=color, fill_type='solid')

    # Loop through each column in the row and set the fill color and font style
    for col in range(1, max_col+1):
        cell = sheet.cell(row=row_num, column=col)
        cell.fill = fill
        cell.font = font

# Save the changes to the Excel file
wb.save(new_filename)

In [142]:
# TÔ MÀU VÀNG CHO DÒNG KHU VỰC CHO TỪNG SHEET
# Select the sheet
sheet = wb['KV Hồ Chí Minh']
# Define the row number
row_num = 4
# Get the maximum column in the sheet
max_col = sheet.max_column

# Create a PatternFill object with the specified color
fill = PatternFill(start_color=color, end_color=color, fill_type='solid')

# Loop through each column in the row and set the fill color and font style
for col in range(1, max_col+1):
    cell = sheet.cell(row=row_num, column=col)
    cell.fill = fill
    cell.font = font

# Select the sheet
sheet = wb['KV Tây Nam Bộ']
# Define the row number
row_num = 4
# Get the maximum column in the sheet
max_col = sheet.max_column

# Create a PatternFill object with the specified color
fill = PatternFill(start_color=color, end_color=color, fill_type='solid')

# Loop through each column in the row and set the fill color and font style
for col in range(1, max_col+1):
    cell = sheet.cell(row=row_num, column=col)
    cell.fill = fill
    cell.font = font

# Select the sheet
sheet = wb['KV ĐNB+Tây Nguyên']
# Define the row number
row_nums = [4,5,6]
# Get the maximum column in the sheet
max_col = sheet.max_column

# Create a PatternFill object with the specified color
fill = PatternFill(start_color=color, end_color=color, fill_type='solid')

# Loop through each column in the row and set the fill color and font style
for row_num in row_nums:
    for col in range(1, max_col+1):
        cell = sheet.cell(row=row_num, column=col)
        cell.fill = fill
        cell.font = font

# Select the sheet
sheet = wb['KV Miền Trung']
# Define the row number
row_nums = [4,5]
# Get the maximum column in the sheet
max_col = sheet.max_column

# Create a PatternFill object with the specified color
fill = PatternFill(start_color=color, end_color=color, fill_type='solid')

# Loop through each column in the row and set the fill color and font style
for row_num in row_nums:
    for col in range(1, max_col+1):
        cell = sheet.cell(row=row_num, column=col)
        cell.fill = fill
        cell.font = font
        
# Select the sheet
sheet = wb['KV Phía Bắc']
# Define the row number
row_nums = [4,5,6]
# Get the maximum column in the sheet
max_col = sheet.max_column

# Create a PatternFill object with the specified color
fill = PatternFill(start_color=color, end_color=color, fill_type='solid')

# Loop through each column in the row and set the fill color and font style
for row_num in row_nums:
    for col in range(1, max_col+1):
        cell = sheet.cell(row=row_num, column=col)
        cell.fill = fill
        cell.font = font

# Save the changes to the Excel file
wb.save(new_filename)

In [143]:
 # tô màu trắng cho các ô B4, B18, B19, and B20
sheets=['Chi Tiet','CMB','CIB','CN','THE']
for sheet_name in sheets:
    ws=wb[sheet_name]
    for cell in ['B4','B18', 'B19', 'B20']:
        ws[cell].font = Font(color="FFFFFF")

# Save the workbook
wb.save(new_filename)

# TẠO CÁC NÚT GROUPING CHO FILE EXCEL

In [144]:
from openpyxl.utils import get_column_letter, column_index_from_string

sheets_to_modify=['Chi Tiet', 'KV ĐNB+Tây Nguyên', 'KV Miền Trung', 'KV Phía Bắc', 'KV Hồ Chí Minh', 'KV Tây Nam Bộ']
# Define the column ranges to group
column_ranges = [("D", "E"),("K", "U"),("Z", "AJ"),("AM", "AN"),("AP", "BA"),("BH", "BI"),("BQ", "CC"),("CI", "CU"),
          ("CX", "CY"),("DB", "DM"),("DR", "EB"),("EE", "EM"),("ES", "FX"),("GR", "GU"),("HA", "HP"),("HV", "IG"),
          ("IM", "IX"),("JD", "JS"),("JY", "KF"),("KL", "KW")]
# Loop through all sheets in the workbook
for sheet_name in sheets_to_modify:
    sheet = wb[sheet_name]
    
    # Loop through the column ranges to group
    for start_column, end_column in column_ranges:
        start_column_index = column_index_from_string(start_column)
        end_column_index = column_index_from_string(end_column)
                
        # Group the columns
        sheet.column_dimensions.group(get_column_letter(start_column_index), get_column_letter(end_column_index), hidden=True)

# Save the changes to the Excel file
wb.save(new_filename)


In [145]:
# CHỌN SHEET CN
ws=wb['CN']
ws.column_dimensions.group(start='D', end='E', hidden=True)
ws.column_dimensions.group(start='G', end='Q', hidden=True)
ws.column_dimensions.group(start='AB', end='AM', hidden=True)
ws.column_dimensions.group(start='AP', end='AV', hidden=True)
ws.column_dimensions['F'].width = 11
# CHỌN SHEET CIB
ws=wb['CIB']
ws.column_dimensions.group(start='F', end='P', hidden=True)
ws.column_dimensions.group(start='V', end='AH', hidden=True)
ws.column_dimensions.group(start='AK', end='AQ', hidden=True)
# CHỌN SHEET CMB
ws=wb['CMB']
ws.column_dimensions.group(start='F', end='P', hidden=True)
ws.column_dimensions.group(start='V', end='AH', hidden=True)
ws.column_dimensions.group(start='AK', end='AQ', hidden=True)
ws=wb['THE']
ws.column_dimensions.group(start='F', end='P', hidden=True)
ws.column_dimensions.group(start='S', end='X', hidden=True)
# Save the changes to the Excel file
wb.save(new_filename)

In [146]:
# TẠO CÁC NÚT GROUPING CHO CÁC DÒNG 5-11 và 21-23
sheets_to_modify=['Chi Tiet', 'CMB', 'CIB', 'CN', 'THE']

# Loop through all sheets
for sheet_name in sheets_to_modify:
    # Select the sheet
    ws = wb[sheet_name]

    # Add grouping buttons to rows 5-11 and 21-23
    ws.row_dimensions.group(5, 11, hidden=True)
    ws.row_dimensions.group(21, 25, hidden=True)

ws=wb['KV Phía Bắc']
ws.row_dimensions.group(4, 5, hidden=True)

ws=wb['KV ĐNB+Tây Nguyên']
ws.row_dimensions.group(4, 5, hidden=True)
ws.row_dimensions.group(7, 9, hidden=True)

ws=wb['KV Miền Trung']
ws.row_dimensions.group(4, 4, hidden=True)

ws=wb['KV Hồ Chí Minh']
ws.row_dimensions.group(5, 6, hidden=True)
# Save the changes to the Excel file
wb.save(new_filename)

# TẠO DẤU PHÂN CÁCH HÀNG NGÀN CHO CÁC SHEET DỮ LIỆU

In [147]:
# Loop through each sheet in the workbook
for sheet in wb.sheetnames:
    # Select the current sheet
    ws = wb[sheet]

    # Find the max column number
    max_col = ws.max_column

    # Set the decimal comma for columns 4 to max_col
    for col in range(4, max_col + 1):
        col_letter = openpyxl.utils.get_column_letter(col)
        for row in range(1, ws.max_row + 1):
            cell = ws[f"{col_letter}{row}"]
            if cell.data_type == "n":
                cell.number_format = "#,##0.0"

# TẠO FREEZE CHO DÒNG VÀ CỘT
# Iterate through each worksheet in the workbook
for sheet in wb.worksheets:
    # Set the freeze panes to freeze the first three columns (A, B, and C) and the fifth row
    sheet.freeze_panes = 'D4'

# Save the changes to the Excel file
wb.save(new_filename)

# TẠO FORMAT PHẦN TRĂM CHO CÁC CỘT 

In [148]:
from openpyxl.styles import numbers

# LẬP DANH SÁCH CÁC CỘT TRONG FILE
column_names = ChiTiet_df.columns.tolist()
# lọc ra các cột có dấu phần trăm ngoại trừ cột 30% UPAS LC
columns_with_percent = [col_name for col_name in column_names if "%" in col_name and "UPAS" not in col_name]

# lập danh sách tên các cột muốn set format phần trăm
columns_to_format = columns_with_percent

# lặp tương tự cho các sheet trong file excel
for ws in wb:
    # lặp tương tự cho các ô ở dòng thứ 3 của 1 sheet
    for cell in ws[3]:
        # kiểm tra xem ô này có chứa tên cột cần set %
        for column_name in columns_to_format:
            if cell.value is not None and column_name in cell.value:
                # set format % cho toàn bộ cột
                column_letter = cell.column_letter
                for col_cell in ws[column_letter]:
                    col_cell.number_format = numbers.FORMAT_PERCENTAGE
                break   # no need to check other column names once a match is found

# TẮT CÁC Ô KẺ TRONG FILE BÁO CÁO
# Loop through all sheets in the workbook
for sheet in wb.worksheets:
    # Set the show_gridlines attribute to False
    sheet.sheet_view.showGridLines = False

# Save the changes to the Excel file
wb.save(new_filename)

# TẠO BORDER CHO CÁC Ô VÀ CÁC CỘT

In [149]:
# TẠO CÁC ĐƯỜNG KẺ TRÁI PHẢI CHO BÁO CÁO
from openpyxl.styles import Border, Side

for sheet in wb.sheetnames:
    # select the current sheet
    ws = wb[sheet]
    
    # get the last row number
    last_row = ws.max_row
    
    # create the border style
    border = Border(
        left=Side(border_style='thin', color='000000'),
        right=Side(border_style='thin', color='000000'),
        top=Side(border_style=None),
        bottom=Side(border_style=None)
    )
    
    # apply the border to each cell in the range
    for row in range(3, last_row + 1):
        for col in range(1, ws.max_column + 1):
            ws.cell(row=row, column=col).border = border

# TẠO ĐƯỜNG KẺ BÁO CÁO CHO DÒNG THỨ 3
# set the border style
border_style = Border(
    left=Side(border_style='thin', color='000000'),
    right=Side(border_style='thin', color='000000'),
    top=Side(border_style='thin', color='000000'),
    bottom=Side(border_style='thin', color='000000')
)

# loop through all sheets
for sheet_name in wb.sheetnames:
    # select the sheet
    worksheet = wb[sheet_name]
    # loop through all active cells in row 3
    for cell in worksheet[3]:
        # set the border style for each cell
        cell.border = border_style

# TẠO ĐƯỜNG KẺ BÁO CÁO CHO DÒNG THỨ 3
for sheet in wb.sheetnames:
    # select the current sheet
    ws = wb[sheet]
    
    # get the last row number
    last_row = ws.max_row
    
    # create the border style
    border = Border(
        left=Side(border_style='thin', color='000000'),
        right=Side(border_style='thin', color='000000'),
        top=Side(border_style=None),
        bottom=Side(border_style=None)
    )
    
    # select the sheet
    ws = wb[sheet_name]
    # loop through all active cells in row 3
    for cell in ws[3]:
        # set the border style for each cell
        cell.border = border_style

# Save the changes to the Excel file
wb.save(new_filename)

# LƯU FILE RA THÀNH NHIỀU FILE TƯƠNG ỨNG TỪNG SHEET:

from openpyxl import load_workbook

  ## Load the Excel file
excel_file = load_workbook(new_filename)

sheets_to_save=['CMB','CIB','CN','THE','KV ĐNB+Tây Nguyên','KV Miền Trung','KV Phía Bắc','KV Hồ Chí Minh','KV Tây Nam Bộ']
 ## Loop through each sheet in the file
for sheet_name in sheets_to_save:
    # Load the sheet into a new workbook:
    new_workbook = load_workbook(new_filename)
    new_sheet = new_workbook[sheet_name]
    
    # Remove all other sheets from the new workbook:
    for sheet in new_workbook.sheetnames:
        if sheet != sheet_name:
            new_workbook.remove(new_workbook[sheet])
    # Define the sheet and file name:
    sheet_mapping = {
    'CMB': 'Khối KHDN',
    'CIB': 'Khối KHDNL',
    'CN': 'Khối KHCN',
    'THE': 'Khối TTT',
    'KV ĐNB+Tây Nguyên': 'KV DNB+TN',
    'KV Miền Trung': 'KV MT',
    'KV Phía Bắc': 'KV HN+PB',
    'KV Hồ Chí Minh': 'KV HCM',
    'KV Tây Nam Bộ': 'KV TNB'
}
    # Define the new file name for each files
    new_new_filename = f'Báo cáo ngày {sheet_mapping[sheet_name]} {date_str}.xlsx'

    # Save the sheet to a new Excel file:
    new_workbook.save(new_new_filename)



In [150]:
def send_email(to_email_list, cc_email_list, department_name):
    print(f'---{department_name}---')
    print(f'Đến: {to_email_list}')
    print(f'Cc: {cc_email_list}')
    print(f'Tiêu đề: Báo cáo ngày {date_str} - {department_name}')
    print('')
    print(f"""Nội dung: Kính gửi Anh/Chị,​
    
    Phòng Kế hoạch, Báo cáo và Phân tích Tài chính kính gửi Anh/Chị: Báo cáo ngày {date_str} như tài liệu đính kèm.
    
    Trân trọng!""")
    print('------------------------------------------------------------------------------------------')
    print('')

# BAN TGD
to_email_list = 'thanh.pq@hdbank.com.vn'
cc_email_list = 'daupv@hdbank.com.vn; ngocdh@hdbank.com.vn; minhdnt@hdbank.com.vn; BCQT@hdbank.com.vn'
send_email(to_email_list, cc_email_list, '')


------
Đến: thanh.pq@hdbank.com.vn
Cc: daupv@hdbank.com.vn; ngocdh@hdbank.com.vn; minhdnt@hdbank.com.vn; BCQT@hdbank.com.vn
Tiêu đề: Báo cáo ngày 09.06.2023 - 

Nội dung: Kính gửi Anh/Chị,​
    
    Phòng Kế hoạch, Báo cáo và Phân tích Tài chính kính gửi Anh/Chị: Báo cáo ngày 09.06.2023 như tài liệu đính kèm.
    
    Trân trọng!
------------------------------------------------------------------------------------------



In [151]:
# KV HN+PB
to_email_list = 'hieunm2@hdbank.com.vn;nhungntt@hdbank.com.vn'
cc_email_list = 'phuongptn@hdbank.com.vn;danhnd@hdbank.com.vn;trangntx@hdbank.com.vn;duyenntt5@hdbank.com.vn;namnh25@hdbank.com.vn'
send_email(to_email_list, cc_email_list, 'KV HN+PB')

# KV MT
to_email_list = 'hoatt@hdbank.com.vn'
cc_email_list = 'nhity@hdbank.com.vn; danhnd@hdbank.com.vn; phuongptn@hdbank.com.vn; trangntx@hdbank.com.vn; duyenntt5@hdbank.com.vn'
send_email(to_email_list, cc_email_list, 'KV MT')

# KV DNB+TN
to_email_list = 'thoa_ltk@hdbank.com.vn'
cc_email_list = 'danhnd@hdbank.com.vn; phuongptn@hdbank.com.vn; trangntx@hdbank.com.vn; duyenntt5@hdbank.com.vn'
send_email(to_email_list, cc_email_list, 'KV DNB+TN')

# KV TNB
to_email_list = 'ninhtd@hdbank.com.vn'
cc_email_list = 'danhnd@hdbank.com.vn; phuongptn@hdbank.com.vn; trangntx@hdbank.com.vn; duyenntt5@hdbank.com.vn'
send_email(to_email_list, cc_email_list, 'KV TNB')

# KV HCM
to_email_list='thudn@hdbank.com.vn; huongtv@hdbank.com.vn'
cc_email_list='huy.vt@hdbank.com.vn; quanghien@hdbank.com.vn; danhnd@hdbank.com.vn; phuongptn@hdbank.com.vn; trangntx@hdbank.com.vn; duyenntt5@hdbank.com.vn'
send_email(to_email_list, cc_email_list, 'KV HCM')


---KV HN+PB---
Đến: hieunm2@hdbank.com.vn;nhungntt@hdbank.com.vn
Cc: phuongptn@hdbank.com.vn;danhnd@hdbank.com.vn;trangntx@hdbank.com.vn;duyenntt5@hdbank.com.vn;namnh25@hdbank.com.vn
Tiêu đề: Báo cáo ngày 09.06.2023 - KV HN+PB

Nội dung: Kính gửi Anh/Chị,​
    
    Phòng Kế hoạch, Báo cáo và Phân tích Tài chính kính gửi Anh/Chị: Báo cáo ngày 09.06.2023 như tài liệu đính kèm.
    
    Trân trọng!
------------------------------------------------------------------------------------------

---KV MT---
Đến: hoatt@hdbank.com.vn
Cc: nhity@hdbank.com.vn; danhnd@hdbank.com.vn; phuongptn@hdbank.com.vn; trangntx@hdbank.com.vn; duyenntt5@hdbank.com.vn
Tiêu đề: Báo cáo ngày 09.06.2023 - KV MT

Nội dung: Kính gửi Anh/Chị,​
    
    Phòng Kế hoạch, Báo cáo và Phân tích Tài chính kính gửi Anh/Chị: Báo cáo ngày 09.06.2023 như tài liệu đính kèm.
    
    Trân trọng!
------------------------------------------------------------------------------------------

---KV DNB+TN---
Đến: thoa_ltk@hdbank.com.vn
Cc:

In [152]:
# Khối KHDNL
to_email_list='anhnh@hdbank.com.vn; linh_tt@hdbank.com.vn'
cc_email_list='phuongntt6@hdbank.com.vn; cibptkd&ht@hdbank.com.vn; danhnd@hdbank.com.vn; phuongptn@hdbank.com.vn; trangntx@hdbank.com.vn; duyenntt5@hdbank.com.vn'
send_email(to_email_list, cc_email_list, 'Khối KHDNL')

# Khối KHDN
to_email_list='dat_pt@hdbank.com.vn;trinh.ntt@hdbank.com.vn;toanvd@hdbank.com.vn;myld@hdbank.com.vn;lanttt1@hdbank.com.vn;anhlth9@hdbank.com.vn;thientdn1@hdbank.com.vn;dieppt1@hdbank.com.vn;giangth7@hdbank.com.vn;vypt5@hdbank.com.vn;ngohaiha@hdbank.com.vn;tanph5@hdbank.com.vn;thaohtt1@hdbank.com.vn;huanpc@hdbank.com.vn;hunglt7@hdbank.com.vn;khanhhq3@hdbank.com.vn;hoangnl6@hdbank.com.vn;oanhttn@hdbank.com.vn;thyntp2@hdbank.com.vn;tuttc2@hdbank.com.vn;maivs3@hdbank.com.vn;vinhnh1@hdbank.com.vn;anhtn4@hdbank.com.vn;oanhnn@hdbank.com.vn;phuonggt1@hdbank.com.vn;huypx3@hdbank.com.vn;hangntt28@hdbank.com.vn;trongnb1@hdbank.com.vn;duongttt3@hdbank.com.vn;thoaltk3@hdbank.com.vn;toanvd@hdbank.com.vn;bichvtn1@hdbank.com.vn;nhatnv@hdbank.com.vn;huongptt4@hdbank.com.vn;chidt2@hdbank.com.vn;hungnc@hdbank.com.vn;tutm@hdbank.com.vn;baolc1@hdbank.com.vn;haitn1@hdbank.com.vn;lynth1@hdbank.com.vn;trangnh3@hdbank.com.vn;mungnt@hdbank.com.vn;'
cc_email_list='danhnd@hdbank.com.vn; phuongptn@hdbank.com.vn; trangntx@hdbank.com.vn; duyenntt5@hdbank.com.vn'
send_email(to_email_list, cc_email_list, 'Khối KHDN')

# Khối KHCN
to_email_list='huongntt4@hdbank.com.vn'
cc_email_list='ssp@hdbank.com.vn; danhnd@hdbank.com.vn; phuongptn@hdbank.com.vn; trangntx@hdbank.com.vn; duyenntt5@hdbank.com.vn'
send_email(to_email_list, cc_email_list, 'Khối KHCN')

# TTT
to_email_list='thanhpm@hdbank.com.vn; ketoanthe@hdbank.com.vn; tienutt1@hdbank.com.vn; nhuntq5@hdbank.com.vn; thaidt@hdbank.com.vn'
cc_email_list='tung_nt@hdbank.com.vn; vudv1@hdbank.com.vn; lyltt3@hdbank.com.vn; danhnd@hdbank.com.vn; phuongptn@hdbank.com.vn; trangntx@hdbank.com.vn; duyenntt5@hdbank.com.vn'
send_email(to_email_list, cc_email_list, 'TTT')


---Khối KHDNL---
Đến: anhnh@hdbank.com.vn; linh_tt@hdbank.com.vn
Cc: phuongntt6@hdbank.com.vn; cibptkd&ht@hdbank.com.vn; danhnd@hdbank.com.vn; phuongptn@hdbank.com.vn; trangntx@hdbank.com.vn; duyenntt5@hdbank.com.vn
Tiêu đề: Báo cáo ngày 09.06.2023 - Khối KHDNL

Nội dung: Kính gửi Anh/Chị,​
    
    Phòng Kế hoạch, Báo cáo và Phân tích Tài chính kính gửi Anh/Chị: Báo cáo ngày 09.06.2023 như tài liệu đính kèm.
    
    Trân trọng!
------------------------------------------------------------------------------------------

---Khối KHDN---
Đến: dat_pt@hdbank.com.vn;trinh.ntt@hdbank.com.vn;toanvd@hdbank.com.vn;myld@hdbank.com.vn;lanttt1@hdbank.com.vn;anhlth9@hdbank.com.vn;thientdn1@hdbank.com.vn;dieppt1@hdbank.com.vn;giangth7@hdbank.com.vn;vypt5@hdbank.com.vn;ngohaiha@hdbank.com.vn;tanph5@hdbank.com.vn;thaohtt1@hdbank.com.vn;huanpc@hdbank.com.vn;hunglt7@hdbank.com.vn;khanhhq3@hdbank.com.vn;hoangnl6@hdbank.com.vn;oanhttn@hdbank.com.vn;thyntp2@hdbank.com.vn;tuttc2@hdbank.com.vn;maivs3@hdbank.

In [153]:
print('Your report is ready now')

Your report is ready now
